## 步驟0：資料預處理與重採樣

In [1]:
from typing import List, Dict, Any, Optional, Tuple

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 自己实现 get_dollar_bars
# =====================================================
def get_dollar_bars_custom(df, threshold=10_000_000):
    """
    自己实现的 dollar bar 计算函数
    
    Parameters:
    -----------
    df : pd.DataFrame
        输入数据，必须包含列: date_time, price, volume
    threshold : float
        Dollar bar 的阈值（累积美元价值）
        
    Returns:
    --------
    pd.DataFrame
        Dollar bars，包含列: date_time, open, high, low, close, volume, 
        cum_buy_volume, cum_ticks, cum_dollar_value
    """
    # 初始化变量
    list_bars = []
    
    # 当前 bar 的状态
    open_price = None
    high_price = -np.inf
    low_price = np.inf
    close_price = None
    
    # 累积统计
    cum_ticks = 0
    cum_dollar_value = 0
    cum_volume = 0
    cum_buy_volume = 0
    
    # Tick rule 相关（用于判断买卖方向）
    prev_price = None
    prev_tick_rule = 0
    tick_num = 0
    
    # 遍历每一行数据
    for idx, row in df.iterrows():
        date_time = row['date_time']
        price = float(row['price'])
        volume = float(row['volume'])
        
        # 跳过无效数据
        if pd.isna(price) or pd.isna(volume) or volume <= 0:
            continue
        
        tick_num += 1
        dollar_value = price * volume
        
        # 初始化 open_price
        if open_price is None:
            open_price = price
        
        # 更新 high 和 low
        high_price = max(high_price, price)
        low_price = min(low_price, price)
        
        # 应用 tick rule 判断买卖方向
        if prev_price is not None:
            tick_diff = price - prev_price
            if tick_diff != 0:
                signed_tick = np.sign(tick_diff)
                prev_tick_rule = signed_tick
            else:
                signed_tick = prev_tick_rule
        else:
            signed_tick = 0
        
        prev_price = price
        
        # 累积统计
        cum_ticks += 1
        cum_dollar_value += dollar_value
        cum_volume += volume
        if signed_tick == 1:  # 上涨，视为买入
            cum_buy_volume += volume
        
        # 检查是否达到阈值
        if cum_dollar_value >= threshold:
            # 确保 high 和 low 包含 open
            high_price = max(high_price, open_price)
            low_price = min(low_price, open_price)
            close_price = price
            
            # 创建 bar
            list_bars.append([
                date_time,
                tick_num,
                open_price,
                high_price,
                low_price,
                close_price,
                cum_volume,
                cum_buy_volume,
                cum_ticks,
                cum_dollar_value
            ])
            
            # 重置状态
            open_price = None
            high_price = -np.inf
            low_price = np.inf
            cum_ticks = 0
            cum_dollar_value = 0
            cum_volume = 0
            cum_buy_volume = 0
    
    # 转换为 DataFrame
    if list_bars:
        bars_df = pd.DataFrame(
            list_bars,
            columns=['date_time', 'tick_num', 'open', 'high', 'low', 'close', 
                     'volume', 'cum_buy_volume', 'cum_ticks', 'cum_dollar_value']
        )
        bars_df['date_time'] = pd.to_datetime(bars_df['date_time'])
        bars_df.set_index('date_time', inplace=True)
        return bars_df
    else:
        return pd.DataFrame()

# =====================================================
# 1. 读取分钟数据
# =====================================================
print("=" * 60)
print("📊 步骤1: 读取分钟数据")
print("=" * 60)

# 读取分钟数据
minute_df = pd.read_csv('minute_prices/NAS100.csv')
minute_df['Date'] = pd.to_datetime(minute_df['Date'])
minute_df.set_index('Date', inplace=True)

print(f"分钟数据总笔数: {len(minute_df):,}")
print(f"数据期间: {minute_df.index.min()} 至 {minute_df.index.max()}")

# 准备 dollar bar 输入格式: [date_time, price, volume]
dollar_bar_input = pd.DataFrame({
    'date_time': minute_df.index,
    'price': minute_df['BidClose'],
    'volume': minute_df['Volume']
})

# 移除无效数据
dollar_bar_input = dollar_bar_input[
    (dollar_bar_input['volume'] > 0) & 
    (dollar_bar_input['volume'].notna()) &
    (dollar_bar_input['price'].notna())
].copy()

print(f"有效数据笔数: {len(dollar_bar_input):,}")

# =====================================================
# 2. 计算 Dollar Bar (每1000万一根)
# =====================================================
print("\n" + "=" * 60)
print("📊 步骤2: 计算 Dollar Bar (threshold=10,000,000)")
print("=" * 60)

dollar_threshold = 100_000_000  # 1000万

# 使用自己实现的函数计算 dollar bar
dollar_bars = get_dollar_bars_custom(dollar_bar_input, threshold=dollar_threshold)

print(f"\nDollar Bar 统计:")
print(f"  Dollar Bar 数量: {len(dollar_bars):,}")
if len(dollar_bars) > 0:
    print(f"  期间: {dollar_bars.index.min()} 至 {dollar_bars.index.max()}")
    print(f"  平均每根 bar 的 dollar value: {dollar_bars['cum_dollar_value'].mean():,.0f}")

# =====================================================
# 3. 读取15分钟数据
# =====================================================
print("\n" + "=" * 60)
print("📊 步骤3: 读取15分钟数据")
print("=" * 60)

# 读取15分钟数据
nas100_15min = pd.read_csv('equity_prices/NAS100.csv')
nas100_15min['Date'] = pd.to_datetime(nas100_15min['Date'])
nas100_15min.set_index('Date', inplace=True)

# 使用 BidClose 作为主要价格
nas100_15min['Close'] = nas100_15min['BidClose']
nas100_15min['High'] = nas100_15min['BidHigh'] 
nas100_15min['Low'] = nas100_15min['BidLow']
nas100_15min['Open'] = nas100_15min['BidOpen']

print(f"15分钟数据总笔数: {len(nas100_15min):,}")
print(f"数据期间: {nas100_15min.index.min()} 至 {nas100_15min.index.max()}")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# K线图绘制函数
# =====================================================
def plot_candlestick(ax, data, x_positions, width=0.6, color_up='green', color_down='red'):
    """
    绘制K线图
    
    Parameters:
    -----------
    ax : matplotlib axes
        绘图轴
    data : pd.DataFrame
        包含 open, high, low, close 列的数据
    x_positions : array-like
        X轴位置（时间索引对应的数值位置）
    width : float
        K线宽度
    color_up : str
        上涨颜色
    color_down : str
        下跌颜色
    """
    for i, (idx, row) in enumerate(data.iterrows()):
        x = x_positions[i]
        open_price = row['open']
        high_price = row['high']
        low_price = row['low']
        close_price = row['close']
        
        # 判断涨跌
        is_up = close_price >= open_price
        color = color_up if is_up else color_down
        
        # 绘制上下影线
        ax.plot([x, x], [low_price, high_price], color='black', linewidth=0.5)
        
        # 绘制实体
        body_low = min(open_price, close_price)
        body_high = max(open_price, close_price)
        body_height = body_high - body_low
        
        # 如果实体高度为0，画一条线
        if body_height == 0:
            ax.plot([x - width/2, x + width/2], [open_price, close_price], 
                   color=color, linewidth=2)
        else:
            rect = Rectangle((x - width/2, body_low), width, body_height,
                           facecolor=color, edgecolor='black', linewidth=0.5, alpha=0.8)
            ax.add_patch(rect)

# =====================================================
# 读取数据（假设已经计算好dollar_bars和nas100_15min）
# =====================================================
# 这里假设你已经有了 dollar_bars 和 nas100_15min 数据
# 如果没有，请先运行前面的代码计算

# 确定最近30天的日期范围
end_date = max(
    dollar_bars.index.max() if len(dollar_bars) > 0 else pd.Timestamp('2000-01-01'),
    nas100_15min.index.max()
)
start_date = end_date - timedelta(days=30)

# 筛选最近30天的数据
dollar_bars_plot = dollar_bars.loc[
    (dollar_bars.index >= start_date) & (dollar_bars.index <= end_date)
] if len(dollar_bars) > 0 else pd.DataFrame()

nas100_15min_plot = nas100_15min.loc[
    (nas100_15min.index >= start_date) & (nas100_15min.index <= end_date)
]

# 准备15分钟bar的K线数据
nas100_15min_ohlc = nas100_15min_plot[['Open', 'High', 'Low', 'Close']].copy()
nas100_15min_ohlc.columns = ['open', 'high', 'low', 'close']

# 准备dollar bar的K线数据
dollar_bars_ohlc = dollar_bars_plot[['open', 'high', 'low', 'close']].copy()

print(f"绘图期间: {start_date.date()} 至 {end_date.date()}")
print(f"15分钟数据: {len(nas100_15min_plot):,} 笔")
print(f"Dollar Bar: {len(dollar_bars_plot):,} 根")

# =====================================================
# 创建时间轴映射（以15分钟bar的时间轴为准）
# =====================================================
# 将时间转换为数值位置（用于X轴）
time_to_position_15min = {time: i for i, time in enumerate(nas100_15min_plot.index)}

# 为dollar bar找到对应的X轴位置
# 如果dollar bar的时间在15分钟bar之间，使用最近的下一个15分钟bar位置
dollar_bar_positions = []
for db_time in dollar_bars_plot.index:
    # 找到最接近的15分钟bar位置
    # 如果dollar bar时间在15分钟bar之前，使用第一个15分钟bar的位置
    # 如果dollar bar时间在15分钟bar之后，使用最后一个15分钟bar的位置
    # 否则使用最接近的15分钟bar位置
    if db_time <= nas100_15min_plot.index[0]:
        pos = 0
    elif db_time >= nas100_15min_plot.index[-1]:
        pos = len(nas100_15min_plot) - 1
    else:
        # 找到最接近的15分钟bar位置（使用下一个15分钟bar的位置）
        idx = nas100_15min_plot.index.searchsorted(db_time, side='right')
        pos = min(idx, len(nas100_15min_plot) - 1)
    dollar_bar_positions.append(pos)

# 为了在同一时间段显示多根dollar bar，我们需要调整位置
# 统计每个15分钟时间段内的dollar bar数量
from collections import defaultdict
dollar_bars_per_15min = defaultdict(list)
for i, db_time in enumerate(dollar_bars_plot.index):
    pos = dollar_bar_positions[i]
    dollar_bars_per_15min[pos].append(i)

# 调整位置：如果同一时间段有多根dollar bar，将它们分散显示
adjusted_dollar_bar_positions = []
dollar_bar_widths = []
for i, db_time in enumerate(dollar_bars_plot.index):
    pos = dollar_bar_positions[i]
    bars_in_slot = dollar_bars_per_15min[pos]
    if len(bars_in_slot) > 1:
        # 多根bar，分散显示
        idx_in_slot = bars_in_slot.index(i)
        slot_width = 0.8  # 每个时间段的宽度
        offset = (idx_in_slot - (len(bars_in_slot) - 1) / 2) * (slot_width / len(bars_in_slot))
        adjusted_pos = pos + offset
        width = 0.8 / len(bars_in_slot) * 0.6  # 根据bar数量调整宽度
    else:
        # 单根bar，居中显示
        adjusted_pos = pos
        width = 0.4
    adjusted_dollar_bar_positions.append(adjusted_pos)
    dollar_bar_widths.append(width)

# =====================================================
# 绘制图表
# =====================================================
fig = plt.figure(figsize=(20, 14))

# 创建子图布局
gs = fig.add_gridspec(3, 1, height_ratios=[2, 2, 1], hspace=0.3)

# 子图1: 15分钟K线图
ax1 = fig.add_subplot(gs[0])
x_positions_15min = np.arange(len(nas100_15min_plot))
plot_candlestick(ax1, nas100_15min_ohlc, x_positions_15min, width=0.6)
ax1.set_title('15min Bars - Candlestick Chart (Last 30 Days)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.set_xticks(x_positions_15min[::max(1, len(nas100_15min_plot)//10)])
ax1.set_xticklabels([nas100_15min_plot.index[i].strftime('%Y-%m-%d %H:%M') 
                     for i in range(0, len(nas100_15min_plot), max(1, len(nas100_15min_plot)//10))],
                    rotation=45, ha='right')

# 子图2: Dollar Bar K线图（使用相同的X轴时间轴）
ax2 = fig.add_subplot(gs[1], sharex=ax1)
plot_candlestick(ax2, dollar_bars_ohlc, adjusted_dollar_bar_positions, 
                 width=np.array(dollar_bar_widths))
ax2.set_title('Dollar Bars (10M) - Candlestick Chart (Last 30 Days)', 
              fontsize=14, fontweight='bold')
ax2.set_ylabel('Price', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.set_xticks(x_positions_15min[::max(1, len(nas100_15min_plot)//10)])
ax2.set_xticklabels([nas100_15min_plot.index[i].strftime('%Y-%m-%d %H:%M') 
                     for i in range(0, len(nas100_15min_plot), max(1, len(nas100_15min_plot)//10))],
                    rotation=45, ha='right')

# 子图3: Dollar Value
ax3 = fig.add_subplot(gs[2], sharex=ax1)
if len(dollar_bars_plot) > 0:
    ax3.bar(adjusted_dollar_bar_positions, dollar_bars_plot['cum_dollar_value'], 
            width=np.array(dollar_bar_widths), alpha=0.7, color='steelblue', edgecolor='black')
    ax3.axhline(y=10_000_000, color='red', linestyle='--', linewidth=2, 
                label=f'Threshold: 10,000,000')
ax3.set_title('Dollar Bar: Cumulative Dollar Value per Bar', 
              fontsize=14, fontweight='bold')
ax3.set_xlabel('Time (15min Bar Timeline)', fontsize=12)
ax3.set_ylabel('Cumulative Dollar Value', fontsize=12)
ax3.legend(loc='best', fontsize=11)
ax3.grid(True, alpha=0.3)
ax3.set_xticks(x_positions_15min[::max(1, len(nas100_15min_plot)//10)])
ax3.set_xticklabels([nas100_15min_plot.index[i].strftime('%Y-%m-%d %H:%M') 
                     for i in range(0, len(nas100_15min_plot), max(1, len(nas100_15min_plot)//10))],
                    rotation=45, ha='right')

plt.tight_layout()
plt.show()

# =====================================================
# 统计信息
# =====================================================
print("\n" + "=" * 60)
print("📊 统计信息")
print("=" * 60)

# 统计每个15分钟时间段内的dollar bar数量
max_bars_per_15min = max(len(bars) for bars in dollar_bars_per_15min.values()) if dollar_bars_per_15min else 0
avg_bars_per_15min = np.mean([len(bars) for bars in dollar_bars_per_15min.values()]) if dollar_bars_per_15min else 0

print(f"\nDollar Bar 分布统计:")
print(f"  每个15分钟时间段最多有 {max_bars_per_15min} 根 dollar bar")
print(f"  每个15分钟时间段平均有 {avg_bars_per_15min:.2f} 根 dollar bar")
print(f"  有 {sum(1 for bars in dollar_bars_per_15min.values() if len(bars) > 1)} 个时间段有多根 dollar bar")

if len(dollar_bars_plot) > 0:
    print(f"\nDollar Bar 统计:")
    print(f"  总 bar 数: {len(dollar_bars_plot):,}")
    print(f"  平均 dollar value: {dollar_bars_plot['cum_dollar_value'].mean():,.0f}")
    print(f"  最小 dollar value: {dollar_bars_plot['cum_dollar_value'].min():,.0f}")
    print(f"  最大 dollar value: {dollar_bars_plot['cum_dollar_value'].max():,.0f}")

print("\n" + "=" * 60)

📊 步骤1: 读取分钟数据
分钟数据总笔数: 3,463,681
数据期间: 2015-11-01 23:00:00 至 2025-11-11 02:35:00
有效数据笔数: 3,463,681

📊 步骤2: 计算 Dollar Bar (threshold=10,000,000)

Dollar Bar 统计:
  Dollar Bar 数量: 189,434
  期间: 2015-11-03 10:04:00 至 2025-11-11 02:28:00
  平均每根 bar 的 dollar value: 114,316,430

📊 步骤3: 读取15分钟数据
15分钟数据总笔数: 386,924
数据期间: 2008-08-19 09:30:00 至 2025-09-12 05:45:00
绘图期间: 2025-10-12 至 2025-11-11
15分钟数据: 0 笔
Dollar Bar: 6,345 根


IndexError: index 0 is out of bounds for axis 0 with size 0

## 步驟1：資料雜訊過濾

In [ ]:
# =====================================================
# 1️⃣ 雜訊過濾 Class
# =====================================================
class NoiseFilter:
    """
    雜訊過濾器：計算波動率、CUSUM 過濾等
    
    CUSUM Filter 說明：
    ==================
    CUSUM (Cumulative Sum) Filter 是一種品質控制方法，用於檢測測量值
    相對於目標值的均值偏移。
    
    用途：
    -----
    1. 過濾市場雜訊，只保留顯著的價格變動事件
    2. 避免在價格在閾值附近徘徊時觸發多個事件（這是 Bollinger Bands 等
       常見市場信號的缺點）
    3. 需要完整的運行長度 h 才會觸發事件，使事件更具意義
    
    數學原理：
    ---------
    對於觀測值 {y_t}_{t=1,...,T}，定義累積和：
    
    S^+_t = max{0, S^+_{t-1} + y_t - E_{t-1}[y_t]},  S^+_0 = 0
    S^-_t = min{0, S^-_{t-1} + y_t - E_{t-1}[y_t]},  S^-_0 = 0
    S_t = max{S^+_t, -S^-_t}
    
    其中 E_{t-1}[y_t] = y_{t-1}（使用前一期值作為期望值）
    
    當 S_t ≥ h（閾值）時，觸發事件並重置 S_t = 0。
    
    對稱 CUSUM Filter 的優勢：
    -------------------------
    - 同時捕捉上漲和下跌的累積偏差
    - 零底線機制：當 y_t ≤ E_{t-1}[y_t] - S_{t-1} 時，S_t = 0
    - 避免在閾值附近反覆觸發事件
    - 需要完整的運行長度才會觸發，使事件更具統計意義
    
    參考文獻：
    --------
    - Lam and Yam (1997): 提出交替買賣信號的投資策略
    - Fama and Blume (1966): Filter trading strategy
    """
    
    def __init__(self, vol_span: int = 100):
        """
        Parameters:
        -----------
        vol_span : int
            日波動率計算的 EWM span
        """
        self.vol_span = vol_span
        self.daily_vol = None
        self.total_bars = None
        self.filtered_bars = None
        
    def calculate_daily_vol(self, close: pd.Series) -> pd.Series:
        """
        計算日波動率
        
        使用指數加權移動標準差 (EWM) 計算波動率，對近期數據給予更高權重。
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
            
        Returns:
        --------
        daily_vol : pd.Series
            日波動率序列
        """
        # 記錄總 bar 數
        self.total_bars = len(close)
        
        # 計算日收益率
        df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
        df0 = df0[df0 > 0]
        df0 = pd.Series(close.index[df0 - 1], 
                        index=close.index[close.shape[0] - df0.shape[0]:])
        df0 = close.loc[df0.index] / close.loc[df0.values].values - 1
        
        # 使用 EWM 計算標準差（波動率）
        df0 = df0.ewm(span=self.vol_span).std()
        
        self.daily_vol = df0
        
        # 統計資訊
        valid_vol_count = df0.notna().sum()
        vol_stats = self.get_vol_stats()
        
        print("=" * 60)
        print("📊 波動率計算統計")
        print("=" * 60)
        print(f"總 K 線數 (Total Bars): {self.total_bars:,}")
        print(f"有效波動率數據數: {valid_vol_count:,}")
        print(f"波動率計算時間窗口 (vol_span): {self.vol_span} 期")
        print(f"  (註: EWM 使用指數加權，不是固定窗口)")
        print(f"\n波動率統計:")
        if vol_stats:
            print(f"  平均值 (Mean): {vol_stats['mean']:.6f}")
            print(f"  標準差 (Std): {vol_stats['std']:.6f}")
            print(f"  最小值 (Min): {vol_stats['min']:.6f}")
            print(f"  最大值 (Max): {vol_stats['max']:.6f}")
            print(f"  中位數 (Median): {vol_stats['median']:.6f}")
        print("=" * 60)
        
        return df0
    
    def cusum_filter(self, close: pd.Series, threshold: float) -> pd.DatetimeIndex:
        """
        對稱 CUSUM 過濾器 (Symmetric CUSUM Filter)
        
        實現原理：
        ---------
        對價格序列的差分（diff = y_t - y_{t-1}）應用對稱 CUSUM 過濾：
        
        S^+_t = max{0, S^+_{t-1} + diff_t}  (上漲累積)
        S^-_t = min{0, S^-_{t-1} + diff_t}  (下跌累積)
        
        當 S^-_t < -h 或 S^+_t > h 時：
        - 觸發事件（記錄時間點）
        - 重置對應的累積和為 0
        
        這樣設計的優勢：
        --------------
        1. 避免在閾值附近反覆觸發（需要完整的運行長度 h）
        2. 同時捕捉上漲和下跌的累積偏差
        3. 零底線機制確保只捕捉單方向的持續偏差
        
        Parameters:
        -----------
        close : pd.Series
            價格序列
        threshold : float
            CUSUM 閾值 (h)
            通常使用波動率的平均值或標準差
            
        Returns:
        --------
        tEvents : pd.DatetimeIndex
            過濾後的事件時間點
            
        參考實現：
        ---------
        基於 Snippet 2.4 (Advances in Financial Machine Learning, Chapter 2)
        """
        # 記錄總 bar 數
        self.total_bars = len(close)
        
        tEvents = []
        sPos, sNeg = 0, 0  # S^+_0 = 0, S^-_0 = 0
        diff = close.diff()  # y_t - y_{t-1}
        
        # 遍歷每個時間點（從第二個開始，因為第一個 diff 是 NaN）
        for i in diff.index[1:]:
            # 更新累積和
            # S^+_t = max{0, S^+_{t-1} + diff_t}
            sPos = max(0, sPos + diff.loc[i])
            # S^-_t = min{0, S^-_{t-1} + diff_t}
            sNeg = min(0, sNeg + diff.loc[i])
            
            # 檢查是否觸發事件
            if sNeg < -threshold:
                # 下跌累積超過閾值：觸發事件並重置
                sNeg = 0
                tEvents.append(i)
            elif sPos > threshold:
                # 上漲累積超過閾值：觸發事件並重置
                sPos = 0
                tEvents.append(i)
        
        filtered_events = pd.DatetimeIndex(tEvents)
        self.filtered_bars = len(filtered_events)
        
        # 統計資訊
        filter_rate = self.filtered_bars / self.total_bars if self.total_bars > 0 else 0
        
        print("=" * 60)
        print("📊 CUSUM 過濾統計")
        print("=" * 60)
        print(f"總 K 線數 (Total Bars): {self.total_bars:,}")
        print(f"篩選後 K 線數 (Filtered Bars): {self.filtered_bars:,}")
        print(f"過濾比例: {filter_rate:.4%}")
        print(f"CUSUM 閾值 (Threshold h): {threshold:.6f}")
        print(f"\n說明:")
        print(f"  - 只保留價格累積變動超過閾值 {threshold:.6f} 的時間點")
        print(f"  - 避免在閾值附近反覆觸發事件")
        print(f"  - 需要完整的運行長度才會觸發，使事件更具統計意義")
        print("=" * 60)
        
        return filtered_events
    
    def get_vol_stats(self) -> dict:
        """取得波動率統計資訊"""
        if self.daily_vol is None:
            return None
        
        return {
            'mean': self.daily_vol.mean(),
            'std': self.daily_vol.std(),
            'min': self.daily_vol.min(),
            'max': self.daily_vol.max(),
            'median': self.daily_vol.median()
        }
    
    def plot_volatility(self):
        """繪製波動率圖"""
        import matplotlib.pyplot as plt
        
        if self.daily_vol is None:
            print("⚠️ 請先執行 calculate_daily_vol()")
            return
        
        fig, ax = plt.subplots(figsize=(15, 5))
        self.daily_vol.plot(ax=ax, title='Daily Volatility', linewidth=1)
        ax.axhline(self.daily_vol.mean(), color='red', linestyle='--', 
                   label=f'Mean: {self.daily_vol.mean():.4f}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

In [ ]:
# 1. 雜訊過濾
noise_filter = NoiseFilter(vol_span=500)
daily_vol = noise_filter.calculate_daily_vol(nas100_raw['Close'])

# 2. CUSUM 篩選數據
# 使用波動率平均值作為閾值
# cusum_threshold = daily_vol.mean()
cusum_threshold = 0.1
cusum_events = noise_filter.cusum_filter(nas100_raw['Close'], cusum_threshold)
df_filtered = nas100_raw.loc[cusum_events].copy()  # ✅ 過濾後的數據
print(f"過濾後的資料筆數: {len(df_filtered)}")
noise_filter.plot_volatility()
df_filtered

## 步驟2 ：策略訊號計算

In [ ]:
# =====================================================
# 2️⃣ L1-OLD 策略 Class
# =====================================================
class L1Strategy:
    """L1-OLD 突破策略"""
    
    def __init__(self, entry_param: float = 0.5, rolling_window: int = 460):
        """
        Parameters:
        -----------
        entry_param : float
            突破參數（預設 0.5）
        rolling_window : int
            計算 HL115 和突破價的滾動窗口（預設 460）
        """
        self.entry_param = entry_param
        self.rolling_window = rolling_window
        self.signals = None
        
    def calculate_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        計算策略指標
        
        Parameters:
        -----------
        df : pd.DataFrame
            需包含 'High', 'Low', 'Open', 'Close' 欄位
            
        Returns:
        --------
        df : pd.DataFrame
            新增 'HL115', '突破signal價', 'signal', 'side' 欄位
        """
        df = df.copy()
        
        # 計算 HL115（過去N根K線的價格範圍）
        df['HL115'] = (
            df['High'].shift(1).rolling(self.rolling_window, 
                                           min_periods=self.rolling_window).max() - 
            df['Low'].shift(1).rolling(self.rolling_window, 
                                         min_periods=self.rolling_window).min()
        )
        df['HL115%'] = df['HL115'] / df['Low'].shift(self.rolling_window+1)
        
        # 計算突破signal價
        df['突破signal價'] = (
            df['Low'].shift(1).rolling(self.rolling_window).min() + 
            (self.entry_param * df['HL115'])
        )
        df['突破signal價%'] = df['突破signal價']/df['Open']
        
        # 生成突破信號
        df['signal'] = False
        df['signal'] = (
            (df['signal'].shift(1) == False) & 
            (df['High'] > df['突破signal價']) & 
            (df['Open'] < df['突破signal價'])
        )
        
        # 轉換為 side（做多方向）
        df['side'] = np.nan
        df.loc[df['signal'] == True, 'side'] = 1.0
        df['side'] = df['side'].ffill()
        
        self.signals = df
        
        # 統計資訊
        total_bars = len(df)
        signal_count = df['signal'].sum()
        signal_rate = signal_count / total_bars if total_bars > 0 else 0
        
        print("=" * 60)
        print("📊 L1-OLD 策略統計")
        print("=" * 60)
        print(f"總 K 線數 (Bars): {total_bars:,}")
        print(f"信號數量 (Signals): {signal_count:,}")
        print(f"信號比例: {signal_rate:.4%}")
        print(f"策略參數: entry_param={self.entry_param}, rolling_window={self.rolling_window}")
        print("=" * 60)
        
        return df
    
    def get_signal_events(self) -> pd.DatetimeIndex:
        """取得信號發生的時間點"""
        if self.signals is None:
            print("⚠️ 請先執行 calculate_indicators()")
            return None
        
        return self.signals[self.signals['signal'] == True].index
    
    def get_signal_stats(self) -> dict:
        """取得信號統計資訊"""
        if self.signals is None:
            return None
        
        signal_count = self.signals['signal'].sum()
        total_bars = len(self.signals)
        
        return {
            'total_signals': signal_count,
            'signal_rate': signal_count / total_bars,
            'avg_HL115': self.signals['HL115'].mean(),
            'avg_price_to_signal': (self.signals['Close'] / self.signals['突破signal價']).mean()
        }
    
    def plot_strategy(self, start_idx: Optional[int] = None, end_idx: Optional[int] = None):
        """
        繪製策略圖
        
        Parameters:
        -----------
        start_idx : int, optional
            起始索引
        end_idx : int, optional
            結束索引
        """
        import matplotlib.pyplot as plt
        
        if self.signals is None:
            print("⚠️ 請先執行 calculate_indicators()")
            return
        
        df = self.signals.iloc[start_idx:end_idx] if start_idx or end_idx else self.signals
        signal_events = df[df['signal'] == True].index
        
        fig, axes = plt.subplots(2, 1, figsize=(15, 10))
        
        # 1. 價格與突破信號
        axes[0].plot(df.index, df['Close'], label='Close Price', alpha=0.7, linewidth=1)
        axes[0].plot(df.index, df['突破signal價'], label='Breakout Signal', 
                    linestyle='--', alpha=0.7, color='orange')
        axes[0].scatter(signal_events, df.loc[signal_events, 'Close'], 
                       color='red', s=50, label='Entry Signal', zorder=5, marker='^')
        axes[0].set_title(f'L1-OLD Strategy (entry_param={self.entry_param})', fontsize=12)
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # 2. HL115 波動指標
        axes[1].plot(df.index, df['HL115'], label='HL115', color='orange', linewidth=1)
        axes[1].set_title(f'Price Range Indicator (window={self.rolling_window})', fontsize=12)
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

In [ ]:


# 3. 在篩選後的數據上計算策略信號
print("\n" + "=" * 60)
print("📈 在篩選後的數據上計算策略信號")
print("=" * 60)

strategy = L1Strategy(entry_param=0.5, rolling_window=460)
df = strategy.calculate_indicators(df_filtered)  # ✅ 使用 df_filtered
tEvents = strategy.get_signal_events()  # 策略信號的事件點
strategy.plot_strategy()

## 步驟3：Meta Labeling 目標變數生成與止盈止損設定

In [ ]:
# =====================================================
# 3️⃣ Meta-Labeling Class
# =====================================================
class MetaLabeling:
    """
    Meta-Labeling：Triple Barrier 和標籤生成
    
    Triple Barrier Method 說明：
    ============================
    Triple Barrier Method 是一種標籤方法，根據三個屏障中第一個被觸及的
    屏障來標記觀測值。
    
    用途：
    -----
    1. 動態設定止盈止損：根據估計的波動率（已實現或隱含）動態調整
    2. 避免目標過高或過低：考慮當前波動率，設定合理的止盈止損水平
    3. 路徑依賴標籤：考慮從 [t_i,0, t_i,0 + h] 的完整價格路徑
    
    三個屏障：
    --------
    1. 上方水平屏障（Profit Taking）：止盈限制
    2. 下方水平屏障（Stop Loss）：止損限制
    3. 垂直屏障（Vertical Barrier）：時間到期限制（持有期）
    
    屏障配置：
    --------
    用三元組 [pt, sl, t1] 表示，其中：
    - 0 表示屏障未啟用
    - 1 表示屏障啟用
    
    常用配置：
    - [1,1,1]: 標準設定（止盈、止損、時間到期）
    - [0,1,1]: 只有止損和時間到期
    - [1,1,0]: 只有止盈和止損（無時間限制）
    
    數學原理：
    ---------
    對於每個事件時間點 t_i,0：
    
    1. 上方屏障（止盈）：
       PT = t_i,0 + pt × σ_t_i,0
       其中 pt 是止盈倍數，σ_t_i,0 是目標波動率
    
    2. 下方屏障（止損）：
       SL = t_i,0 - sl × σ_t_i,0
       其中 sl 是止損倍數
    
    3. 垂直屏障（時間到期）：
       t_i,1 = t_i,0 + h
       其中 h 是持有期（期數）
    
    4. 第一個觸及的屏障：
       t_i,1 = min{PT_touch, SL_touch, t_i,0 + h}
    
    5. 標籤生成：
       - bin = 1:  觸及上方屏障（成功）
       - bin = -1: 觸及下方屏障（失敗）
       - bin = 0:  觸及垂直屏障（時間到期）
    
    路徑依賴性：
    ----------
    為了標記觀測值，必須考慮從 t_i,0 到 t_i,0 + h 的完整價格路徑。
    這使得標籤是路徑依賴的，而非僅依賴終點價格。
    
    動態閾值：
    --------
    使用 getDailyVol() 計算日波動率，作為動態調整止盈止損的基礎：
    
    σ_t = EWM_std(returns, span=span0)
    
    其中 returns 是日收益率序列。
    
    參考文獻：
    --------
    - Advances in Financial Machine Learning, Chapter 3
    - Snippet 3.1: getDailyVol - 計算日波動率
    - Snippet 3.2: applyPtSlOnT1 - Triple Barrier 標籤方法
    - Snippet 3.3: getEvents - 取得第一個屏障觸及時間
    - Snippet 3.4: getVerticalBarriers - 設定垂直屏障
    - Snippet 3.5: getBins - 生成標籤
    """
    
    def __init__(self, ptSl: List[float] = [1, 2], numPeriods: int = 100, 
                 minRet: float = 0.01, min_label_pct: float = 0.05):
        """
        Parameters:
        -----------
        ptSl : list
            [止盈倍數, 止損倍數]
            - ptSl[0]: 上方屏障的倍數（Profit Taking）
            - ptSl[1]: 下方屏障的倍數（Stop Loss）
            - 如果為 0，則該屏障不啟用
        numPeriods : int
            垂直屏障期數（例如：100 根 K 線）
            對應論文中的 h（持有期）
        minRet : float
            最小波動率閾值
            只有當 trgt > minRet 時才會生成事件
        min_label_pct : float
            最小標籤比例（用於 dropLabels）
            低於此比例的標籤會被移除
        """
        self.ptSl = ptSl
        self.numPeriods = numPeriods
        self.minRet = minRet
        self.min_label_pct = min_label_pct
        self.events = None
        self.bins = None
        
    def get_vertical_barriers(self, close: pd.Series, tEvents: pd.DatetimeIndex) -> pd.Series:
        """
        設定垂直時間屏障（以期數為單位）
        
        實現原理（基於 Snippet 3.4）：
        ----------------------------
        對每個事件時間點 t_i,0，找到 numPeriods 期後的時間點。
        
        公式：
        t_i,1 = close.index[position(t_i,0) + numPeriods]
        
        如果超出數據範圍，則使用最後一個時間點。
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        tEvents : pd.DatetimeIndex
            事件時間點
            
        Returns:
        --------
        t1 : pd.Series
            垂直屏障時間點（index=事件起點, value=屏障時間點）
        """
        t1 = []
        close_index = close.index
        
        for tEvent in tEvents:
            # 找到 tEvent 在 close 中的位置
            try:
                event_pos = close_index.get_loc(tEvent)
                # 計算 numPeriods 期後的位置
                barrier_pos = event_pos + self.numPeriods
                
                # 確保不超過數據範圍
                if barrier_pos < len(close_index):
                    t1.append(close_index[barrier_pos])
                else:
                    # 如果超出範圍，使用最後一個時間點
                    t1.append(close_index[-1])
            except KeyError:
                # 如果找不到該時間點，跳過
                continue
        
        if len(t1) == 0:
            return pd.Series(dtype='datetime64[ns]')
        
        # 確保 t1 的索引與 tEvents 對應
        t1_series = pd.Series(t1, index=tEvents[:len(t1)])
        return t1_series
    
    def apply_pt_sl_on_t1(self, close: pd.Series, events: pd.DataFrame, 
                          molecule: pd.DatetimeIndex) -> pd.DataFrame:
        """
        套用止盈止損（基於 Snippet 3.2）
        
        實現原理：
        ---------
        對每個事件，計算從事件開始到垂直屏障期間的價格路徑：
        
        path_prices = close[loc:t1]
        path_returns = (path_prices / close[loc] - 1) × side
        
        然後找出：
        - 最早觸及止盈的時間：path_returns > pt × trgt
        - 最早觸及止損的時間：path_returns < -sl × trgt
        
        公式：
        - PT_level = pt × trgt
        - SL_level = -sl × trgt
        - path_returns = (close[t] / close[loc] - 1) × side
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        events : pd.DataFrame
            事件 DataFrame（包含 t1, trgt, side）
        molecule : pd.DatetimeIndex
            要處理的事件子集（用於並行計算）
            
        Returns:
        --------
        out : pd.DataFrame
            包含每個屏障觸及時間的 DataFrame
        """
        events_ = events.loc[molecule]
        out = events_[['t1']].copy(deep=True)
        
        # 計算止盈水平
        if self.ptSl[0] > 0:
            pt = self.ptSl[0] * events_['trgt']  # PT = pt × σ
        else:
            pt = pd.Series(index=events.index)  # 未啟用
        
        # 計算止損水平
        if self.ptSl[1] > 0:
            sl = -self.ptSl[1] * events_['trgt']  # SL = -sl × σ
        else:
            sl = pd.Series(index=events.index)  # 未啟用
        
        # 對每個事件計算路徑
        for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
            # 取得價格路徑
            df0 = close[loc:t1]
            # 計算路徑報酬（考慮方向）
            df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side']
            
            # 找出最早觸及止損的時間
            out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()
            # 找出最早觸及止盈的時間
            out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()
        
        return out
    
    def get_events(self, close: pd.Series, tEvents: pd.DatetimeIndex, 
                   trgt: pd.Series, side: pd.Series) -> pd.DataFrame:
        """
        生成事件並套用 Triple Barrier（基於 Snippet 3.3）
        
        實現流程：
        ---------
        1. 過濾目標：只保留 trgt > minRet 的事件
        2. 設定垂直屏障：計算每個事件的時間到期點
        3. 套用 Triple Barrier：計算止盈止損觸及時間
        4. 找出第一個觸及的屏障：t1 = min{pt_touch, sl_touch, vertical_barrier}
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
        tEvents : pd.DatetimeIndex
            事件時間點（來自 CUSUM 過濾或策略信號）
        trgt : pd.Series
            目標波動率（daily_vol，用於動態調整止盈止損）
        side : pd.Series
            方向（1 為做多，-1 為做空）
            
        Returns:
        --------
        events : pd.DataFrame
            包含 t1, trgt, side, pt, sl 的事件 DataFrame
            - t1: 第一個屏障觸及時間
            - trgt: 目標波動率
            - side: 方向
            - pt: 止盈倍數
            - sl: 止損倍數
        """
        # 1. 過濾目標（只保留波動率足夠大的事件）
        trgt = trgt.loc[tEvents]
        trgt = trgt[trgt > self.minRet]
        
        # 2. 設定垂直屏障
        t1 = self.get_vertical_barriers(close, tEvents)
        
        # 3. 取得方向
        side_ = side.loc[trgt.index]
        
        # 4. 組合事件物件
        events = pd.concat({
            't1': t1.loc[trgt.index], 
            'trgt': trgt, 
            'side': side_
        }, axis=1)
        events = events.dropna(subset=['trgt'])
        
        # 5. 計算止盈止損觸及時間
        df0 = self.apply_pt_sl_on_t1(close, events, events.index)
        
        # 6. 找出第一個觸及的屏障（最早的時間）
        events['t1'] = df0.dropna(how='all').min(axis=1)
        
        # 7. 儲存屏障配置
        events['pt'] = self.ptSl[0]
        events['sl'] = self.ptSl[1]
        
        self.events = events
        return events
    
    def get_bins(self, close: pd.Series) -> pd.DataFrame:
        """
        生成標籤（基於 Snippet 3.5，並擴展為支援 Meta-Labeling）
        
        實現原理：
        ---------
        1. 對齊價格：取得事件開始和結束時間的所有價格點
        2. 計算實際報酬：
           ret = (close[t1] / close[t0] - 1) × side
        3. 判斷觸及的屏障：
           - 如果 ret > 0 且 ret > pt × trgt → bin = 1（觸及止盈）
           - 如果 ret < 0 且 ret < -sl × trgt → bin = -1（觸及止損）
           - 否則 → bin = 0（觸及垂直屏障）
        
        標籤定義：
        --------
        - bin = 1:  成功（觸及止盈屏障）
        - bin = 0:  時間到期（觸及垂直屏障）
        - bin = -1: 虧損（觸及止損屏障）
        
        Parameters:
        -----------
        close : pd.Series
            收盤價序列
            
        Returns:
        --------
        bins : pd.DataFrame
            包含 ret, trgt, bin, side 的標籤 DataFrame
        """
        if self.events is None:
            print("⚠️ 請先執行 get_events()")
            return None
        
        events_ = self.events.dropna(subset=['t1'])
        px = events_.index.union(events_['t1'].values).drop_duplicates()
        px = close.reindex(px, method='bfill')
        
        out = pd.DataFrame(index=events_.index)
        out['ret'] = px.loc[events_['t1'].values].values / px.loc[events_.index] - 1
        out['ret'] *= events_['side']
        out['trgt'] = events_['trgt']
        
        # 判斷觸及哪個屏障
        out['bin'] = 0  # 預設為時間到期
        for date_time, values in out.iterrows():
            ret = values['ret']
            target = values['trgt']
            pt_level = ret > target * self.events.loc[date_time, 'pt']
            sl_level = ret < -target * self.events.loc[date_time, 'sl']
            
            if ret > 0.0 and pt_level:
                # 觸及止盈屏障 → bin = 1 (成功)
                out.loc[date_time, 'bin'] = 1
            elif ret < 0.0 and sl_level:
                # 觸及止損屏障 → bin = -1 (虧損)
                out.loc[date_time, 'bin'] = -1
            else:
                # 時間到期（垂直屏障）→ bin = 0
                out.loc[date_time, 'bin'] = 0
        
        # 如果有 side（Meta-Labeling），保留 side 資訊
        if 'side' in events_:
            out['side'] = events_['side']
        
        self.bins = out
        return out
    
    def drop_rare_labels(self) -> pd.DataFrame:
        """
        移除稀有標籤（基於 Snippet 3.8）
        
        目的：
        -----
        移除樣本數過少的標籤類別，避免模型學習偏斜。
        
        流程：
        -----
        1. 計算每個標籤的比例
        2. 如果最小比例 < min_label_pct，移除該標籤
        3. 重複直到所有標籤比例都 >= min_label_pct
        """
        if self.bins is None:
            print("⚠️ 請先執行 get_bins()")
            return None
        
        bins = self.bins.copy()
        
        while True:
            df0 = bins['bin'].value_counts(normalize=True)
            if df0.min() > self.min_label_pct or df0.shape[0] < 3:
                break
            print(f"Dropped label {df0.idxmin()}, percentage: {df0.min():.4f}")
            bins = bins[bins['bin'] != df0.idxmin()]
        
        self.bins = bins
        self.events = self.events.loc[bins.index]
        
        return bins
    
    def get_label_stats(self) -> dict:
        """取得標籤統計資訊"""
        if self.bins is None:
            return None
        
        label_counts = self.bins['bin'].value_counts()
        
        return {
            'total_labels': len(self.bins),
            'label_distribution': label_counts.to_dict(),
            'avg_return': self.bins['ret'].mean(),
            'win_rate': (self.bins['bin'] == 1).sum() / len(self.bins),
            'loss_rate': (self.bins['bin'] == -1).sum() / len(self.bins),
            'timeout_rate': (self.bins['bin'] == 0).sum() / len(self.bins)
        }
    
    def plot_labels(self):
        """繪製標籤分布圖"""
        import matplotlib.pyplot as plt
        
        if self.bins is None:
            print("⚠️ 請先執行 get_bins()")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # 標籤分布
        label_counts = self.bins['bin'].value_counts().sort_index()
        colors = {-1: 'red', 0: 'gray', 1: 'green'}
        label_colors = [colors.get(idx, 'steelblue') for idx in label_counts.index]
        
        label_counts.plot(kind='bar', ax=axes[0], color=label_colors)
        axes[0].set_title('Label Distribution', fontsize=12)
        axes[0].set_xlabel('Label (-1: Loss, 0: Timeout, 1: Win)')
        axes[0].set_ylabel('Count')
        axes[0].grid(True, alpha=0.3, axis='y')
        
        # 回報分布
        self.bins['ret'].hist(bins=50, ax=axes[1], alpha=0.7, color='steelblue')
        axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Return')
        axes[1].set_title('Return Distribution', fontsize=12)
        axes[1].set_xlabel('Return')
        axes[1].set_ylabel('Frequency')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

In [ ]:
# 3. Meta-Labeling（使用原始完整數據）
meta = MetaLabeling(ptSl=[0.3, 0.3], numPeriods=16)
events = meta.get_events(
    nas100_raw['Close'],  # ✅ 原始完整數據
    tEvents,              # 策略信號的事件點
    daily_vol,            # 基於原始數據計算
    df['side']            # 策略信號的方向（來自過濾後的數據）
)
bins = meta.get_bins(nas100_raw['Close'])  # ✅ 原始完整數據
meta.plot_labels()
meta.drop_rare_labels()
meta.get_label_stats()

## 步驟4 ： 樣本權重平衡

In [ ]:
# =====================================================
# 4️⃣ 樣本權重 Class（並發度、唯一性、時間衰減）
# =====================================================
class SampleWeight:
    """
    樣本權重計算：並發度、唯一性、時間衰減
    
    Chapter 4: Sample Weights 說明
    ===============================
    
    問題背景：
    --------
    在金融應用中，觀測值不是由獨立同分布（IID）過程生成的。
    當兩個標籤 y_i 和 y_j 的時間區間有重疊時（t_i,1 > t_j,0），
    它們會依賴共同的報酬 r_{t_j,0, min{t_i,1, t_j,1}}。
    
    這導致標籤序列 {y_i}_{i=1,...,I} 不是 IID，違反了大多數 ML 算法的假設。
    
    解決方案：
    --------
    通過設計採樣和加權方案來糾正重疊結果的不當影響：
    1. 計算並發度（Concurrency）：每個時間點有多少標籤同時存活
    2. 計算唯一性（Uniqueness）：每個標籤的平均唯一性
    3. 應用時間衰減（Time Decay）：讓舊樣本權重降低
    4. 組合權重：最終權重 = 唯一性 × 時間衰減（可選：× 報酬歸因）
    
    並發度（Concurrency）：
    ---------------------
    定義：兩個標籤 y_i 和 y_j 在時間 t 是並發的，當它們都依賴至少一個
    共同的報酬 r_{t-1,t} = p_t / p_{t-1} - 1。
    
    計算方式：
    對每個時間點 t = 1,...,T，形成二元陣列 {1_{t,i}}_{i=1,...,I}：
    - 1_{t,i} = 1：如果 [t_i,0, t_i,1] 與 [t-1, t] 重疊
    - 1_{t,i} = 0：否則
    
    並發度：c_t = Σ_{i=1}^I 1_{t,i}
    
    唯一性（Uniqueness）：
    --------------------
    定義：標籤 i 在時間 t 的唯一性為 u_{t,i} = 1_{t,i} / c_t
    
    平均唯一性：ū_i = (Σ_{t=1}^T 1_{t,i})^{-1} × (Σ_{t=1}^T u_{t,i})
    
    也可以解釋為：標籤 i 存活期間的並發度倒數的調和平均。
    
    報酬歸因（Return Attribution）：
    -----------------------------
    當標籤是報酬符號的函數時（{-1,1} 或 {0,1}），樣本權重可以定義為：
    
    ̃w_i = |Σ_{t=t_i,0}^{t_i,1} (r_{t-1,t} / c_t)|
    
    其中 r_{t-1,t} 是對數報酬，c_t 是時間 t 的並發度。
    
    然後標準化：w_i = ̃w_i × I / (Σ_{j=1}^I ̃w_j)
    
    時間衰減（Time Decay）：
    ----------------------
    市場是適應性系統，舊樣本不如新樣本相關。
    
    線性時間衰減：
    d[x] = max{0, a + bx}
    
    邊界條件：
    1. d[Σ_{i=1}^I ū_i] = 1（最新樣本權重為 1）
    2. d[0] = c（最舊樣本權重為 c，c ∈ [0,1]）
    
    參數 c 的意義：
    - c = 1：無時間衰減
    - 0 < c < 1：線性衰減，所有樣本權重 > 0
    - c = 0：最舊樣本權重為 0
    - c < 0：最舊的 |c|×T 部分樣本權重為 0
    
    注意：時間衰減是基於累積唯一性，而非時間順序，因為在存在冗餘
    觀測值的情況下，按時間順序衰減會使權重降低太快。
    
    參考文獻：
    --------
    - Advances in Financial Machine Learning, Chapter 4
    - Snippet 4.1: mpNumCoEvents - 計算並發度
    - Snippet 4.2: mpSampleTW - 計算平均唯一性
    - Snippet 4.10: mpSampleW - 報酬歸因權重
    - Snippet 4.11: getTimeDecay - 時間衰減因子
    """
    
    def __init__(self):
        self.num_co_events = None
        self.uniqueness = None
        self.time_decay = None
        self.sample_weights = None
        
    def compute_num_co_events(self, close_idx: pd.DatetimeIndex, 
                              t1: pd.Series) -> pd.Series:
        """
        計算每個時間點的並發事件數（Concurrency）
        基於 Snippet 4.1: mpNumCoEvents
        
        實現原理：
        ---------
        對每個時間點 t，計算有多少個事件的存活期間 [t_i,0, t_i,1] 
        與時間點 t 重疊。
        
        公式：
        c_t = Σ_{i=1}^I 1_{t,i}
        
        其中 1_{t,i} = 1 如果 [t_i,0, t_i,1] 與 [t-1, t] 重疊，否則為 0。
        
        參數:
        -----------
        close_idx : pd.DatetimeIndex
            價格數據的索引
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
            
        Returns:
        --------
        num_co_events : pd.Series
            每個時間點的並發事件數
        """
        t1 = t1.fillna(close_idx[-1])  # 未結束事件仍要計算
        
        # 初始化計數序列
        iloc = close_idx.searchsorted(np.array([t1.index[0], t1.max()]))
        count = pd.Series(0, index=close_idx[iloc[0]:iloc[1]+1])
        
        # 對每個事件，在其存活期間 +1
        for tIn, tOut in t1.items():
            count.loc[tIn:tOut] += 1.0
        
        self.num_co_events = count
        
        print("=" * 60)
        print("📊 並發度統計")
        print("=" * 60)
        print(f"總時間點數: {len(count):,}")
        print(f"平均並發事件數: {count.mean():.2f}")
        print(f"最大並發事件數: {count.max():.0f}")
        print(f"\n說明:")
        print(f"  - 並發度衡量每個時間點有多少個標籤同時存活")
        print(f"  - 高並發度表示標籤之間重疊多，資訊冗餘")
        print(f"  - 低並發度表示標籤相對獨立")
        print(f"並發度分布:")
        print(count.describe())
        print("=" * 60)
        
        return count

    def compute_uniqueness(self, t1: pd.Series, 
                          num_co_events: pd.Series) -> pd.Series:
        """
        計算每個事件的唯一性（Uniqueness）
        基於 Snippet 4.2: mpSampleTW
        
        實現原理：
        ---------
        對每個事件 i，計算其在存活期間的平均唯一性：
        
        u_{t,i} = 1_{t,i} / c_t  （時間 t 的唯一性）
        ū_i = (1 / T_i) × Σ_{t=t_i,0}^{t_i,1} u_{t,i}
        
        其中 T_i 是事件 i 的存活期數。
        
        也可以解釋為：事件存活期間的並發度倒數的調和平均。
        
        公式：
        ū_i = mean(1 / c_t) for t ∈ [t_i,0, t_i,1]
        
        參數:
        -----------
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
        num_co_events : pd.Series
            每個時間點的並發事件數序列
            
        Returns:
        --------
        uniqueness : pd.Series
            每個事件的唯一性（1 / 平均並發度）
        """
        wght = pd.Series(index=t1.index)
        
        for tIn, tOut in t1.items():
            # 計算事件存活期間的平均並發度倒數
            avg_co_events = num_co_events.loc[tIn:tOut].mean()
            wght.loc[tIn] = 1.0 / avg_co_events if avg_co_events > 0 else 0
        
        self.uniqueness = wght
        
        print("=" * 60)
        print("📊 唯一性統計")
        print("=" * 60)
        print(f"總事件數: {len(wght):,}")
        print(f"平均唯一性: {wght.mean():.4f}")
        print(f"\n說明:")
        print(f"  - 唯一性衡量每個標籤的非重疊程度")
        print(f"  - 唯一性 = 1 / 平均並發度")
        print(f"  - 唯一性越高，表示該標籤越獨立，權重應該越大")
        print(f"  - 唯一性越低，表示該標籤與其他標籤重疊多，權重應該越小")
        print(f"唯一性分布:")
        print(wght.describe())
        print("=" * 60)
        
        return wght

    def compute_return_attribution(self, 
                                  close: pd.Series,
                                  t1: pd.Series,
                                  num_co_events: pd.Series) -> pd.Series:
        """
        計算報酬歸因權重（Return Attribution）
        基於 Snippet 4.10: mpSampleW
        
        實現原理：
        ---------
        對每個事件，計算其存活期間的歸因報酬：
        
        ̃w_i = |Σ_{t=t_i,0}^{t_i,1} (r_{t-1,t} / c_t)|
        
        其中：
        - r_{t-1,t} = log(close_t / close_{t-1}) 是對數報酬
        - c_t 是時間 t 的並發度
        
        然後標準化：w_i = ̃w_i × I / (Σ_{j=1}^I ̃w_j)
        
        這樣設計的意義：
        - 將報酬按並發度分配，避免重疊事件重複計算報酬
        - 絕對報酬大的事件權重更高
        - 標準化後權重總和為 I（樣本數）
        
        參數:
        -----------
        close : pd.Series
            價格序列
        t1 : pd.Series
            事件結束時間
        num_co_events : pd.Series
            並發度序列
            
        Returns:
        --------
        return_weights : pd.Series
            報酬歸因權重
        """
        ret = np.log(close).diff()  # 對數報酬
        wght = pd.Series(index=t1.index)
        
        for tIn, tOut in t1.items():
            # 計算歸因報酬：報酬 / 並發度
            attributed_ret = (ret.loc[tIn:tOut] / 
                            num_co_events.loc[tIn:tOut]).sum()
            wght.loc[tIn] = abs(attributed_ret)
        
        # 標準化：權重總和 = 樣本數
        wght = wght * len(wght) / wght.sum()
        
        print("=" * 60)
        print("📊 報酬歸因權重統計")
        print("=" * 60)
        print(f"總事件數: {len(wght):,}")
        print(f"平均權重: {wght.mean():.4f}")
        print(f"權重總和: {wght.sum():.0f} (應等於樣本數)")
        print(f"\n說明:")
        print(f"  - 根據歸因的絕對報酬來加權")
        print(f"  - 報酬按並發度分配，避免重疊事件重複計算")
        print(f"  - 絕對報酬大的事件權重更高")
        print(f"權重分布:")
        print(wght.describe())
        print("=" * 60)
        
        return wght

    def get_time_decay_linear(self, tW: pd.Series, 
                              clf_last_w: float = 1.0) -> pd.Series:
        """
        線性時間衰減（基於 Snippet 4.11: getTimeDecay）
        
        實現原理：
        ---------
        應用分段線性衰減到觀測的唯一性（tW）：
        
        d[x] = max{0, a + bx}
        
        邊界條件：
        1. d[Σ_{i=1}^I ū_i] = 1  （最新樣本權重為 1）
        2. d[0] = clf_last_w    （最舊樣本權重為 clf_last_w）
        
        求解：
        a = 1 - b × Σ_{i=1}^I ū_i
        b = (1 - clf_last_w) / Σ_{i=1}^I ū_i  （當 clf_last_w >= 0）
        
        注意：時間衰減是基於累積唯一性 x ∈ [0, Σ_{i=1}^I ū_i]，
        而非時間順序，因為在存在冗餘觀測值的情況下，按時間順序
        衰減會使權重降低太快。
        
        參數:
        -----------
        tW : pd.Series
            唯一性權重（通常是 uniqueness）
        clf_last_w : float
            最舊樣本的權重（預設 1.0，表示不衰減）
            - c = 1: 無時間衰減
            - 0 < c < 1: 線性衰減，所有樣本權重 > 0
            - c = 0: 最舊樣本權重為 0
            - c < 0: 最舊的 |c|×T 部分樣本權重為 0
            
        Returns:
        --------
        time_decay : pd.Series
            時間衰減權重
        """
        clfW = tW.sort_index().cumsum()
        
        if clfW.iloc[-1] > 0:
            if clf_last_w >= 0:
                # 線性衰減：d[0] = clf_last_w, d[Σū] = 1
                slope = (1.0 - clf_last_w) / clfW.iloc[-1]
            else:
                # 當 clf_last_w < 0 時的特殊處理
                slope = 1 / ((clf_last_w + 1) * clfW.iloc[-1])
        else:
            slope = 0
        
        const = 1.0 - slope * clfW.iloc[-1]
        clfW = const + slope * clfW
        clfW[clfW < 0] = 0
        
        self.time_decay = clfW
        
        print("=" * 60)
        print("📊 線性時間衰減")
        print("=" * 60)
        print(f"時間衰減參數: const={const:.4f}, slope={slope:.6f}")
        print(f"最舊樣本權重 (c): {clf_last_w:.4f}")
        print(f"最新樣本權重: 1.0")
        print(f"平均時間衰減權重: {clfW.mean():.4f}")
        print(f"\n說明:")
        if clf_last_w == 1.0:
            print(f"  - c = 1: 無時間衰減")
        elif 0 < clf_last_w < 1:
            print(f"  - 0 < c < 1: 線性衰減，所有樣本權重 > 0")
        elif clf_last_w == 0:
            print(f"  - c = 0: 最舊樣本權重為 0")
        elif clf_last_w < 0:
            print(f"  - c < 0: 最舊的 |c|×T 部分樣本權重為 0")
        print(f"  - 衰減基於累積唯一性，而非時間順序")
        print("=" * 60)
        
        return clfW
    
    def get_time_decay_exp(self, tW: pd.Series, 
                          decay_rate: float = 1.0,
                          percent_of_zero_wts: float = 0.0) -> pd.Series:
        """
        指數時間衰減
        
        實現原理：
        ---------
        使用指數函數進行時間衰減：
        
        d[x] = exp((decay_rate - 1) × (Σū - x))
        
        其中 x 是累積唯一性。
        
        參數:
        -----------
        tW : pd.Series
            唯一性權重（通常是 uniqueness）
        decay_rate : float
            衰減率（>1 表示衰減更快，<1 表示衰減更慢）
        percent_of_zero_wts : float
            最舊樣本中設為 0 的比例
            
        Returns:
        --------
        time_decay : pd.Series
            時間衰減權重
        """
        clf_w = tW.sort_index().cumsum()
        last_value = clf_w.iloc[-1]
        
        # 計算指數衰減權重
        out_wts = []
        zero_threshold = int(round(len(clf_w) * percent_of_zero_wts))
        
        for i in range(len(clf_w)):
            if i < zero_threshold:
                out_wts.append(0.0)
            else:
                # 指數衰減：exp((decay_rate - 1) * (last_value - current_value))
                decay_factor = np.exp((decay_rate - 1.0) * (last_value - clf_w.iloc[i]))
                out_wts.append(decay_factor)
        
        time_decay = pd.Series(out_wts, index=clf_w.index)
        self.time_decay = time_decay
        
        print("=" * 60)
        print("📊 指數時間衰減統計")
        print("=" * 60)
        print(f"衰減率 (decay_rate): {decay_rate}")
        print(f"零權重樣本比例: {percent_of_zero_wts:.2%}")
        print(f"平均時間衰減權重: {time_decay.mean():.4f}")
        print(f"時間衰減分布:")
        print(time_decay.describe())
        print("=" * 60)
        
        return time_decay
    
    def compute_sample_weights(self, 
                              t1: pd.Series,
                              close_idx: pd.DatetimeIndex,
                              close: Optional[pd.Series] = None,
                              use_uniqueness: bool = True,
                              use_time_decay: bool = True,
                              use_return_attribution: bool = False,
                              time_decay_type: str = 'linear',
                              time_decay_params: dict = None) -> pd.Series:
        """
        計算最終樣本權重（可選擇使用哪些機制）
        
        權重組合方式：
        ------------
        1. 基礎權重 = 唯一性（如果啟用）
        2. 可選：× 報酬歸因權重（如果啟用）
        3. 可選：× 時間衰減（如果啟用）
        
        最終權重 = uniqueness × (return_attribution) × (time_decay)
        
        參數:
        -----------
        t1 : pd.Series
            事件結束時間（index 為事件開始時間）
        close_idx : pd.DatetimeIndex
            價格數據的索引
        close : pd.Series, optional
            價格序列（用於計算報酬歸因）
        use_uniqueness : bool
            是否使用唯一性權重（預設 True）
        use_time_decay : bool
            是否使用時間衰減（預設 True）
        use_return_attribution : bool
            是否使用報酬歸因權重（預設 False）
        time_decay_type : str
            時間衰減類型：'linear' 或 'exp'（預設 'linear'）
        time_decay_params : dict, optional
            時間衰減參數
            - linear: {'clf_last_w': 0.5}
            - exp: {'decay_rate': 1.2, 'percent_of_zero_wts': 0.0}
            
        Returns:
        --------
        sample_weights : pd.Series
            最終樣本權重（如果都不使用，返回全為 1 的權重）
        """
        print("=" * 60)
        print("📊 樣本權重計算配置")
        print("=" * 60)
        print(f"使用唯一性: {use_uniqueness}")
        print(f"使用報酬歸因: {use_return_attribution}")
        print(f"使用時間衰減: {use_time_decay}")
        if use_time_decay:
            print(f"時間衰減類型: {time_decay_type}")
        print("=" * 60)
        
        # 如果都不使用，返回均勻權重
        if not use_uniqueness and not use_time_decay and not use_return_attribution:
            sample_weights = pd.Series(1.0, index=t1.index)
            self.sample_weights = sample_weights
            
            print("=" * 60)
            print("📊 樣本權重統計（均勻權重）")
            print("=" * 60)
            print(f"總樣本數: {len(sample_weights):,}")
            print(f"所有樣本權重: 1.0")
            print("=" * 60)
            return sample_weights
        
        # 計算並發度（唯一性和報酬歸因都需要）
        if use_uniqueness or use_return_attribution:
            num_co_events = self.compute_num_co_events(close_idx, t1)
        else:
            num_co_events = None
        
        # 計算唯一性（如果需要）
        if use_uniqueness:
            uniqueness = self.compute_uniqueness(t1, num_co_events)
        else:
            uniqueness = pd.Series(1.0, index=t1.index)
            print("⚠️ 未使用唯一性，設為 1.0")
        
        # 計算報酬歸因（如果需要）
        if use_return_attribution:
            if close is None:
                raise ValueError("使用報酬歸因時需要提供 close 參數")
            return_weights = self.compute_return_attribution(close, t1, num_co_events)
        else:
            return_weights = pd.Series(1.0, index=t1.index)
        
        # 計算時間衰減（如果需要）
        if use_time_decay:
            if time_decay_params is None:
                time_decay_params = {}
            
            if time_decay_type == 'linear':
                clf_last_w = time_decay_params.get('clf_last_w', 1.0)
                time_decay = self.get_time_decay_linear(uniqueness, clf_last_w)
            elif time_decay_type == 'exp':
                decay_rate = time_decay_params.get('decay_rate', 1.0)
                percent_of_zero_wts = time_decay_params.get('percent_of_zero_wts', 0.0)
                time_decay = self.get_time_decay_exp(uniqueness, decay_rate, percent_of_zero_wts)
            else:
                raise ValueError(f"不支援的時間衰減類型: {time_decay_type}")
        else:
            time_decay = pd.Series(1.0, index=t1.index)
            print("⚠️ 未使用時間衰減，設為 1.0")
        
        # 計算最終權重
        common_idx = uniqueness.index.intersection(time_decay.index).intersection(return_weights.index)
        sample_weights = (uniqueness.loc[common_idx] * 
                          return_weights.loc[common_idx] * 
                          time_decay.loc[common_idx])
        
        self.sample_weights = sample_weights
        
        print("=" * 60)
        print("📊 樣本權重統計")
        print("=" * 60)
        print(f"總樣本數: {len(sample_weights):,}")
        print(f"平均權重: {sample_weights.mean():.4f}")
        print(f"權重分布:")
        print(sample_weights.describe())
        print("=" * 60)
        
        return sample_weights
    
    def plot_concurrency_vs_volatility(self, daily_vol: pd.Series):
        """
        繪製並發度 vs 波動率的關係圖
        
        參數:
        -----------
        daily_vol : pd.Series
            日波動率序列
        """
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        if self.num_co_events is None:
            print("⚠️ 請先執行 compute_num_co_events()")
            return
        
        # 對齊數據（使用原始時間點）
        to_plot = pd.DataFrame({
            'conc_events': self.num_co_events,
            'daily_vol': daily_vol
        }).dropna()
        
        if len(to_plot) == 0:
            print("⚠️ 對齊後無有效數據")
            return
        
        fig, axes = plt.subplots(2, 1, figsize=(15, 10))
        
        # 1. 時間序列圖
        ax1 = axes[0]
        ax1_twin = ax1.twinx()
        
        line1 = ax1.plot(to_plot.index, to_plot['conc_events'], 
                        label='Concurrent Events', color='blue', linewidth=1.5)
        line2 = ax1_twin.plot(to_plot.index, to_plot['daily_vol'], 
                            label='Daily Volatility', color='red', linewidth=1.5, alpha=0.7)
        
        ax1.set_xlabel('Time', fontsize=12)
        ax1.set_ylabel('Concurrent Events', color='blue', fontsize=12)
        ax1_twin.set_ylabel('Daily Volatility', color='red', fontsize=12)
        ax1.set_title('Concurrent Events vs Daily Volatility', fontsize=14)
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1_twin.tick_params(axis='y', labelcolor='red')
        ax1.grid(True, alpha=0.3)
        ax1.legend(loc='upper left')
        ax1_twin.legend(loc='upper right')
        
        # 2. 散點圖
        sns.regplot(x=to_plot['conc_events'], y=to_plot['daily_vol'], 
                ax=axes[1], scatter_kws={'alpha': 0.6, 's': 50})
        axes[1].set_xlabel('Concurrent Events', fontsize=12)
        axes[1].set_ylabel('Daily Volatility', fontsize=12)
        axes[1].set_title('Concurrent Events vs Daily Volatility (Scatter Plot)', fontsize=14)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 打印相關性
        correlation = to_plot['conc_events'].corr(to_plot['daily_vol'])
        print(f"\n📊 並發度與波動率的相關係數: {correlation:.4f}")
    
    def plot_weights_distribution(self):
        """繪製權重分布圖"""
        import matplotlib.pyplot as plt
        
        if self.sample_weights is None:
            print("⚠️ 請先執行 compute_sample_weights()")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. 唯一性分布
        if self.uniqueness is not None:
            self.uniqueness.hist(bins=50, ax=axes[0, 0], alpha=0.7, color='blue')
            axes[0, 0].set_title('Uniqueness Distribution')
            axes[0, 0].set_xlabel('Uniqueness')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].grid(True, alpha=0.3)
        
        # 2. 時間衰減分布
        if self.time_decay is not None:
            self.time_decay.hist(bins=50, ax=axes[0, 1], alpha=0.7, color='green')
            axes[0, 1].set_title('Time Decay Distribution')
            axes[0, 1].set_xlabel('Time Decay Weight')
            axes[0, 1].set_ylabel('Frequency')
            axes[0, 1].grid(True, alpha=0.3)
        
        # 3. 樣本權重分布
        self.sample_weights.hist(bins=50, ax=axes[1, 0], alpha=0.7, color='orange')
        axes[1, 0].set_title('Sample Weights Distribution')
        axes[1, 0].set_xlabel('Sample Weight')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].grid(True, alpha=0.3)
        
        # 4. 權重隨時間變化
        if len(self.sample_weights) > 0:
            sorted_weights = self.sample_weights.sort_index()
            axes[1, 1].plot(sorted_weights.index, sorted_weights.values, 
                           linewidth=1, alpha=0.7, color='purple')
            axes[1, 1].set_title('Sample Weights Over Time')
            axes[1, 1].set_xlabel('Time')
            axes[1, 1].set_ylabel('Sample Weight')
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def get_uniqueness_autocorr(self, lag: int = 1) -> float:
        """
        計算唯一性的序列相關性（AR(1)）
        
        參數:
        -----------
        lag : int
            滯後期數（預設 1）
            
        Returns:
        --------
        autocorr : float
            序列相關性
        """
        if self.uniqueness is None:
            print("⚠️ 請先執行 compute_uniqueness()")
            return None
        
        autocorr = self.uniqueness.autocorr(lag=lag)
        
        print("=" * 60)
        print("📊 唯一性序列相關性")
        print("=" * 60)
        print(f"AR({lag}) 自相關係數: {autocorr:.4f}")
        
        # 統計顯著性檢驗（簡化版）
        n = len(self.uniqueness)
        critical_value = 1.96 / np.sqrt(n)  # 95% 信心水準
        
        if abs(autocorr) > critical_value:
            print(f"✅ 統計顯著 (|{autocorr:.4f}| > {critical_value:.4f})")
            print(f"\n解釋:")
            print(f"  - 唯一性存在序列相關性，表示市場狀態具有持續性")
            print(f"  - 這在金融數據中是常見且預期的現象")
            print(f"  - 時間衰減機制有助於降低舊數據的影響")
        else:
            print(f"❌ 統計不顯著 (|{autocorr:.4f}| <= {critical_value:.4f})")
        print("=" * 60)
        
        return autocorr

In [ ]:
# =====================================================
# 完整範例：樣本權重計算（所有選項）
# =====================================================

sample_weight = SampleWeight()

# =====================================================
# 選項 A: 標準配置（唯一性 + 線性時間衰減）
# =====================================================
final_weights_A = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    use_uniqueness=True,
    use_time_decay=True,
    time_decay_type='linear',
    time_decay_params={'clf_last_w': 0.5}
)

# =====================================================
# 選項 B: 完整配置（唯一性 + 報酬歸因 + 時間衰減）
# =====================================================
final_weights_B = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    close=nas100_raw['Close'],
    use_uniqueness=True,
    use_time_decay=True,
    use_return_attribution=True,  # 加入報酬歸因
    time_decay_type='linear',
    time_decay_params={'clf_last_w': 0.5}
)

# =====================================================
# 選項 C: 指數時間衰減
# =====================================================
final_weights_C = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    use_uniqueness=True,
    use_time_decay=True,
    time_decay_type='exp',
    time_decay_params={'decay_rate': 1.2, 'percent_of_zero_wts': 0.0}
)

# =====================================================
# 選項 D: 只使用唯一性（最簡單）
# =====================================================
final_weights_D = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    use_uniqueness=True,
    use_time_decay=False,
    use_return_attribution=False
)

# =====================================================
# 選項 E: 只使用報酬歸因
# =====================================================
final_weights_E = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    close=nas100_raw['Close'],
    use_uniqueness=False,
    use_time_decay=False,
    use_return_attribution=True
)

# =====================================================
# 選項 F: 均勻權重（基準對照）
# =====================================================
final_weights_F = sample_weight.compute_sample_weights(
    t1=events['t1'],
    close_idx=nas100_raw['Close'].index,
    use_uniqueness=False,
    use_time_decay=False,
    use_return_attribution=False
)

# =====================================================
# 比較不同配置的權重統計
# =====================================================
print("\n" + "="*60)
print("📊 不同配置的權重比較")
print("="*60)

comparison = pd.DataFrame({
    'A: 唯一性+線性衰減': final_weights_A.describe(),
    'B: 唯一性+報酬歸因+線性衰減': final_weights_B.describe(),
    'C: 唯一性+指數衰減': final_weights_C.describe(),
    'D: 只唯一性': final_weights_D.describe(),
    'E: 只報酬歸因': final_weights_E.describe(),
    'F: 均勻權重': final_weights_F.describe()
})

print(comparison)

# =====================================================
# 視覺化比較
# =====================================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. 權重分布比較
axes[0, 0].hist(final_weights_A, bins=50, alpha=0.5, label='A: Uniqueness+Linear', color='blue')
axes[0, 0].hist(final_weights_B, bins=50, alpha=0.5, label='B: 完整配置', color='red')
axes[0, 0].hist(final_weights_D, bins=50, alpha=0.5, label='D: 只唯一性', color='green')
axes[0, 0].set_title('Weight Distribution Comparison')
axes[0, 0].set_xlabel('Sample Weight')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 權重隨時間變化
sorted_A = final_weights_A.sort_index()
axes[0, 1].plot(sorted_A.index, sorted_A.values, label='A: 唯一性+線性', alpha=0.7)
axes[0, 1].set_title('Weights Over Time')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Weight')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 權重分位數比較
weights_list = [final_weights_A, final_weights_B, final_weights_D]
labels_list = ['A: 唯一性+線性', 'B: 完整配置', 'D: 只唯一性']
percentiles = [25, 50, 75, 90, 95]

for i, (w, label) in enumerate(zip(weights_list, labels_list)):
    values = [w.quantile(p/100) for p in percentiles]
    axes[1, 0].plot(percentiles, values, marker='o', label=label)

axes[1, 0].set_title('Weight Percentiles Comparison')
axes[1, 0].set_xlabel('Percentile')
axes[1, 0].set_ylabel('Weight Value')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. 權重統計摘要
stats_data = {
    'Mean': [w.mean() for w in weights_list],
    'Std': [w.std() for w in weights_list],
    'Min': [w.min() for w in weights_list],
    'Max': [w.max() for w in weights_list]
}
stats_df = pd.DataFrame(stats_data, index=labels_list)
stats_df.plot(kind='bar', ax=axes[1, 1], alpha=0.7)
axes[1, 1].set_title('Weight Statistics Comparison')
axes[1, 1].set_ylabel('Value')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# =====================================================
# 最終選擇（根據需求選擇一個）
# =====================================================
# 推薦：使用選項 A 或 B
final_weights = final_weights_A  # 或 final_weights_B

print("\n" + "="*60)
print("✅ 最終選擇的權重配置")
print("="*60)
print(f"配置: 唯一性 + 線性時間衰減")
print(f"樣本數: {len(final_weights):,}")
print(f"平均權重: {final_weights.mean():.4f}")
print(f"權重範圍: [{final_weights.min():.4f}, {final_weights.max():.4f}]")
print("="*60)

## 步驟5 ： 分階差平穩價格特徵提取

主商品

In [ ]:
# =====================================================
# 完整代碼：為所有商品產生分數階差分特徵
# =====================================================

import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 2. 修正後的 FractionalDiff class（確保保留所有 index）
# =====================================================

class FractionalDiff:
    """分數階差分特徵生成"""
    
    def __init__(self):
        self.optimal_d = None
        self.fracdiff_series = None
        self.features = None
        
    def get_weights(self, d: float, size: int) -> np.ndarray:
        """計算權重（擴展窗口）"""
        w = [1.0]
        for k in range(1, size):
            w_ = -w[-1] / k * (d - k + 1)
            w.append(float(w_))
        return np.array(w[::-1]).reshape(-1, 1)
    
    def get_weights_FFD(self, d: float, thres: float = 1e-5) -> np.ndarray:
        """計算 FFD 權重（固定窗口）"""
        w = [1.0]
        k = 1
        while True:
            w_ = -w[-1] / k * (d - k + 1)
            if abs(w_) < thres:
                break
            w.append(float(w_))
            k += 1
            if k > 10000:
                break
        return np.array(w[::-1]).reshape(-1, 1)
    
    def frac_diff_FFD(self, series: pd.Series, d: float, thres: float = 1e-5) -> pd.Series:
        """
        分數階差分（固定窗口方法，FFD）
        
        重要：保留所有原始 index，只在前 width 個點設為 NaN
        """
        # 1. 計算權重（固定窗口）
        w = self.get_weights_FFD(d, thres)
        width = len(w) - 1
        
        if width < 1:
            return pd.Series(index=series.index, dtype=float)
        
        # 2. 應用權重到數值
        # 重要：先填充 NaN，但保留原始 index
        series_filled = series.fillna(method='ffill')
        
        # 建立結果 Series，使用原始 series 的完整 index
        df_ = pd.Series(index=series.index, dtype=float)
        
        # 3. 從 width 開始計算（前 width 個點會是 NaN）
        valid_count = 0
        for iloc1 in range(width, len(series_filled)):
            if iloc1 >= len(series.index):
                break
                
            loc0 = series_filled.index[iloc1 - width]
            loc1 = series_filled.index[iloc1]
            
            # 檢查原始 series 在 loc1 是否有有效值
            if loc1 not in series.index or not np.isfinite(series.loc[loc1]):
                continue
            
            # 使用固定窗口：[X_{t-width}, X_{t-width+1}, ..., X_t]
            window_data = series_filled.loc[loc0:loc1].values
            
            if len(window_data) == len(w):
                result = np.dot(w.flatten(), window_data)
                df_[loc1] = result
                valid_count += 1
        
        return df_
    
    def frac_diff(self, series: pd.Series, d: float, thres: float = 0.01) -> pd.Series:
        """分數階差分（擴展窗口）"""
        w = self.get_weights(d, series.shape[0])
        w_ = np.cumsum(np.abs(w))
        w_ /= w_[-1]
        skip = w_[w_ > thres].shape[0]
        
        series_filled = series.fillna(method='ffill')
        df_ = pd.Series(index=series.index, dtype=float)
        
        for iloc in range(skip, series_filled.shape[0]):
            loc = series_filled.index[iloc]
            if not np.isfinite(series.loc[loc]):
                continue
            
            window_data = series_filled.loc[:loc].values
            w_subset = w[-(iloc + 1):, :].flatten()
            
            if len(window_data) == len(w_subset):
                df_[loc] = np.dot(w_subset, window_data)
        
        return df_
    
    def find_optimal_d(self, series: pd.Series, 
                      d_range: Tuple[float, float] = (0, 1),
                      d_step: float = 0.05,
                      method: str = 'FFD',
                      thres: float = 1e-5,
                      target_pvalue: float = 0.05,
                      min_corr: float = 0.5) -> dict:
        """找出最適合的 d 值"""
        from statsmodels.tsa.stattools import adfuller
        from scipy.stats import jarque_bera
        
        results = []
        d_values = np.arange(d_range[0], d_range[1] + d_step, d_step)
        
        for d in d_values:
            try:
                if method == 'FFD':
                    fracdiff_series = self.frac_diff_FFD(series, d, thres).dropna()
                else:
                    fracdiff_series = self.frac_diff(series, d, thres).dropna()
                
                if len(fracdiff_series) < 10:
                    continue
                
                adf_result = adfuller(fracdiff_series, maxlag=1, regression='c', autolag=None)
                adf_stat = adf_result[0]
                p_value = adf_result[1]
                
                common_idx = series.index.intersection(fracdiff_series.index)
                if len(common_idx) > 0:
                    corr = np.corrcoef(
                        series.loc[common_idx].values,
                        fracdiff_series.loc[common_idx].values
                    )[0, 1]
                else:
                    corr = 0
                
                jb_stat, jb_pvalue = jarque_bera(fracdiff_series)[:2]
                
                results.append({
                    'd': d,
                    'adf_stat': adf_stat,
                    'p_value': p_value,
                    'correlation': corr,
                    'jb_stat': jb_stat,
                    'jb_pvalue': jb_pvalue,
                    'is_stationary': p_value < target_pvalue,
                    'meets_corr': corr >= min_corr,
                    'sample_size': len(fracdiff_series)
                })
                
            except Exception as e:
                continue
        
        results_df = pd.DataFrame(results)
        
        if len(results_df) == 0:
            return None
        
        valid_results = results_df[
            (results_df['is_stationary']) & 
            (results_df['meets_corr'])
        ]
        
        if len(valid_results) > 0:
            optimal = valid_results.loc[valid_results['p_value'].idxmin()]
            self.optimal_d = optimal['d']
        else:
            optimal = results_df.loc[results_df['p_value'].idxmin()]
            self.optimal_d = optimal['d']
        
        return {
            'optimal_d': self.optimal_d,
            'results': results_df,
            'optimal_stats': optimal.to_dict()
        }

    def find_min_d_adf(self, series: pd.Series,
                    d_range: Tuple[float, float] = (0, 1),
                    n_points: int = 11,
                    method: str = 'FFD',
                    thres: float = 0.01,
                    target_pvalue: float = 0.1) -> dict:
        """
        尋找最小 d 值（基於 Snippet 5.4）
        
        這個方法與 find_optimal_d 類似，但專門用於尋找滿足平穩性條件的最小 d 值。
        這是 AFML 書中 Snippet 5.4 的實現。
        
        參數:
        -----------
        series : pd.Series
            價格序列
        d_range : tuple
            d 值搜尋範圍 (min, max)
        n_points : int
            測試的 d 值數量（對應 np.linspace(d_range[0], d_range[1], n_points)）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值（FFD 方法）或累積權重閾值（expanding 方法）
        target_pvalue : float
            目標 p-value（ADF 檢驗）
            
        Returns:
        --------
        result : dict
            包含以下鍵值：
            - 'min_d': 最小 d 值（如果找到）
            - 'results': DataFrame，包含所有測試結果
            - 'min_d_stats': 最小 d 值的統計資訊
        """
        from statsmodels.tsa.stattools import adfuller
        
        results = []
        d_values = np.linspace(d_range[0], d_range[1], n_points)
        
        print(f"🔍 搜尋最小 d 值 (範圍: {d_range}, 測試點數: {n_points})")
        print(f"   方法: {method}, 閾值: {thres}, 目標 p-value: {target_pvalue}")
        
        for i, d in enumerate(d_values):
            try:
                # 計算分數階差分
                if method == 'FFD':
                    fracdiff_series = self.frac_diff_FFD(series, d, thres).dropna()
                else:
                    fracdiff_series = self.frac_diff(series, d, thres).dropna()
                
                if len(fracdiff_series) < 10:
                    continue
                
                # ADF 檢驗
                adf_result = adfuller(fracdiff_series, maxlag=1, regression='c', autolag=None)
                adf_stat = adf_result[0]
                p_value = adf_result[1]
                
                # 計算與原始序列的相關性
                common_idx = series.index.intersection(fracdiff_series.index)
                if len(common_idx) > 0:
                    corr = np.corrcoef(
                        series.loc[common_idx].values,
                        fracdiff_series.loc[common_idx].values
                    )[0, 1]
                else:
                    corr = 0
                
                is_stationary = p_value < target_pvalue
                
                results.append({
                    'd': d,
                    'adf_stat': adf_stat,
                    'p_value': p_value,
                    'correlation': corr,
                    'is_stationary': is_stationary,
                    'sample_size': len(fracdiff_series)
                })
                
                if (i + 1) % 5 == 0 or i == len(d_values) - 1:
                    print(f"   進度: {i+1}/{len(d_values)} (d={d:.3f}, p={p_value:.4f}, stationary={is_stationary})")
                    
            except Exception as e:
                continue
        
        results_df = pd.DataFrame(results)
        
        if len(results_df) == 0:
            print("❌ 未找到任何有效結果")
            return None
        
        # 尋找滿足平穩性條件的最小 d 值
        stationary_results = results_df[results_df['is_stationary'] == True]
        
        if len(stationary_results) > 0:
            min_d_row = stationary_results.loc[stationary_results['d'].idxmin()]
            min_d = min_d_row['d']
            self.optimal_d = min_d
            
            print(f"\n✅ 找到最小 d = {min_d:.4f}")
            print(f"   ADF p-value: {min_d_row['p_value']:.4f}")
            print(f"   相關性: {min_d_row['correlation']:.4f}")
        else:
            # 如果沒有滿足條件的，選擇 p-value 最小的
            min_d_row = results_df.loc[results_df['p_value'].idxmin()]
            min_d = min_d_row['d']
            self.optimal_d = min_d
            
            print(f"\n⚠️ 未找到滿足平穩性條件的 d 值")
            print(f"   使用 p-value 最小的 d = {min_d:.4f}")
            print(f"   ADF p-value: {min_d_row['p_value']:.4f} (目標: < {target_pvalue})")
        
        return {
            'min_d': min_d,
            'results': results_df,
            'min_d_stats': min_d_row.to_dict()
        }

    def _plot_d_search_results(self, result: dict):
        """
        繪製 d 值搜尋結果
        
        參數:
        -----------
        result : dict
            find_min_d_adf 或 find_optimal_d 的返回結果
        """
        import matplotlib.pyplot as plt
        
        if result is None or 'results' not in result:
            print("⚠️ 無效的結果")
            return
        
        results_df = result['results']
        min_d = result.get('min_d') or result.get('optimal_d')
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. ADF p-value vs d
        axes[0, 0].plot(results_df['d'], results_df['p_value'], 'o-', linewidth=2, markersize=6)
        if min_d is not None:
            min_d_pvalue = results_df[results_df['d'] == min_d]['p_value'].values[0]
            axes[0, 0].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[0, 0].scatter([min_d], [min_d_pvalue], color='red', s=100, zorder=5)
        axes[0, 0].axhline(0.05, color='gray', linestyle=':', alpha=0.5, label='p=0.05')
        axes[0, 0].axhline(0.1, color='gray', linestyle=':', alpha=0.5, label='p=0.1')
        axes[0, 0].set_xlabel('d', fontsize=12)
        axes[0, 0].set_ylabel('ADF p-value', fontsize=12)
        axes[0, 0].set_title('ADF Test p-value vs d', fontsize=14)
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. ADF Statistic vs d
        axes[0, 1].plot(results_df['d'], results_df['adf_stat'], 'o-', linewidth=2, markersize=6, color='green')
        if min_d is not None:
            min_d_stat = results_df[results_df['d'] == min_d]['adf_stat'].values[0]
            axes[0, 1].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[0, 1].scatter([min_d], [min_d_stat], color='red', s=100, zorder=5)
        axes[0, 1].axhline(0, color='gray', linestyle=':', alpha=0.5)
        axes[0, 1].set_xlabel('d', fontsize=12)
        axes[0, 1].set_ylabel('ADF Statistic', fontsize=12)
        axes[0, 1].set_title('ADF Test Statistic vs d', fontsize=14)
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Correlation vs d
        axes[1, 0].plot(results_df['d'], results_df['correlation'], 'o-', linewidth=2, markersize=6, color='orange')
        if min_d is not None:
            min_d_corr = results_df[results_df['d'] == min_d]['correlation'].values[0]
            axes[1, 0].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[1, 0].scatter([min_d], [min_d_corr], color='red', s=100, zorder=5)
        axes[1, 0].set_xlabel('d', fontsize=12)
        axes[1, 0].set_ylabel('Correlation with Original', fontsize=12)
        axes[1, 0].set_title('Correlation vs d', fontsize=14)
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # 4. Stationarity Status
        if 'is_stationary' in results_df.columns:
            colors = ['red' if not stat else 'green' for stat in results_df['is_stationary']]
            axes[1, 1].scatter(results_df['d'], results_df['is_stationary'].astype(int), 
                            c=colors, s=100, alpha=0.6)
            if min_d is not None:
                axes[1, 1].axvline(min_d, color='red', linestyle='--', linewidth=2, label=f'Min d = {min_d:.3f}')
            axes[1, 1].set_xlabel('d', fontsize=12)
            axes[1, 1].set_ylabel('Is Stationary (1=Yes, 0=No)', fontsize=12)
            axes[1, 1].set_title('Stationarity Status vs d', fontsize=14)
            axes[1, 1].set_ylim(-0.1, 1.1)
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 1].text(0.5, 0.5, 'No stationarity data', 
                            ha='center', va='center', fontsize=12)
            axes[1, 1].set_title('Stationarity Status vs d', fontsize=14)
        
        plt.tight_layout()
        plt.show()

    def _plot_min_d_results(self, result: dict, d: float):
        """
        繪製使用最小 d 值後的結果
        
        參數:
        -----------
        result : dict
            find_min_d_adf 的返回結果
        d : float
            使用的 d 值
        """
        import matplotlib.pyplot as plt
        
        if result is None:
            print("⚠️ 無效的結果")
            return
        
        # 這個方法可以繪製使用 d 值後的時間序列對比
        # 但需要實際的分數階差分序列，所以這裡只顯示統計資訊
        min_d_stats = result.get('min_d_stats', {})
        
        print("\n" + "=" * 60)
        print("📊 最小 d 值統計資訊")
        print("=" * 60)
        print(f"使用的 d 值: {d:.4f}")
        if min_d_stats:
            print(f"ADF Statistic: {min_d_stats.get('adf_stat', 'N/A'):.4f}")
            print(f"ADF p-value: {min_d_stats.get('p_value', 'N/A'):.4f}")
            print(f"與原始序列相關性: {min_d_stats.get('correlation', 'N/A'):.4f}")
            print(f"樣本數: {min_d_stats.get('sample_size', 'N/A'):,}")
            print(f"是否平穩: {min_d_stats.get('is_stationary', 'N/A')}")
        print("=" * 60)

    def generate_features(self, df: pd.DataFrame, event_indices: pd.DatetimeIndex,
                      price_col: str = 'Close', d: Optional[float] = None,
                      method: str = 'FFD', thres: float = 1e-5) -> pd.DataFrame:
        """
        為事件點生成分數階差分特徵
        
        參數:
        -----------
        df : pd.DataFrame
            包含價格資料的 DataFrame
        event_indices : pd.DatetimeIndex
            事件時間點
        price_col : str
            價格欄位名稱
        d : float, optional
            分數階差分階數（如果為 None，使用 self.optimal_d）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值
            
        Returns:
        --------
        features : pd.DataFrame
            分數階差分特徵（index 為 event_indices）
        """
        if price_col not in df.columns:
            raise ValueError(f"找不到價格欄位: {price_col}")
        
        # 使用指定的 d 值或最優 d 值
        if d is None:
            if self.optimal_d is None:
                raise ValueError("請先執行 find_optimal_d() 或 find_min_d_adf()，或指定 d 參數")
            d = self.optimal_d
        
        print(f"📊 生成分數階差分特徵 (d={d:.4f}, method={method})")
        
        # 計算整個序列的分數階差分
        price_series = df[price_col]
        
        if method == 'FFD':
            fracdiff_series = self.frac_diff_FFD(price_series, d, thres)
        else:
            fracdiff_series = self.frac_diff(price_series, d, thres)
        
        # 提取事件點的特徵值
        features = pd.DataFrame(index=event_indices)
        features['fracdiff'] = fracdiff_series.loc[event_indices].values
        
        # 統計資訊
        valid_count = features['fracdiff'].notna().sum()
        print(f"✅ 特徵生成完成")
        print(f"   事件數: {len(event_indices):,}")
        print(f"   有效特徵數: {valid_count:,}")
        print(f"   NaN 數: {len(event_indices) - valid_count:,}")
        
        return features
    
    def plot_price_vs_fracdiff(self, price_series: pd.Series, 
                            d: Optional[float] = None,
                            method: str = 'FFD',
                            thres: float = 1e-5,
                            n_points: Optional[int] = None,
                            normalize: bool = False):
        """
        繪製原始價格與分數階差分特徵的對比圖
        
        參數:
        -----------
        price_series : pd.Series
            原始價格序列
        d : float, optional
            分數階差分階數（如果為 None，使用 self.optimal_d 或預設值 0.5）
        method : str
            方法：'FFD' 或 'expanding'
        thres : float
            權重截斷閾值
        n_points : int, optional
            如果指定，只繪製最近 n_points 個點（用於大量數據時）
        normalize : bool
            是否標準化後繪製在同一張圖上（預設 False，分開兩個子圖）
            
        Returns:
        --------
        fracdiff_series : pd.Series
            計算出的分數階差分序列
        """
        import matplotlib.pyplot as plt
        
        # 確定使用的 d 值
        if d is None:
            if self.optimal_d is not None:
                d_value = self.optimal_d
            else:
                d_value = 0.5
                print(f"⚠️ 未指定 d 值，使用預設值 {d_value}")
        else:
            d_value = d
        
        # 計算整個序列的分數階差分
        print(f"📊 計算分數階差分 (d={d_value:.4f}, method={method})...")
        if method == 'FFD':
            fracdiff_series = self.frac_diff_FFD(price_series, d_value, thres)
        else:
            fracdiff_series = self.frac_diff(price_series, d_value, thres)
        
        # 選擇要繪製的數據範圍
        if n_points is not None and len(price_series) > n_points:
            plot_idx = price_series.index[-n_points:]
            price_plot = price_series.loc[plot_idx]
            fracdiff_plot = fracdiff_series.loc[plot_idx]
            title_suffix = f" (Last {n_points:,} Points)"
        else:
            plot_idx = price_series.index
            price_plot = price_series
            fracdiff_plot = fracdiff_series
            title_suffix = ""
        
        # 繪製圖表
        if normalize:
            # 方法 1: 標準化後繪製在同一張圖上
            common_idx = plot_idx.intersection(fracdiff_plot.dropna().index)
            if len(common_idx) > 0:
                price_normalized = (price_plot.loc[common_idx] - price_plot.loc[common_idx].mean()) / price_plot.loc[common_idx].std()
                fracdiff_normalized = (fracdiff_plot.loc[common_idx] - fracdiff_plot.loc[common_idx].mean()) / fracdiff_plot.loc[common_idx].std()
                
                fig, ax = plt.subplots(figsize=(15, 6))
                
                ax.plot(common_idx, price_normalized, 
                    label='Original Price (Normalized)', linewidth=1.5, color='blue', alpha=0.7)
                ax.plot(common_idx, fracdiff_normalized, 
                    label='Fractional Differentiation (Normalized)', linewidth=1.5, color='red', alpha=0.7)
                ax.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.3)
                ax.set_xlabel('Time', fontsize=12)
                ax.set_ylabel('Normalized Value', fontsize=12)
                ax.set_title(f'Normalized Comparison: Price vs Fractional Differentiation{title_suffix}\n(d={d_value:.4f}, method={method})', fontsize=14)
                ax.legend(loc='upper left')
                ax.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
        else:
            # 方法 2: 分開兩個子圖（預設）
            fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
            
            # 上圖：原始價格
            axes[0].plot(plot_idx, price_plot.values, 
                        label='Original Price', linewidth=1.5, color='blue', alpha=0.7)
            axes[0].set_ylabel('Price', fontsize=12)
            axes[0].set_title(f'Original Price vs Fractional Differentiation{title_suffix}\n(d={d_value:.4f}, method={method})', fontsize=14)
            axes[0].legend(loc='upper left')
            axes[0].grid(True, alpha=0.3)
            
            # 下圖：分數階差分特徵
            valid_fracdiff = fracdiff_plot.dropna()
            if len(valid_fracdiff) > 0:
                axes[1].plot(valid_fracdiff.index, valid_fracdiff.values, 
                            label='Fractional Differentiation', linewidth=1.5, color='red', alpha=0.7)
            axes[1].axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
            axes[1].set_xlabel('Time', fontsize=12)
            axes[1].set_ylabel('Fractional Differentiation', fontsize=12)
            axes[1].set_title('Fractional Differentiation Feature', fontsize=14)
            axes[1].legend(loc='upper left')
            axes[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
        
        # 統計資訊
        valid_count = fracdiff_series.notna().sum()
        print(f"✅ 繪圖完成")
        print(f"   原始數據點數: {len(price_series):,}")
        print(f"   分數階差分有效值: {valid_count:,}")
        print(f"   NaN 值: {len(price_series) - valid_count:,}")
        
        return fracdiff_series

In [ ]:
# =====================================================
# 使用範例：尋找最小 d 值
# =====================================================

fracdiff = FractionalDiff()

# 方法 1: 基本使用（與 Snippet 5.4 類似）
result = fracdiff.find_min_d_adf(
    series=nas100_raw['Close'],
    d_range=(0, 1),
    n_points=11,  # 對應 np.linspace(0, 1, 11)
    method='FFD',
    thres=0.01,   # 與 Snippet 5.4 一致
    target_pvalue=0.05
)

# # 方法 3: 更精細的搜尋
# result = fracdiff.find_min_d_adf(
#     series=nas100_raw['Close'],
#     d_range=(0, 0.6),  # 根據研究，通常 d < 0.6 即可
#     n_points=25,       # 更密集的測試
#     method='FFD',
#     thres=0.01
# )

# 使用找到的最小 d 值
if result and result['min_d'] is not None:
    min_d = result['min_d']
    print(f"\n✅ 使用最小 d = {min_d:.4f} 生成特徵")
    
    # 生成特徵
    features = fracdiff.generate_features(
        df=nas100_raw,
        event_indices=events.index,
        price_col='Close',
        d=min_d,
        method='FFD'
    )
    
# 繪製結果
fracdiff._plot_d_search_results(result)
fracdiff._plot_min_d_results(result, min_d)
# 基本使用（兩個子圖）
fracdiff.plot_price_vs_fracdiff(
    price_series=nas100_raw['Close'],
    d=None,  # 使用 self.optimal_d 或預設值
    method='FFD',
    thres=1e-5
)

其他商品

In [ ]:
# =====================================================
# 1. 資料載入函數
# =====================================================

def load_price_data(file_path: str, price_col: str = 'BidClose') -> Optional[pd.Series]:
    """載入價格資料"""
    try:
        df = pd.read_csv(file_path)
        
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)
        else:
            df.index = pd.to_datetime(df.index)
        
        if price_col in df.columns:
            price_series = df[price_col].copy()
        elif 'Close' in df.columns:
            price_series = df['Close'].copy()
        elif 'BidClose' in df.columns:
            price_series = df['BidClose'].copy()
        else:
            print(f"⚠️ {file_path}: 找不到價格欄位")
            return None
        
        price_series = price_series.dropna()
        
        if len(price_series) < 100:
            print(f"⚠️ {file_path}: 資料點太少 ({len(price_series)})")
            return None
        
        return price_series
    
    except Exception as e:
        print(f"❌ 讀取 {file_path} 失敗: {e}")
        return None

# =====================================================
# 2. 為所有商品產生多階分數階差分特徵
# =====================================================

def generate_fracdiff_features_for_all_products(
    equity_prices_dir: str = 'equity_prices',
    main_product: str = 'NAS100',
    price_col: str = 'BidClose',
    d_values: list = [0.5, 0.75, 1.0],
    method: str = 'FFD',
    thres: float = 1e-5) -> pd.DataFrame:
    """
    為所有商品產生多階分數階差分特徵
    
    參數:
    -----------
    equity_prices_dir : str
        商品資料夾路徑
    main_product : str
        主商品名稱（用於設定 index）
    price_col : str
        價格欄位名稱
    d_values : list
        要計算的分數階差分階數列表 [0.5, 0.75, 1.0]
    method : str
        方法：'FFD' 或 'expanding'
    thres : float
        權重截斷閾值
        
    Returns:
    --------
    features_df : pd.DataFrame
        所有商品的多階分數階差分特徵
        columns: [商品名_d0.5, 商品名_d0.75, 商品名_d1.0, ...]
        index: 與主商品相同
    """
    print("=" * 60)
    print("📊 為所有商品產生多階分數階差分特徵")
    print("=" * 60)
    
    fracdiff = FractionalDiff()
    
    # 找到所有 CSV 檔案
    equity_dir = Path(equity_prices_dir)
    csv_files = []
    
    for csv_file in equity_dir.glob('*.csv'):
        if csv_file.name != 'database_refresh.py':
            csv_files.append(csv_file)
    
    commodity_dir = equity_dir / 'commodity_prices'
    if commodity_dir.exists():
        for csv_file in commodity_dir.glob('*.csv'):
            csv_files.append(csv_file)
    
    print(f"找到 {len(csv_files)} 個商品檔案")
    
    # 載入主商品資料（用於設定 index）
    main_file = equity_dir / f'{main_product}.csv'
    if not main_file.exists():
        raise FileNotFoundError(f"找不到主商品檔案: {main_file}")
    
    main_price = load_price_data(str(main_file), price_col)
    if main_price is None:
        raise ValueError(f"無法載入主商品: {main_product}")
    
    print(f"\n主商品: {main_product}")
    print(f"主商品資料期間: {main_price.index[0]} 至 {main_price.index[-1]}")
    print(f"主商品資料點數: {len(main_price):,}")
    print(f"計算階數: {d_values}")
    
    # 建立 DataFrame，使用主商品的完整 index
    features_df = pd.DataFrame(index=main_price.index)
    
    # 為每個商品計算多階分數階差分
    for csv_file in csv_files:
        product_name = csv_file.stem
        
        price_series = load_price_data(str(csv_file), price_col)
        if price_series is None:
            continue
        
        print(f"\n處理商品: {product_name}")
        print(f"  原始資料點數: {len(price_series):,}")
        
        # 為每個 d 值計算分數階差分
        for d in d_values:
            try:
                if method == 'FFD':
                    fracdiff_series = fracdiff.frac_diff_FFD(price_series, d, thres)
                else:
                    fracdiff_series = fracdiff.frac_diff(price_series, d, thres)
                
                # 對齊到主商品的完整 index
                fracdiff_aligned = fracdiff_series.reindex(
                    main_price.index,
                    method=None
                )
                
                # 儲存特徵（欄位名稱：商品名_d階數）
                col_name = f'{product_name}_d{d}'
                features_df[col_name] = fracdiff_aligned
                
                valid_points = fracdiff_aligned.notna().sum()
                print(f"  d={d}: 有效值 {valid_points:,}/{len(fracdiff_aligned):,}")
                
            except Exception as e:
                print(f"  ❌ d={d} 計算失敗: {e}")
                continue
    
    print(f"\n" + "=" * 60)
    print("✅ 特徵生成完成")
    print("=" * 60)
    print(f"總商品數: {len(csv_files)}")
    print(f"總特徵數: {len(features_df.columns)}")
    print(f"總時間點數: {len(features_df):,}")
    print("=" * 60)
    
    return features_df

# =====================================================
# 3. 執行並儲存
# =====================================================

# 執行計算
fracdiff_features_all = generate_fracdiff_features_for_all_products(
    equity_prices_dir='equity_prices',
    main_product='NAS100',
    price_col='BidClose',
    d_values=[0.5, 0.75, 1.0],  # 每個商品計算這三個階數
    method='FFD',
    thres=1e-5
)

# 向前填充 NaN 值
fracdiff_features_filled = fracdiff_features_all.fillna(method='ffill')

# 儲存結果
output_file = 'fracdiff_features_all_products.csv'
fracdiff_features_filled.to_csv(output_file)
print(f"\n✅ 已保存到: {output_file}")
print(f"儲存樣本數: {len(fracdiff_features_filled):,}")
print(f"特徵數: {len(fracdiff_features_filled.columns)}")

## 步驟6 ： 其他特徵產生

技術指標

In [ ]:
import pandas as pd
import numpy as np

def calculate_technical_indicators(df):
    """15分鐘資料用的技術指標（四組尺度：SS=4H、S=9H、M=24H、L=72H）"""
    tech_df = pd.DataFrame(index=df.index)

    Open = df['Open']
    High = df['High']
    Low = df['Low']
    Close = df['Close']
    Volume = df['Volume']

    # 通用工具
    def wilders_rma(s, period):
        return s.ewm(alpha=1/period, adjust=False).mean()

    def true_range(high, low, close):
        prev_close = close.shift(1)
        return pd.concat([
            (high - low),
            (high - prev_close).abs(),
            (low - prev_close).abs()
        ], axis=1).max(axis=1)

    def mfi(high, low, close, volume, period=14):
        tp = (high + low + close) / 3.0
        rmf = tp * volume
        pos = (tp > tp.shift(1)).astype(int)
        neg = (tp < tp.shift(1)).astype(int)
        pos_mf = (rmf * pos).rolling(period).sum()
        neg_mf = (rmf * neg).rolling(period).sum()
        mr = pos_mf / neg_mf
        return 100 - (100 / (1 + mr))

    def dm_di_dx(high, low, close, period=14):
        up_move = high.diff()
        down_move = -low.diff()
        plus_dm = ((up_move > down_move) & (up_move > 0)) * up_move
        minus_dm = ((down_move > up_move) & (down_move > 0)) * down_move
        tr = true_range(high, low, close)
        atr = wilders_rma(tr, period)
        plus_di = 100 * wilders_rma(plus_dm, period) / atr
        minus_di = 100 * wilders_rma(minus_dm, period) / atr
        dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di)
        return plus_dm, minus_dm, plus_di, minus_di, dx

    # ========= TRANGE/ATR/NATR（加入 SS）=========
    # 15分鐘資料：4H=16, 14H=56, 24H=96, 72H=288
    tech_df['TRANGE'] = true_range(High, Low, Close)
    for p, tag in [(16,'SS'), (56,'S'), (96,'M'), (288,'L')]:
        atr = wilders_rma(tech_df['TRANGE'], p)
        tech_df[f'ATR_{tag}'] = atr
        tech_df[f'NATR_{tag}'] = atr / Close * 100.0

    # ========= Momentum & Oscillators =========
    # MOM / ROC / ROCR（加入 SS）
    # 15分鐘資料：4H=16, 10H=40, 24H=96, 72H=288
    for p, tag in [(16,'SS'), (40,'S'), (96,'M'), (288,'L')]:
        tech_df[f'MOM_{tag}'] = Close - Close.shift(p)
        tech_df[f'ROC_{tag}'] = Close.pct_change(p) * 100.0
        tech_df[f'ROCR_{tag}'] = Close / Close.shift(p)

    # MFI（加入 SS）
    # 15分鐘資料：4H=16, 14H=56, 24H=96, 72H=288
    for p, tag in [(16,'SS'), (56,'S'), (96,'M'), (288,'L')]:
        tech_df[f'MFI_{tag}'] = mfi(High, Low, Close, Volume, p)

    # ADX/DI/DM/DX（只保留 MINUS_DM）
    # 15分鐘資料：4H=16, 14H=56, 24H=96, 72H=288
    for p, tag in [(16,'SS'), (56,'S'), (96,'M'), (288,'L')]:
        _, minus_dm, _, _, _ = dm_di_dx(High, Low, Close, p)
        tech_df[f'MINUS_DM_{tag}'] = minus_dm

    # ========= Volume & Accumulation =========
    # Volume_SMA_20: 20個15分鐘K線（約5小時），保持不變
    tech_df['Volume_SMA_20'] = Volume.rolling(20).mean()
    tech_df['Volume_Ratio_20'] = Volume / tech_df['Volume_SMA_20']

    return tech_df

# 計算技術指標
tech_indicators = calculate_technical_indicators(nas100_raw)  # 使用15分鐘原始資料
print(f"技術指標計算完成，共 {len(tech_indicators.columns)} 個指標")

# 檢查資料品質
print(f"\n資料品質檢查:")
print(f"總筆數: {len(tech_indicators)}")
print(f"缺失值統計:")
missing_stats = tech_indicators.isnull().sum()
print(missing_stats[missing_stats > 0])

基礎統計

In [ ]:
# 波動率

def rogers_satchell_volatility(df, window=None):
    """
    計算 Rogers–Satchell Volatility（允許漂移）
    
    公式:
    σ_RS = sqrt((1/N) * Σ[ln(H_t/C_t) * ln(H_t/O_t) + ln(L_t/C_t) * ln(L_t/O_t)])
    
    參數:
    -----------
    df : pd.DataFrame
        需包含 'Open', 'High', 'Low', 'Close' 欄位
    window : int, optional
        滾動窗口（如果為 None，則計算整體波動率）
        
    Returns:
    --------
    volatility : pd.Series or float
        如果 window 為 None，返回單一數值
        如果 window 有值，返回滾動波動率序列
    """
    # 確保有必要的欄位
    required_cols = ['Open', 'High', 'Low', 'Close']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"缺少必要欄位: {col}")
    
    Open = df['Open']
    High = df['High']
    Low = df['Low']
    Close = df['Close']
    
    # 計算對數項
    # ln(H_t/C_t) * ln(H_t/O_t) + ln(L_t/C_t) * ln(L_t/O_t)
    term1 = np.log(High / Close) * np.log(High / Open)
    term2 = np.log(Low / Close) * np.log(Low / Open)
    
    # 組合項
    rs_terms = term1 + term2
    
    # 處理無效值（避免 log(0) 或負數）
    rs_terms = rs_terms.replace([np.inf, -np.inf], np.nan)
    
    if window is None:
        # 計算整體波動率
        rs_terms_clean = rs_terms.dropna()
        if len(rs_terms_clean) == 0:
            return np.nan
        
        mean_term = rs_terms_clean.mean()
        volatility = np.sqrt(mean_term)
        return volatility
    else:
        # 計算滾動波動率
        mean_terms = rs_terms.rolling(window=window, min_periods=1).mean()
        volatility = np.sqrt(mean_terms)
        return volatility

# =====================================================
# 載入資料並計算
# =====================================================

file_path = r'C:\Users\a124a\OneDrive\桌面\策略開發\Xgboost_MlFinlab\minute_prices\NAS100.csv'

print("=" * 60)
print("📊 計算 Rogers–Satchell Volatility")
print("=" * 60)

# 讀取資料
print(f"\n載入資料: {file_path}")
try:
    df = pd.read_csv(file_path)
    
    # 轉換時間格式
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
    elif df.index.dtype == 'object':
        df.index = pd.to_datetime(df.index)
    
    # 檢查並選擇價格欄位
    if 'BidOpen' in df.columns and 'BidHigh' in df.columns:
        df['Open'] = df['BidOpen']
        df['High'] = df['BidHigh']
        df['Low'] = df['BidLow']
        df['Close'] = df['BidClose']
    elif 'Open' not in df.columns:
        print("⚠️ 找不到價格欄位，嘗試使用 Close 欄位")
        if 'Close' in df.columns:
            df['Open'] = df['Close'].shift(1).fillna(df['Close'])
            df['High'] = df['Close']
            df['Low'] = df['Close']
            df['Close'] = df['Close']
    
    print(f"✅ 資料載入成功")
    print(f"   資料筆數: {len(df):,}")
    print(f"   資料期間: {df.index[0]} 至 {df.index[-1]}")
    print(f"   欄位: {list(df.columns)}")
    
except Exception as e:
    print(f"❌ 載入資料失敗: {e}")
    raise

# 計算整體 Rogers–Satchell Volatility
print(f"\n計算整體 Rogers–Satchell Volatility...")
rs_vol_overall = rogers_satchell_volatility(df, window=None)
print(f"✅ 整體波動率: {rs_vol_overall:.6f}")

# 計算滾動 Rogers–Satchell Volatility（可選）
print(f"\n計算滾動 Rogers–Satchell Volatility...")
# 使用不同的窗口大小
windows = [20, 60, 240, 1440]  # 20分鐘, 1小時, 4小時, 1天（假設是分鐘資料）

for window in windows:
    rs_vol_rolling = rogers_satchell_volatility(df, window=window)
    rs_vol_rolling_clean = rs_vol_rolling.dropna()
    
    if len(rs_vol_rolling_clean) > 0:
        mean_vol = rs_vol_rolling_clean.mean()
        std_vol = rs_vol_rolling_clean.std()
        print(f"  窗口 {window} ({window}分鐘):")
        print(f"    平均波動率: {mean_vol:.6f}")
        print(f"    標準差: {std_vol:.6f}")
        print(f"    最小值: {rs_vol_rolling_clean.min():.6f}")
        print(f"    最大值: {rs_vol_rolling_clean.max():.6f}")

# 將滾動波動率加入 DataFrame
df['RS_Volatility_20'] = rogers_satchell_volatility(df, window=20)
df['RS_Volatility_60'] = rogers_satchell_volatility(df, window=60)
df['RS_Volatility_240'] = rogers_satchell_volatility(df, window=240)
df['RS_Volatility_1440'] = rogers_satchell_volatility(df, window=1440)

print(f"\n" + "=" * 60)
print("✅ 計算完成")
print("=" * 60)
print(f"\nDataFrame 新增欄位:")
print(f"  - RS_Volatility_20 (20分鐘滾動)")
print(f"  - RS_Volatility_60 (60分鐘滾動)")
print(f"  - RS_Volatility_240 (4小時滾動)")
print(f"  - RS_Volatility_1440 (1天滾動)")

# 顯示最後幾筆資料
print(f"\n最後 5 筆資料:")
print(df[['Open', 'High', 'Low', 'Close', 'RS_Volatility_20', 'RS_Volatility_60']].tail())

# 可選：儲存結果
# output_file = 'NAS100_RS_Volatility.csv'
# df.to_csv(output_file)
# print(f"\n✅ 已保存到: {output_file}")

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 1️⃣ Skewness / Kurtosis
# =====================================================

def rolling_skewness_kurtosis(returns, window=60):
    """
    計算滾動偏度和峰度
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    window : int
        滾動窗口大小（分鐘數）
        
    Returns:
    --------
    skewness : pd.Series
        偏度序列
    kurtosis : pd.Series
        峰度序列
    """
    skewness = returns.rolling(window=window, min_periods=window).skew()
    kurtosis = returns.rolling(window=window, min_periods=window).kurt()
    
    return skewness, kurtosis

# =====================================================
# 2️⃣ Drawdown / Recovery Rate
# =====================================================

def drawdown_recovery_rate(prices, returns, window=None):
    """
    計算最大回撤和回復速度
    
    參數:
    -----------
    prices : pd.Series
        價格序列
    returns : pd.Series
        報酬序列
    window : int, optional
        滾動窗口（如果為 None，計算整體回撤）
        
    Returns:
    --------
    drawdown : pd.Series
        回撤序列
    recovery_rate : pd.Series
        回復速度序列
    max_drawdown : pd.Series
        最大回撤序列
    """
    if window is None:
        # 整體回撤
        cummax = prices.cummax()
        drawdown = 1 - prices / cummax
        
        # 回復速度
        recovery_rate = returns / drawdown.replace(0, np.nan)
        
        # 最大回撤（累積）
        max_drawdown = drawdown.cummax()
        
    else:
        # 滾動窗口回撤
        drawdown_list = []
        recovery_rate_list = []
        max_drawdown_list = []
        
        for i in range(len(prices)):
            if i < window:
                drawdown_list.append(np.nan)
                recovery_rate_list.append(np.nan)
                max_drawdown_list.append(np.nan)
                continue
            
            window_prices = prices.iloc[i-window:i+1]
            window_returns = returns.iloc[i-window:i+1]
            
            # 窗口內的最高價
            window_cummax = window_prices.cummax()
            
            # 回撤
            window_dd = 1 - window_prices / window_cummax
            drawdown_list.append(window_dd.iloc[-1])
            
            # 回復速度
            if window_dd.iloc[-1] > 0:
                recovery = window_returns.iloc[-1] / window_dd.iloc[-1]
            else:
                recovery = np.nan
            recovery_rate_list.append(recovery)
            
            # 最大回撤
            max_drawdown_list.append(window_dd.max())
        
        drawdown = pd.Series(drawdown_list, index=prices.index)
        recovery_rate = pd.Series(recovery_rate_list, index=prices.index)
        max_drawdown = pd.Series(max_drawdown_list, index=prices.index)
    
    return drawdown, recovery_rate, max_drawdown

# =====================================================
# 3️⃣ Volatility Clustering Index (VCI)
# =====================================================

def volatility_clustering_index(returns, window=60):
    """
    計算波動群聚指數
    
    使用報酬絕對值的自相關係數
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    window : int
        滾動窗口大小
        
    Returns:
    --------
    vci : pd.Series
        波動群聚指數序列
    """
    abs_returns = np.abs(returns)
    
    def calc_vci(x):
        if len(x) < 2:
            return np.nan
        
        # 計算一階自相關
        autocorr = pd.Series(x).autocorr(lag=1)
        return autocorr if not pd.isna(autocorr) else 0.0
    
    vci = abs_returns.rolling(window=window, min_periods=window).apply(calc_vci, raw=False)
    
    return vci

def volatility_clustering_garch_approximation(returns, window=60):
    """
    使用 GARCH(1,1) 近似計算波動群聚指數
    
    簡化版本：使用 α + β 的估計值
    其中 α 和 β 來自簡化的 GARCH 模型
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    window : int
        滾動窗口大小
        
    Returns:
    --------
    vci_garch : pd.Series
        GARCH 近似的波動群聚指數
    """
    def calc_garch_params(x):
        if len(x) < 20:
            return np.nan
        
        # 簡化 GARCH(1,1) 參數估計
        # 使用 ARCH(1) 模型近似
        squared_returns = x ** 2
        lagged_squared = squared_returns[:-1]
        current_squared = squared_returns[1:]
        
        # 簡單線性迴歸：σ²_t = α₀ + α₁ * r²_{t-1}
        if len(lagged_squared) < 10:
            return np.nan
        
        # 計算自相關作為近似
        autocorr = pd.Series(squared_returns).autocorr(lag=1)
        
        # α + β 的近似（簡化）
        alpha_beta_sum = autocorr if not pd.isna(autocorr) else 0.0
        
        return alpha_beta_sum
    
    vci_garch = returns.rolling(window=window, min_periods=window).apply(calc_garch_params, raw=True)
    
    return vci_garch

# =====================================================
# 4️⃣ Entropy of Volume
# =====================================================

def entropy_of_volume(volume, window=60):
    """
    計算成交量資訊熵
    
    對成交量正規化後取香農熵
    
    參數:
    -----------
    volume : pd.Series
        成交量序列
    window : int
        滾動窗口大小
        
    Returns:
    --------
    volume_entropy : pd.Series
        成交量熵序列
    """
    def calc_volume_entropy(x):
        if len(x) == 0 or x.sum() == 0:
            return np.nan
        
        # 正規化（轉為機率分布）
        p = x / x.sum()
        
        # 移除零值（避免 log(0)）
        p = p[p > 0]
        
        if len(p) == 0:
            return 0.0
        
        # 計算香農熵
        entropy = -np.sum(p * np.log2(p))
        
        return entropy
    
    volume_entropy = volume.rolling(window=window, min_periods=window).apply(calc_volume_entropy, raw=True)
    
    return volume_entropy

def entropy_of_volume_binned(volume, window=60, n_bins=10):
    """
    使用分箱方法計算成交量熵（更穩定）
    
    將成交量分為 n_bins 個區間，計算各區間出現頻率的熵
    
    參數:
    -----------
    volume : pd.Series
        成交量序列
    window : int
        滾動窗口大小
    n_bins : int
        分箱數量
        
    Returns:
    --------
    volume_entropy : pd.Series
        成交量熵序列
    """
    def calc_entropy_binned(x):
        if len(x) < n_bins:
            return np.nan
        
        # 分箱
        try:
            bins = pd.cut(x, bins=n_bins, labels=False, duplicates='drop')
            # 計算各箱的頻率
            counts = pd.Series(bins).value_counts()
            p = counts / len(bins)
            
            # 移除零值
            p = p[p > 0]
            
            if len(p) == 0:
                return 0.0
            
            # 計算熵
            entropy = -np.sum(p * np.log2(p))
            
            return entropy
        except:
            return np.nan
    
    volume_entropy = volume.rolling(window=window, min_periods=window).apply(calc_entropy_binned, raw=True)
    
    return volume_entropy

# =====================================================
# 主程式：載入資料並計算所有附加特徵
# =====================================================

file_path = r'C:\Users\a124a\OneDrive\桌面\策略開發\Xgboost_MlFinlab\minute_prices\NAS100.csv'

print("=" * 60)
print("📊 附加延伸特徵計算")
print("=" * 60)

# 載入資料
print(f"\n載入資料: {file_path}")
# 先讀取前10萬筆測試（可根據需要調整）
df = pd.read_csv(file_path, nrows=100000)

# 轉換時間格式
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
else:
    df.index = pd.to_datetime(df.index)

# 選擇價格和成交量欄位
if 'BidClose' in df.columns:
    df['Close'] = df['BidClose']
    if 'BidHigh' in df.columns:
        df['High'] = df['BidHigh']
    if 'BidLow' in df.columns:
        df['Low'] = df['BidLow']
    if 'BidOpen' in df.columns:
        df['Open'] = df['BidOpen']
elif 'Close' not in df.columns:
    raise ValueError("找不到價格欄位")

if 'Volume' not in df.columns:
    if 'BidVolume' in df.columns:
        df['Volume'] = df['BidVolume']
    else:
        raise ValueError("找不到成交量欄位")

print(f"✅ 資料載入成功")
print(f"   資料筆數: {len(df):,}")
print(f"   資料期間: {df.index[0]} 至 {df.index[-1]}")

# 計算報酬
df['Returns'] = df['Close'].pct_change()

# =====================================================
# 1. Skewness / Kurtosis
# =====================================================
print(f"\n1️⃣ 計算 Skewness / Kurtosis...")
df['Skewness_30'], df['Kurtosis_30'] = rolling_skewness_kurtosis(df['Returns'], window=30)
df['Skewness_60'], df['Kurtosis_60'] = rolling_skewness_kurtosis(df['Returns'], window=60)
df['Skewness_240'], df['Kurtosis_60'] = rolling_skewness_kurtosis(df['Returns'], window=240)
print(f"   ✅ 完成")

# =====================================================
# 2. Drawdown / Recovery Rate
# =====================================================
print(f"\n2️⃣ 計算 Drawdown / Recovery Rate...")
# 整體回撤
dd_overall, rr_overall, mdd_overall = drawdown_recovery_rate(
    df['Close'], df['Returns'], window=None
)
df['Drawdown_Overall'] = dd_overall
df['Recovery_Rate_Overall'] = rr_overall
df['Max_Drawdown_Overall'] = mdd_overall

# 滾動回撤
dd_60, rr_60, mdd_60 = drawdown_recovery_rate(
    df['Close'], df['Returns'], window=60
)
df['Drawdown_60'] = dd_60
df['Recovery_Rate_60'] = rr_60
df['Max_Drawdown_60'] = mdd_60

dd_240, rr_240, mdd_240 = drawdown_recovery_rate(
    df['Close'], df['Returns'], window=240
)
df['Drawdown_240'] = dd_240
df['Recovery_Rate_240'] = rr_240
df['Max_Drawdown_240'] = mdd_240
print(f"   ✅ 完成")

# =====================================================
# 3. Volatility Clustering Index (VCI)
# =====================================================
print(f"\n3️⃣ 計算 Volatility Clustering Index...")
df['VCI_30'] = volatility_clustering_index(df['Returns'], window=30)
df['VCI_60'] = volatility_clustering_index(df['Returns'], window=60)
df['VCI_240'] = volatility_clustering_index(df['Returns'], window=240)

# GARCH 近似版本
df['VCI_GARCH_60'] = volatility_clustering_garch_approximation(df['Returns'], window=60)
df['VCI_GARCH_240'] = volatility_clustering_garch_approximation(df['Returns'], window=240)
print(f"   ✅ 完成")

# =====================================================
# 4. Entropy of Volume
# =====================================================
print(f"\n4️⃣ 計算 Entropy of Volume...")
df['Volume_Entropy_30'] = entropy_of_volume(df['Volume'], window=30)
df['Volume_Entropy_60'] = entropy_of_volume(df['Volume'], window=60)
df['Volume_Entropy_240'] = entropy_of_volume(df['Volume'], window=240)

# 分箱版本（更穩定）
df['Volume_Entropy_Binned_60'] = entropy_of_volume_binned(df['Volume'], window=60, n_bins=10)
df['Volume_Entropy_Binned_240'] = entropy_of_volume_binned(df['Volume'], window=240, n_bins=10)
print(f"   ✅ 完成")

# =====================================================
# 結果統計
# =====================================================
print(f"\n" + "=" * 60)
print("✅ 計算完成")
print("=" * 60)

print(f"\n新增欄位:")
print(f"  - Skewness_30/60/240: 偏度（不同窗口）")
print(f"  - Kurtosis_30/60/240: 峰度（不同窗口）")
print(f"  - Drawdown_Overall/60/240: 回撤（整體/滾動）")
print(f"  - Recovery_Rate_Overall/60/240: 回復速度")
print(f"  - Max_Drawdown_Overall/60/240: 最大回撤")
print(f"  - VCI_30/60/240: 波動群聚指數（自相關方法）")
print(f"  - VCI_GARCH_60/240: 波動群聚指數（GARCH 近似）")
print(f"  - Volume_Entropy_30/60/240: 成交量熵")
print(f"  - Volume_Entropy_Binned_60/240: 成交量熵（分箱版本）")

# 顯示統計資訊
print(f"\n特徵統計:")
extended_cols = [
    col for col in df.columns 
    if any(x in col for x in ['Skewness', 'Kurtosis', 'Drawdown', 'Recovery', 'Max_Drawdown', 
                               'VCI', 'Volume_Entropy'])
]
for col in extended_cols:
    valid = df[col].notna().sum()
    if valid > 0:
        mean_val = df[col].mean()
        std_val = df[col].std()
        print(f"  {col}: 有效值 {valid:,}, 平均 {mean_val:.6f}, 標準差 {std_val:.6f}")

# 顯示最後幾筆
print(f"\n最後 5 筆資料:")
display_cols = [
    'Close', 'Volume', 'Returns',
    'Skewness_60', 'Kurtosis_60',
    'Drawdown_60', 'Recovery_Rate_60',
    'VCI_60', 'Volume_Entropy_60'
]
available_cols = [col for col in display_cols if col in df.columns]
print(df[available_cols].tail())

# # 儲存結果
# output_file = 'NAS100_Extended_Features.csv'
# df.to_csv(output_file)
# print(f"\n✅ 已保存到: {output_file}")

量價行為

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 1️⃣ CUSUM Statistic（累積和檢定）
# =====================================================

def cusum_statistic(returns, window=None):
    """
    計算 CUSUM Statistic（累積和檢定）
    
    公式: S_t = Σ(x_i - x̄)
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    window : int, optional
        滾動窗口（如果為 None，則使用整體均值）
        
    Returns:
    --------
    cusum_stat : pd.Series
        CUSUM 統計量
    trigger_flag : pd.Series
        觸發事件標記（當 |S_t - S_{t-1}| > threshold 時）
    """
    if window is None:
        mean_return = returns.mean()
    else:
        mean_return = returns.rolling(window=window).mean()
    
    # 計算累積偏差
    deviation = returns - mean_return
    cusum_stat = deviation.cumsum()
    
    # 計算變化量
    cusum_change = cusum_stat.diff().abs()
    
    return cusum_stat, cusum_change

def cusum_filter_events(returns, threshold, window=None):
    """
    CUSUM Filter - 偵測事件時間點
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    threshold : float
        觸發閾值
    window : int, optional
        滾動窗口
        
    Returns:
    --------
    events : pd.DatetimeIndex
        事件時間點
    """
    _, cusum_change = cusum_statistic(returns, window)
    
    # 觸發事件
    events = cusum_change[cusum_change > threshold].index
    
    return pd.DatetimeIndex(events)

# =====================================================
# 2️⃣ Brown–Durbin–Evans (BDE) CUSUM Test
# =====================================================

def bde_cusum_test(y, X, k=None):
    """
    Brown–Durbin–Evans CUSUM Test（遞迴殘差CUSUM）
    
    檢測迴歸模型係數是否隨時間改變
    
    參數:
    -----------
    y : pd.Series or np.array
        因變數
    X : pd.DataFrame or np.array
        自變數（需包含常數項）
    k : int, optional
        初始樣本數（如果為 None，使用 X.shape[1] + 1）
        
    Returns:
    --------
    cusum_stat : pd.Series
        CUSUM 統計量
    break_flag : pd.Series
        結構變動標記
    """
    y = np.array(y)
    X = np.array(X)
    
    if len(y) != len(X):
        raise ValueError("y 和 X 長度不一致")
    
    n = len(y)
    if k is None:
        k = X.shape[1] + 1
    
    # 儲存遞迴殘差
    recursive_residuals = []
    cusum_stats = []
    
    # 從 k+1 開始計算遞迴殘差
    for t in range(k, n):
        # 使用前 t 個觀測值估計參數
        y_t = y[:t]
        X_t = X[:t]
        
        try:
            # OLS 估計
            model = OLS(y_t, X_t).fit()
            
            # 預測第 t+1 期
            x_t1 = X[t].reshape(1, -1)
            y_pred = model.predict(x_t1)[0]
            
            # 遞迴殘差
            residual = y[t] - y_pred
            
            # 標準化
            # 簡化版本：使用殘差標準差
            if t > k:
                sigma_hat = np.std(recursive_residuals)
            else:
                sigma_hat = model.resid.std()
            
            if sigma_hat > 0:
                standardized_residual = residual / sigma_hat
            else:
                standardized_residual = 0
            
            recursive_residuals.append(standardized_residual)
            
            # CUSUM 統計量
            cusum = np.sum(recursive_residuals) / np.sqrt(len(recursive_residuals))
            cusum_stats.append(cusum)
            
        except:
            recursive_residuals.append(0)
            cusum_stats.append(0)
    
    # 建立結果 Series
    index = pd.RangeIndex(start=k, stop=n)
    cusum_stat = pd.Series(cusum_stats, index=index)
    
    # 臨界值（簡化：使用 ±2 作為近似）
    critical_value = 2.0
    break_flag = (cusum_stat.abs() > critical_value).astype(int)
    
    return cusum_stat, break_flag

# =====================================================
# 3️⃣ Chu–Stinchcombe–White (CSW) CUSUM Test
# =====================================================

def csw_cusum_test(log_prices, n_start=1, alpha=0.05):
    """
    Chu–Stinchcombe–White CUSUM Test（價格水準CUSUM）
    
    檢測 log-price 是否偏離穩態
    
    參數:
    -----------
    log_prices : pd.Series
        Log 價格序列
    n_start : int
        起始點 n
    alpha : float
        顯著水準（預設 0.05，對應 b_alpha ≈ 4.6）
        
    Returns:
    --------
    cusum_stat : pd.Series
        CUSUM 統計量 S_{n,t}
    break_flag : pd.Series
        結構變動標記
    """
    y = log_prices.values
    n = len(y)
    
    # b_alpha 值（根據文獻，alpha=0.05 時 b≈4.6）
    b_alpha_dict = {0.05: 4.6, 0.01: 5.2, 0.10: 4.0}
    b_alpha = b_alpha_dict.get(alpha, 4.6)
    
    cusum_stats = []
    break_flags = []
    
    # 計算每個 t 的 S_{n,t}
    for t in range(n_start + 1, n):
        # 計算 σ̂_t^2
        delta_y = np.diff(y[1:t+1])
        sigma_sq = np.var(delta_y) if len(delta_y) > 0 else 1.0
        sigma_t = np.sqrt(sigma_sq) if sigma_sq > 0 else 1.0
        
        # 計算 S_{n,t}
        y_t = y[t]
        y_n = y[n_start]
        t_minus_n = t - n_start
        
        if sigma_t > 0 and t_minus_n > 0:
            s_n_t = (y_t - y_n) / (sigma_t * np.sqrt(t_minus_n))
        else:
            s_n_t = 0
        
        cusum_stats.append(s_n_t)
        
        # 臨界值 c_alpha[n,t] = sqrt(b_alpha + log(t-n))
        c_alpha = np.sqrt(b_alpha + np.log(t_minus_n))
        
        # 檢定
        break_flag = 1 if abs(s_n_t) > c_alpha else 0
        break_flags.append(break_flag)
    
    # 建立結果 Series
    index = log_prices.index[n_start + 1:]
    cusum_stat = pd.Series(cusum_stats, index=index)
    break_flag = pd.Series(break_flags, index=index)
    
    return cusum_stat, break_flag

# =====================================================
# 4️⃣ SADF / GSADF Tests（泡沫檢定）
# =====================================================

def chow_df_test(log_prices, tau_star):
    """
    Chow-type Dickey–Fuller Test
    
    測試從隨機漫步 → 爆發過程的轉折點
    
    參數:
    -----------
    log_prices : pd.Series
        Log 價格序列
    tau_star : int
        假設的斷點位置
        
    Returns:
    --------
    df_stat : float
        Dickey-Fuller 統計量
    p_value : float
        p 值
    """
    y = log_prices.values
    n = len(y)
    
    if tau_star >= n - 1 or tau_star < 1:
        return np.nan, np.nan
    
    # 建立虛擬變數 D_t[tau*]
    D = np.zeros(n)
    D[tau_star:] = 1
    
    # 建立迴歸變數
    delta_y = np.diff(y)
    y_lag = y[:-1]
    D_lag = D[:-1]
    interaction = y_lag * D_lag
    
    # 建立 DataFrame
    df_reg = pd.DataFrame({
        'delta_y': delta_y,
        'y_lag': y_lag,
        'interaction': interaction
    }).dropna()
    
    if len(df_reg) < 10:
        return np.nan, np.nan
    
    try:
        # OLS 迴歸
        X = df_reg[['y_lag', 'interaction']].values
        y_reg = df_reg['delta_y'].values
        
        model = OLS(y_reg, X).fit()
        
        # 檢定 interaction 係數（對應 δ）
        coeff = model.params[1]  # interaction 係數
        std_err = model.bse[1]
        
        if std_err > 0:
            df_stat = coeff / std_err
        else:
            df_stat = 0
        
        # 簡化 p 值計算（使用 t 檢定）
        p_value = 2 * (1 - stats.t.cdf(abs(df_stat), df=model.df_resid))
        
        return df_stat, p_value
    
    except:
        return np.nan, np.nan

def sadf_test(log_prices, min_window=20, max_window=None):
    """
    SADF (Supremum Augmented Dickey–Fuller) Test
    
    用不斷擴展的視窗找出最大右尾 DF 統計量
    
    參數:
    -----------
    log_prices : pd.Series
        Log 價格序列
    min_window : int
        最小窗口大小
    max_window : int, optional
        最大窗口大小（如果為 None，使用 len(log_prices)）
        
    Returns:
    --------
    sadf_stat : float
        SADF 統計量
    sadf_scores : pd.Series
        每個時間點的 SADF 分數
    """
    y = log_prices.values
    n = len(y)
    
    if max_window is None:
        max_window = n
    
    sadf_scores = []
    
    # 對每個結束點 t，計算擴展窗口的 SADF
    for t in range(min_window, n):
        # 從 1 到 t-min_window 擴展窗口
        max_df = -np.inf
        
        for t0 in range(1, t - min_window + 1):
            # 提取子序列
            y_sub = y[t0:t+1]
            
            if len(y_sub) < min_window:
                continue
            
            # 計算 ADF 統計量（簡化版本）
            try:
                delta_y = np.diff(y_sub)
                y_lag = y_sub[:-1]
                
                if len(delta_y) < 5:
                    continue
                
                # 簡化 ADF：檢定 y_lag 係數
                X = y_lag.reshape(-1, 1)
                model = OLS(delta_y, X).fit()
                
                coeff = model.params[0]
                std_err = model.bse[0] if model.bse[0] > 0 else 1.0
                
                df_stat = coeff / std_err
                max_df = max(max_df, df_stat)
                
            except:
                continue
        
        sadf_scores.append(max_df if max_df > -np.inf else 0)
    
    # SADF 統計量 = 所有時間點的最大值
    sadf_stat = max(sadf_scores) if sadf_scores else 0
    
    # 建立結果 Series
    index = log_prices.index[min_window:]
    sadf_scores_series = pd.Series(sadf_scores, index=index)
    
    return sadf_stat, sadf_scores_series

def gsadf_test(log_prices, min_window=20, max_window=None):
    """
    GSADF (Generalized SADF) Test
    
    允許滾動與擴展視窗同時移動，可偵測多重泡沫
    
    參數:
    -----------
    log_prices : pd.Series
        Log 價格序列
    min_window : int
        最小窗口大小
    max_window : int, optional
        最大窗口大小
        
    Returns:
    --------
    gsadf_stat : float
        GSADF 統計量
    gsadf_scores : pd.Series
        每個時間點的 GSADF 分數
    """
    y = log_prices.values
    n = len(y)
    
    if max_window is None:
        max_window = n
    
    gsadf_scores = []
    
    # 對每個結束點 t
    for t in range(min_window, n):
        max_df = -np.inf
        
        # 允許滾動窗口：r0 和 r1 都可以移動
        for r0 in range(0, t - min_window + 1):
            for r1 in range(r0 + min_window, t + 1):
                y_sub = y[r0:r1+1]
                
                if len(y_sub) < min_window:
                    continue
                
                try:
                    delta_y = np.diff(y_sub)
                    y_lag = y_sub[:-1]
                    
                    if len(delta_y) < 5:
                        continue
                    
                    X = y_lag.reshape(-1, 1)
                    model = OLS(delta_y, X).fit()
                    
                    coeff = model.params[0]
                    std_err = model.bse[0] if model.bse[0] > 0 else 1.0
                    
                    df_stat = coeff / std_err
                    max_df = max(max_df, df_stat)
                    
                except:
                    continue
        
        gsadf_scores.append(max_df if max_df > -np.inf else 0)
    
    gsadf_stat = max(gsadf_scores) if gsadf_scores else 0
    
    index = log_prices.index[min_window:]
    gsadf_scores_series = pd.Series(gsadf_scores, index=index)
    
    return gsadf_stat, gsadf_scores_series

# =====================================================
# 5️⃣ Structural Break Regime Label
# =====================================================

def structural_break_regime_label(df, methods=['cusum', 'csw', 'sadf']):
    """
    結合多種檢定結果，產生結構變動 regime 標籤
    
    參數:
    -----------
    df : pd.DataFrame
        需包含價格資料和各種檢定結果
    methods : list
        使用的檢定方法
        
    Returns:
    --------
    regime : pd.Series
        Regime 標籤（0=穩定, 1=結構變動）
    """
    regime = pd.Series(0, index=df.index)
    
    # 結合各種檢定結果
    if 'cusum' in methods and 'CUSUM_Break' in df.columns:
        regime = regime | df['CUSUM_Break']
    
    if 'csw' in methods and 'CSW_Break' in df.columns:
        regime = regime | df['CSW_Break']
    
    if 'sadf' in methods and 'SADF_Break' in df.columns:
        regime = regime | df['SADF_Break']
    
    return regime

# =====================================================
# 主程式：載入資料並計算所有檢定
# =====================================================

file_path = r'C:\Users\a124a\OneDrive\桌面\策略開發\Xgboost_MlFinlab\minute_prices\NAS100.csv'

print("=" * 60)
print("📊 結構性變動檢測")
print("=" * 60)

# 載入資料
print(f"\n載入資料: {file_path}")
df = pd.read_csv(file_path)

# 轉換時間格式
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
else:
    df.index = pd.to_datetime(df.index)

# 選擇價格欄位
if 'BidClose' in df.columns:
    df['Close'] = df['BidClose']
elif 'Close' not in df.columns:
    raise ValueError("找不到價格欄位")

print(f"✅ 資料載入成功")
print(f"   資料筆數: {len(df):,}")
print(f"   資料期間: {df.index[0]} 至 {df.index[-1]}")

# 計算 log 價格和報酬
df['Log_Price'] = np.log(df['Close'])
df['Returns'] = df['Close'].pct_change()

# =====================================================
# 1. CUSUM Statistic
# =====================================================
print(f"\n1️⃣ 計算 CUSUM Statistic...")
cusum_stat, cusum_change = cusum_statistic(df['Returns'].dropna(), window=100)
df['CUSUM_Stat'] = cusum_stat
df['CUSUM_Change'] = cusum_change

# CUSUM 事件
cusum_threshold = df['Returns'].std() * 2
cusum_events = cusum_filter_events(df['Returns'].dropna(), cusum_threshold, window=100)
df['CUSUM_Event'] = 0
df.loc[cusum_events, 'CUSUM_Event'] = 1
df['CUSUM_Break'] = (df['CUSUM_Change'] > cusum_threshold).astype(int)

print(f"   CUSUM 事件數: {df['CUSUM_Event'].sum():,}")

# =====================================================
# 2. CSW CUSUM Test
# =====================================================
# print(f"\n2️⃣ 計算 CSW CUSUM Test...")
# csw_stat, csw_break = csw_cusum_test(df['Log_Price'].dropna(), n_start=100, alpha=0.05)
# df['CSW_Stat'] = csw_stat
# df['CSW_Break'] = csw_break
# print(f"   CSW 結構變動數: {df['CSW_Break'].sum():,}")

# =====================================================
# 3. SADF Test
# =====================================================
# print(f"\n3️⃣ 計算 SADF Test...")
# sadf_stat, sadf_scores = sadf_test(df['Log_Price'].dropna(), min_window=60, max_window=1440)
# df['SADF_Score'] = sadf_scores
# # 使用臨界值判斷（簡化：使用分位數）
# sadf_critical = sadf_scores.quantile(0.95)
# df['SADF_Break'] = (df['SADF_Score'] > sadf_critical).astype(int)
# print(f"   SADF 統計量: {sadf_stat:.4f}")
# print(f"   SADF 結構變動數: {df['SADF_Break'].sum():,}")

# =====================================================
# 4. GSADF Test（可選，計算較慢）
# =====================================================
# print(f"\n4️⃣ 計算 GSADF Test...")
# print("   ⚠️ GSADF 計算較慢，使用較小窗口...")
# gsadf_stat, gsadf_scores = gsadf_test(df['Log_Price'].dropna(), min_window=60, max_window=240)
# df['GSADF_Score'] = gsadf_scores
# gsadf_critical = gsadf_scores.quantile(0.95)
# df['GSADF_Break'] = (df['GSADF_Score'] > gsadf_critical).astype(int)
# print(f"   GSADF 統計量: {gsadf_stat:.4f}")
# print(f"   GSADF 結構變動數: {df['GSADF_Break'].sum():,}")

# =====================================================
# 5. Structural Break Regime Label
# =====================================================
print(f"\n5️⃣ 產生 Structural Break Regime Label...")
df['Regime'] = structural_break_regime_label(df, methods=['cusum', 'csw', 'sadf'])
print(f"   結構變動期間數: {df['Regime'].sum():,}")
print(f"   穩定期間數: {(df['Regime'] == 0).sum():,}")

# =====================================================
# 結果統計
# =====================================================
print(f"\n" + "=" * 60)
print("✅ 計算完成")
print("=" * 60)
print(f"\n新增欄位:")
print(f"  - CUSUM_Stat: CUSUM 統計量")
print(f"  - CUSUM_Event: CUSUM 事件標記")
print(f"  - CUSUM_Break: CUSUM 結構變動標記")
print(f"  - CSW_Stat: CSW CUSUM 統計量")
print(f"  - CSW_Break: CSW 結構變動標記")
print(f"  - SADF_Score: SADF 分數")
print(f"  - SADF_Break: SADF 結構變動標記")
print(f"  - GSADF_Score: GSADF 分數")
print(f"  - GSADF_Break: GSADF 結構變動標記")
print(f"  - Regime: 綜合結構變動標籤 (0=穩定, 1=變動)")

# 顯示最後幾筆
print(f"\n最後 5 筆資料:")
display_cols = ['Close', 'CUSUM_Break', 'CSW_Break', 'SADF_Break', 'Regime']
available_cols = [col for col in display_cols if col in df.columns]
print(df[available_cols].tail())

# # 儲存結果
# output_file = 'NAS100_Structural_Breaks.csv'
# df.to_csv(output_file)
# print(f"\n✅ 已保存到: {output_file}")

資訊熵

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 1️⃣ Shannon Entropy of Return Signs
# =====================================================

def shannon_entropy_return_signs(returns, window=30):
    """
    計算報酬符號的香農熵
    
    公式: H = -Σ p_i * log2(p_i)
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    window : int
        滾動窗口大小（分鐘數）
        
    Returns:
    --------
    entropy : pd.Series
        香農熵序列
    """
    # 將報酬轉為符號（+1, -1, 0）
    signs = returns.copy()
    signs[signs > 0] = 1
    signs[signs < 0] = -1
    signs[signs == 0] = 0
    
    entropy_series = []
    
    for i in range(len(returns)):
        if i < window:
            entropy_series.append(np.nan)
            continue
        
        # 提取窗口內的符號
        window_signs = signs.iloc[i-window:i]
        
        # 計算各符號出現機率
        p_up = (window_signs == 1).sum() / len(window_signs)
        p_down = (window_signs == -1).sum() / len(window_signs)
        p_zero = (window_signs == 0).sum() / len(window_signs)
        
        # 計算熵（避免 log(0)）
        entropy = 0
        if p_up > 0:
            entropy -= p_up * np.log2(p_up)
        if p_down > 0:
            entropy -= p_down * np.log2(p_down)
        if p_zero > 0:
            entropy -= p_zero * np.log2(p_zero)
        
        entropy_series.append(entropy)
    
    return pd.Series(entropy_series, index=returns.index)

def shannon_entropy_rolling(returns, window=30):
    """
    使用滾動窗口計算香農熵（更高效版本）
    """
    # 轉為符號
    signs = np.sign(returns)
    
    # 使用滾動窗口計算機率
    def calc_entropy(x):
        p_up = (x == 1).sum() / len(x)
        p_down = (x == -1).sum() / len(x)
        p_zero = (x == 0).sum() / len(x)
        
        entropy = 0
        if p_up > 0:
            entropy -= p_up * np.log2(p_up)
        if p_down > 0:
            entropy -= p_down * np.log2(p_down)
        if p_zero > 0:
            entropy -= p_zero * np.log2(p_zero)
        
        return entropy
    
    entropy = signs.rolling(window=window, min_periods=window).apply(calc_entropy, raw=True)
    
    return entropy

# =====================================================
# 2️⃣ Lempel–Ziv Entropy
# =====================================================

def lempel_ziv_entropy(sequence):
    """
    計算 Lempel–Ziv 熵
    
    參數:
    -----------
    sequence : array-like
        二進位序列（0/1 或 -1/1）
        
    Returns:
    --------
    h_lz : float
        Lempel–Ziv 熵
    """
    # 轉為字串
    if isinstance(sequence, pd.Series):
        seq = sequence.values
    else:
        seq = np.array(sequence)
    
    # 轉為 0/1 二進位
    binary_seq = (seq > 0).astype(int)
    n = len(binary_seq)
    
    if n == 0:
        return 0
    
    # 計算最短不重複片段數 c(n)
    i = 0
    c = 0
    substrings = set()
    
    while i < n:
        j = i + 1
        found = False
        
        while j <= n:
            substring = tuple(binary_seq[i:j])
            
            if substring not in substrings:
                substrings.add(substring)
                c += 1
                i = j
                found = True
                break
            j += 1
        
        if not found:
            i += 1
    
    # 計算 LZ 熵
    if n > 0:
        h_lz = (c * np.log2(n)) / n
    else:
        h_lz = 0
    
    return h_lz

def lempel_ziv_entropy_rolling(returns, window=60):
    """
    滾動窗口計算 Lempel–Ziv 熵
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    window : int
        滾動窗口大小
        
    Returns:
    --------
    lz_entropy : pd.Series
        LZ 熵序列
    """
    lz_series = []
    
    for i in range(len(returns)):
        if i < window:
            lz_series.append(np.nan)
            continue
        
        window_returns = returns.iloc[i-window:i]
        lz_val = lempel_ziv_entropy(window_returns)
        lz_series.append(lz_val)
    
    return pd.Series(lz_series, index=returns.index)

# =====================================================
# 3️⃣ Hurst Exponent
# =====================================================

def hurst_exponent_rs(series, max_lag=None):
    """
    使用 R/S (Rescaled Range) 分析計算 Hurst 指數
    
    參數:
    -----------
    series : pd.Series
        時間序列（log price 或 return）
    max_lag : int, optional
        最大滯後期數（如果為 None，使用 len(series) // 4）
        
    Returns:
    --------
    H : float
        Hurst 指數
    """
    n = len(series)
    if max_lag is None:
        max_lag = n // 4
    
    if max_lag < 2:
        return 0.5
    
    # 計算不同 lag 的 R/S
    lags = []
    rs_values = []
    
    for lag in range(2, min(max_lag, n // 2)):
        # 將序列分成多個子區間
        n_segments = n // lag
        if n_segments < 2:
            continue
        
        rs_list = []
        
        for i in range(n_segments):
            segment = series.iloc[i*lag:(i+1)*lag]
            
            if len(segment) < 2:
                continue
            
            # 計算均值
            mean_seg = segment.mean()
            
            # 計算累積偏差
            cumdev = (segment - mean_seg).cumsum()
            
            # Range
            R = cumdev.max() - cumdev.min()
            
            # Standard deviation
            S = segment.std()
            
            if S > 0:
                rs_list.append(R / S)
        
        if rs_list:
            lags.append(lag)
            rs_values.append(np.mean(rs_list))
    
    if len(lags) < 2:
        return 0.5
    
    # 線性迴歸：log(R/S) vs log(lag)
    log_lags = np.log(lags)
    log_rs = np.log(rs_values)
    
    # 移除無效值
    valid = np.isfinite(log_lags) & np.isfinite(log_rs)
    if valid.sum() < 2:
        return 0.5
    
    log_lags_clean = log_lags[valid]
    log_rs_clean = log_rs[valid]
    
    # 線性迴歸
    slope, intercept = np.polyfit(log_lags_clean, log_rs_clean, 1)
    H = slope
    
    return H

def hurst_exponent_rolling(series, window=240, max_lag=None):
    """
    滾動窗口計算 Hurst 指數
    
    參數:
    -----------
    series : pd.Series
        時間序列
    window : int
        滾動窗口大小
    max_lag : int, optional
        最大滯後期數
        
    Returns:
    --------
    hurst : pd.Series
        Hurst 指數序列
    """
    hurst_series = []
    
    for i in range(len(series)):
        if i < window:
            hurst_series.append(np.nan)
            continue
        
        window_series = series.iloc[i-window:i]
        
        if len(window_series) < 10:
            hurst_series.append(np.nan)
            continue
        
        try:
            H = hurst_exponent_rs(window_series, max_lag=max_lag)
            hurst_series.append(H)
        except:
            hurst_series.append(np.nan)
    
    return pd.Series(hurst_series, index=series.index)

def hurst_exponent_dfa(series, window=240):
    """
    使用 DFA (Detrended Fluctuation Analysis) 計算 Hurst 指數
    
    參數:
    -----------
    series : pd.Series
        時間序列
    window : int
        滾動窗口大小
        
    Returns:
    --------
    hurst : pd.Series
        Hurst 指數序列
    """
    def dfa_analysis(x):
        n = len(x)
        if n < 10:
            return 0.5
        
        # 累積和
        y = np.cumsum(x - x.mean())
        
        # 計算不同尺度下的波動
        scales = np.logspace(1, np.log10(n//4), 10).astype(int)
        scales = scales[scales < n//4]
        scales = scales[scales > 1]
        
        if len(scales) < 2:
            return 0.5
        
        fluctuations = []
        
        for scale in scales:
            n_segments = n // scale
            if n_segments < 2:
                continue
            
            f_list = []
            
            for i in range(n_segments):
                segment = y[i*scale:(i+1)*scale]
                
                # 去趨勢（線性）
                x_seg = np.arange(len(segment))
                coeffs = np.polyfit(x_seg, segment, 1)
                trend = np.polyval(coeffs, x_seg)
                detrended = segment - trend
                
                f_list.append(np.sqrt(np.mean(detrended**2)))
            
            if f_list:
                fluctuations.append(np.mean(f_list))
        
        if len(fluctuations) < 2:
            return 0.5
        
        # 線性迴歸：log(F) vs log(scale)
        log_scales = np.log(scales[:len(fluctuations)])
        log_fluct = np.log(fluctuations)
        
        valid = np.isfinite(log_scales) & np.isfinite(log_fluct)
        if valid.sum() < 2:
            return 0.5
        
        slope, _ = np.polyfit(log_scales[valid], log_fluct[valid], 1)
        H = slope
        
        return H
    
    hurst = series.rolling(window=window, min_periods=window).apply(dfa_analysis, raw=True)
    
    return hurst

# =====================================================
# 4️⃣ Market Efficiency Index (MEI)
# =====================================================

def market_efficiency_index(returns, window=30, entropy_window=30):
    """
    計算市場效率指數
    
    MEI = (H_Shannon / H_max) * σ_returns
    
    或簡化版本：
    MEI = entropy * returns.std()
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    window : int
        計算波動率的窗口大小
    entropy_window : int
        計算熵的窗口大小
        
    Returns:
    --------
    mei : pd.Series
        市場效率指數
    """
    # 計算香農熵
    entropy = shannon_entropy_rolling(returns, window=entropy_window)
    
    # 計算波動率
    volatility = returns.rolling(window=window).std()
    
    # 計算 MEI
    # 方法1：簡化版本
    mei = entropy * volatility
    
    # 方法2：標準化版本（可選）
    # H_max = np.log2(3)  # 三種符號（+1, -1, 0）的最大熵
    # mei_normalized = (entropy / H_max) * volatility
    
    return mei

# =====================================================
# 主程式：載入資料並計算所有熵特徵
# =====================================================

file_path = r'C:\Users\a124a\OneDrive\桌面\策略開發\Xgboost_MlFinlab\minute_prices\NAS100.csv'

print("=" * 60)
print("📊 Entropy Features 計算")
print("=" * 60)

# 載入資料
print(f"\n載入資料: {file_path}")
df = pd.read_csv(file_path, nrows=100000)  # 先讀取前10萬筆測試

# 轉換時間格式
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
else:
    df.index = pd.to_datetime(df.index)

# 選擇價格欄位
if 'BidClose' in df.columns:
    df['Close'] = df['BidClose']
elif 'Close' not in df.columns:
    raise ValueError("找不到價格欄位")

print(f"✅ 資料載入成功")
print(f"   資料筆數: {len(df):,}")
print(f"   資料期間: {df.index[0]} 至 {df.index[-1]}")

# 計算 log 價格和報酬
df['Log_Price'] = np.log(df['Close'])
df['Returns'] = df['Close'].pct_change()

# =====================================================
# 1. Shannon Entropy of Return Signs
# =====================================================
print(f"\n1️⃣ 計算 Shannon Entropy of Return Signs...")
df['Shannon_Entropy_30'] = shannon_entropy_rolling(df['Returns'], window=30)
df['Shannon_Entropy_60'] = shannon_entropy_rolling(df['Returns'], window=60)
df['Shannon_Entropy_240'] = shannon_entropy_rolling(df['Returns'], window=240)
print(f"   ✅ 完成")

# =====================================================
# 2. Lempel–Ziv Entropy
# =====================================================
print(f"\n2️⃣ 計算 Lempel–Ziv Entropy...")
print("   ⚠️ LZ 熵計算較慢，使用較小窗口...")
df['LZ_Entropy_60'] = lempel_ziv_entropy_rolling(df['Returns'], window=60)
df['LZ_Entropy_240'] = lempel_ziv_entropy_rolling(df['Returns'], window=240)
print(f"   ✅ 完成")

# =====================================================
# 3. Hurst Exponent
# =====================================================
# print(f"\n3️⃣ 計算 Hurst Exponent (R/S 方法)...")
# print("   ⚠️ Hurst 計算較慢，使用較小窗口...")
# df['Hurst_RS_240'] = hurst_exponent_rolling(df['Log_Price'], window=240, max_lag=60)
# df['Hurst_RS_Returns_240'] = hurst_exponent_rolling(df['Returns'], window=240, max_lag=60)
# print(f"   ✅ 完成")

# print(f"\n3️⃣ 計算 Hurst Exponent (DFA 方法)...")
# df['Hurst_DFA_240'] = hurst_exponent_dfa(df['Returns'], window=240)
# print(f"   ✅ 完成")

# =====================================================
# 4. Market Efficiency Index
# =====================================================
print(f"\n4️⃣ 計算 Market Efficiency Index...")
df['MEI_30'] = market_efficiency_index(df['Returns'], window=30, entropy_window=30)
df['MEI_60'] = market_efficiency_index(df['Returns'], window=60, entropy_window=60)
df['MEI_240'] = market_efficiency_index(df['Returns'], window=240, entropy_window=240)
print(f"   ✅ 完成")

# =====================================================
# 結果統計
# =====================================================
print(f"\n" + "=" * 60)
print("✅ 計算完成")
print("=" * 60)

print(f"\n新增欄位:")
print(f"  - Shannon_Entropy_30/60/240: 香農熵（不同窗口）")
print(f"  - LZ_Entropy_60/240: Lempel–Ziv 熵")
print(f"  - Hurst_RS_240: Hurst 指數（R/S 方法，log price）")
print(f"  - Hurst_RS_Returns_240: Hurst 指數（R/S 方法，returns）")
print(f"  - Hurst_DFA_240: Hurst 指數（DFA 方法）")
print(f"  - MEI_30/60/240: 市場效率指數")

# 顯示統計資訊
print(f"\n特徵統計:")
entropy_cols = [col for col in df.columns if 'Entropy' in col or 'Hurst' in col or 'MEI' in col]
for col in entropy_cols:
    valid = df[col].notna().sum()
    if valid > 0:
        mean_val = df[col].mean()
        std_val = df[col].std()
        print(f"  {col}: 有效值 {valid:,}, 平均 {mean_val:.4f}, 標準差 {std_val:.4f}")

# 顯示最後幾筆
print(f"\n最後 5 筆資料:")
display_cols = ['Close', 'Returns', 'Shannon_Entropy_30', 'LZ_Entropy_60', 'Hurst_RS_240', 'MEI_30']
available_cols = [col for col in display_cols if col in df.columns]
print(df[available_cols].tail())

# # 儲存結果
# output_file = 'NAS100_Entropy_Features.csv'
# df.to_csv(output_file)
# print(f"\n✅ 已保存到: {output_file}")

市場微結構

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 1️⃣ Roll Spread Estimate
# =====================================================

def roll_spread_estimate(returns, window=60):
    """
    計算 Roll 隱含買賣價差
    
    公式: S_Roll = 2 * sqrt(-Cov(r_t, r_{t-1}))
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    window : int
        滾動窗口大小（分鐘數）
        
    Returns:
    --------
    roll_spread : pd.Series
        Roll 價差序列
    """
    def calc_roll_spread(x):
        if len(x) < 2:
            return np.nan
        
        # 計算一階自相關
        autocorr = x.autocorr(lag=1)
        
        if pd.isna(autocorr) or autocorr >= 0:
            return 0.0
        
        # Roll spread
        roll_spread = 2 * np.sqrt(-autocorr)
        return roll_spread
    
    roll_spread = returns.rolling(window=window, min_periods=window).apply(calc_roll_spread, raw=False)
    
    return roll_spread

# =====================================================
# 2️⃣ Amihud Illiquidity
# =====================================================

def amihud_illiquidity(returns, volume, window=60):
    """
    計算 Amihud 流動性缺乏指標
    
    公式: I_t = |r_t| / V_t
    平均: Amihud = (1/N) * Σ |r_t| / V_t
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    volume : pd.Series
        成交量序列
    window : int
        滾動窗口大小
        
    Returns:
    --------
    amihud : pd.Series
        Amihud 指標序列
    """
    # 避免除以零
    volume_safe = volume.replace(0, np.nan)
    
    # 計算每期的 Amihud
    amihud_daily = np.abs(returns) / volume_safe
    
    # 滾動平均
    amihud = amihud_daily.rolling(window=window, min_periods=1).mean()
    
    return amihud

# =====================================================
# 3️⃣ Kyle's λ (Kyle Lambda)
# =====================================================

def kyle_lambda(returns, volume, window=60):
    """
    計算 Kyle Lambda（價格衝擊係數）
    
    公式: λ = Cov(r_t, q_t) / Var(q_t)
    其中 q_t 為 signed volume
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    volume : pd.Series
        成交量序列
    window : int
        滾動窗口大小
        
    Returns:
    --------
    kyle_lambda : pd.Series
        Kyle Lambda 序列
    """
    # 計算 signed volume（用報酬符號推斷方向）
    signed_volume = np.sign(returns) * volume
    
    def calc_kyle_lambda(window_data):
        rets = window_data['returns'].values
        signed_vols = window_data['signed_volume'].values
        
        # 移除 NaN
        valid = ~(np.isnan(rets) | np.isnan(signed_vols))
        rets_clean = rets[valid]
        signed_vols_clean = signed_vols[valid]
        
        if len(rets_clean) < 10:
            return np.nan
        
        # 計算共變異數和變異數
        cov = np.cov(rets_clean, signed_vols_clean)[0, 1]
        var = np.var(signed_vols_clean)
        
        if var == 0 or np.isnan(cov) or np.isnan(var):
            return np.nan
        
        kyle_lambda = cov / var
        return kyle_lambda
    
    # 建立臨時 DataFrame
    temp_df = pd.DataFrame({
        'returns': returns,
        'signed_volume': signed_volume
    })
    
    kyle_lambda_series = []
    
    for i in range(len(returns)):
        if i < window:
            kyle_lambda_series.append(np.nan)
            continue
        
        window_data = temp_df.iloc[i-window:i]
        kyle_val = calc_kyle_lambda(window_data)
        kyle_lambda_series.append(kyle_val)
    
    return pd.Series(kyle_lambda_series, index=returns.index)

# =====================================================
# 4️⃣ Order Flow Imbalance (OFI)
# =====================================================

def order_flow_imbalance(returns, volume=None, window=30):
    """
    計算訂單流不均衡
    
    簡化版本: OFI_t ≈ sign(r_t)
    含成交量版本: OFI_t = (volume * sign(returns)).rolling().mean()
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    volume : pd.Series, optional
        成交量序列（如果提供，會使用加權版本）
    window : int
        滾動窗口大小
        
    Returns:
    --------
    ofi : pd.Series
        Order Flow Imbalance 序列
    """
    # 基本版本：使用報酬符號
    ofi_basic = np.sign(returns)
    
    if volume is not None:
        # 加權版本：使用成交量加權
        signed_volume = np.sign(returns) * volume
        ofi = signed_volume.rolling(window=window, min_periods=1).mean()
    else:
        # 簡單版本：滾動平均符號
        ofi = ofi_basic.rolling(window=window, min_periods=1).mean()
    
    return ofi

# =====================================================
# 5️⃣ Volume Imbalance Ratio (VIR)
# =====================================================

def volume_imbalance_ratio(returns, volume, window=60):
    """
    計算成交量不對稱比
    
    公式: VIR = (V_up - V_down) / (V_up + V_down)
    
    參數:
    -----------
    returns : pd.Series
        報酬序列
    volume : pd.Series
        成交量序列
    window : int
        滾動窗口大小
        
    Returns:
    --------
    vir : pd.Series
        Volume Imbalance Ratio 序列
    """
    def calc_vir(window_data):
        rets = window_data['returns'].values
        vols = window_data['volume'].values
        
        # 計算上漲和下跌的成交量
        v_up = vols[rets > 0].sum()
        v_down = vols[rets < 0].sum()
        
        total = v_up + v_down
        
        if total == 0:
            return 0.0
        
        vir = (v_up - v_down) / total
        return vir
    
    # 建立臨時 DataFrame
    temp_df = pd.DataFrame({
        'returns': returns,
        'volume': volume
    })
    
    vir_series = []
    
    for i in range(len(returns)):
        if i < window:
            vir_series.append(np.nan)
            continue
        
        window_data = temp_df.iloc[i-window:i]
        vir_val = calc_vir(window_data)
        vir_series.append(vir_val)
    
    return pd.Series(vir_series, index=returns.index)

# =====================================================
# 6️⃣ Realized Spread / Price Impact
# =====================================================

def realized_spread_price_impact(prices, returns, delta_minutes=5):
    """
    計算 Realized Spread 和 Price Impact
    
    Realized Spread: RS_t = 2 * (p_t - p_{t+Δ}) * d_t
    Price Impact: PI_t = (p_{t+Δ} - p_t) * d_t
    
    其中 d_t = sign(r_t) 為交易方向
    
    參數:
    -----------
    prices : pd.Series
        價格序列
    returns : pd.Series
        報酬序列
    delta_minutes : int
        延後時間（分鐘數）
        
    Returns:
    --------
    realized_spread : pd.Series
        Realized Spread 序列
    price_impact : pd.Series
        Price Impact 序列
    """
    # 交易方向
    direction = np.sign(returns)
    
    # 延後價格
    future_prices = prices.shift(-delta_minutes)
    
    # 計算價格變動
    price_change = future_prices - prices
    
    # Realized Spread
    realized_spread = 2 * price_change * direction
    
    # Price Impact
    price_impact = price_change * direction
    
    return realized_spread, price_impact

# =====================================================
# 主程式：載入資料並計算所有微結構特徵
# =====================================================

file_path = r'C:\Users\a124a\OneDrive\桌面\策略開發\Xgboost_MlFinlab\minute_prices\NAS100.csv'

print("=" * 60)
print("📊 Microstructure Features 計算")
print("=" * 60)

# 載入資料
print(f"\n載入資料: {file_path}")
# 先讀取前10萬筆測試（可根據需要調整）
df = pd.read_csv(file_path, nrows=100000)

# 轉換時間格式
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
else:
    df.index = pd.to_datetime(df.index)

# 選擇價格和成交量欄位
if 'BidClose' in df.columns:
    df['Close'] = df['BidClose']
elif 'Close' not in df.columns:
    raise ValueError("找不到價格欄位")

if 'Volume' not in df.columns:
    if 'BidVolume' in df.columns:
        df['Volume'] = df['BidVolume']
    else:
        raise ValueError("找不到成交量欄位")

print(f"✅ 資料載入成功")
print(f"   資料筆數: {len(df):,}")
print(f"   資料期間: {df.index[0]} 至 {df.index[-1]}")

# 計算報酬
df['Returns'] = df['Close'].pct_change()

# =====================================================
# 1. Roll Spread Estimate
# =====================================================
print(f"\n1️⃣ 計算 Roll Spread Estimate...")
df['Roll_Spread_30'] = roll_spread_estimate(df['Returns'], window=30)
df['Roll_Spread_60'] = roll_spread_estimate(df['Returns'], window=60)
df['Roll_Spread_240'] = roll_spread_estimate(df['Returns'], window=240)
print(f"   ✅ 完成")

# =====================================================
# 2. Amihud Illiquidity
# =====================================================
print(f"\n2️⃣ 計算 Amihud Illiquidity...")
df['Amihud_30'] = amihud_illiquidity(df['Returns'], df['Volume'], window=30)
df['Amihud_60'] = amihud_illiquidity(df['Returns'], df['Volume'], window=60)
df['Amihud_240'] = amihud_illiquidity(df['Returns'], df['Volume'], window=240)
print(f"   ✅ 完成")

# =====================================================
# 3. Kyle's λ
# =====================================================
print(f"\n3️⃣ 計算 Kyle's λ...")
print("   ⚠️ Kyle Lambda 計算較慢...")
df['Kyle_Lambda_60'] = kyle_lambda(df['Returns'], df['Volume'], window=60)
df['Kyle_Lambda_240'] = kyle_lambda(df['Returns'], df['Volume'], window=240)
print(f"   ✅ 完成")

# =====================================================
# 4. Order Flow Imbalance (OFI)
# =====================================================
print(f"\n4️⃣ 計算 Order Flow Imbalance...")
df['OFI_Basic'] = order_flow_imbalance(df['Returns'], volume=None, window=30)
df['OFI_Volume_30'] = order_flow_imbalance(df['Returns'], volume=df['Volume'], window=30)
df['OFI_Volume_60'] = order_flow_imbalance(df['Returns'], volume=df['Volume'], window=60)
print(f"   ✅ 完成")

# =====================================================
# 5. Volume Imbalance Ratio (VIR)
# =====================================================
print(f"\n5️⃣ 計算 Volume Imbalance Ratio...")
df['VIR_30'] = volume_imbalance_ratio(df['Returns'], df['Volume'], window=30)
df['VIR_60'] = volume_imbalance_ratio(df['Returns'], df['Volume'], window=60)
df['VIR_240'] = volume_imbalance_ratio(df['Returns'], df['Volume'], window=240)
print(f"   ✅ 完成")

# =====================================================
# 6. Realized Spread / Price Impact
# =====================================================
print(f"\n6️⃣ 計算 Realized Spread / Price Impact...")
df['Realized_Spread_5min'], df['Price_Impact_5min'] = realized_spread_price_impact(
    df['Close'], df['Returns'], delta_minutes=5
)
df['Realized_Spread_15min'], df['Price_Impact_15min'] = realized_spread_price_impact(
    df['Close'], df['Returns'], delta_minutes=15
)
print(f"   ✅ 完成")

# =====================================================
# 結果統計
# =====================================================
print(f"\n" + "=" * 60)
print("✅ 計算完成")
print("=" * 60)

print(f"\n新增欄位:")
print(f"  - Roll_Spread_30/60/240: Roll 價差（不同窗口）")
print(f"  - Amihud_30/60/240: Amihud 流動性缺乏指標")
print(f"  - Kyle_Lambda_60/240: Kyle Lambda（價格衝擊係數）")
print(f"  - OFI_Basic: 基本訂單流不均衡")
print(f"  - OFI_Volume_30/60: 成交量加權訂單流不均衡")
print(f"  - VIR_30/60/240: 成交量不對稱比")
print(f"  - Realized_Spread_5min/15min: 成交後價格回復幅度")
print(f"  - Price_Impact_5min/15min: 價格永久影響")

# 顯示統計資訊
print(f"\n特徵統計:")
microstructure_cols = [
    col for col in df.columns 
    if any(x in col for x in ['Roll', 'Amihud', 'Kyle', 'OFI', 'VIR', 'Realized', 'Price_Impact'])
]
for col in microstructure_cols:
    valid = df[col].notna().sum()
    if valid > 0:
        mean_val = df[col].mean()
        std_val = df[col].std()
        print(f"  {col}: 有效值 {valid:,}, 平均 {mean_val:.6f}, 標準差 {std_val:.6f}")

# 顯示最後幾筆
print(f"\n最後 5 筆資料:")
display_cols = [
    'Close', 'Volume', 'Returns', 
    'Roll_Spread_60', 'Amihud_60', 'Kyle_Lambda_60', 
    'OFI_Volume_30', 'VIR_60', 'Realized_Spread_5min'
]
available_cols = [col for col in display_cols if col in df.columns]
print(df[available_cols].tail())

# # 儲存結果
# output_file = 'NAS100_Microstructure_Features.csv'
# df.to_csv(output_file)
# print(f"\n✅ 已保存到: {output_file}")

## 步驟7：ML Model構建

In [ ]:
# =====================================================
# 步驟7：ML Model 構建與訓練（簡化版）
# =====================================================

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, 
                            roc_auc_score, precision_recall_curve, 
                            roc_curve, accuracy_score, f1_score)
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Optional, Dict, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

class ModelTrainer:
    """
    簡單的模型訓練器：支援多種模型，易於擴展
    
    功能:
    -----
    1. 支援 XGBoost 和 Random Forest
    2. 使用 PurgedKFold 交叉驗證
    3. 使用 clfHyperFit 進行超參數調優
    4. 模型評估和可視化
    """
    
    def __init__(self, model_type: str = 'xgboost'):
        """
        初始化模型訓練器
        
        參數:
        -----------
        model_type : str
            模型類型：'xgboost' 或 'rf'
        """
        self.model_type = model_type.lower()
        self.model = None
        self.best_params = None
        self.cv_scores = None
        
    def _create_model(self, **kwargs):
        """
        創建模型實例
        
        參數:
        -----------
        **kwargs : dict
            模型參數
            
        Returns:
        --------
        model : classifier
            分類器模型
        """
        if self.model_type == 'xgboost':
            default_params = {
                'random_state': 42,
                'eval_metric': 'logloss',
                'use_label_encoder': False
            }
            default_params.update(kwargs)
            return XGBClassifier(**default_params)
            
        elif self.model_type == 'rf':
            default_params = {
                'random_state': 42
            }
            default_params.update(kwargs)
            return RandomForestClassifier(**default_params)
            
        else:
            raise ValueError(f"不支援的模型類型: {self.model_type}")
    
    def _get_param_grid(self, use_randomized: bool = False) -> Dict[str, Any]:
        """
        取得參數網格或參數分布
        
        參數:
        -----------
        use_randomized : bool
            是否使用 RandomizedSearchCV
            
        Returns:
        --------
        param_grid : dict
            參數網格或參數分布
        """
        if self.model_type == 'xgboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                    # 移除 gamma, reg_alpha, reg_lambda（這些可能導致預測偏向某個類別）
                }
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1, 0.2],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                }
        
        elif self.model_type == 'rf':
            if use_randomized:
                param_grid = {
                    'n_estimators': [100, 200, 300, 500, 1000],
                    'max_depth': [5, 10, 15, 20, None],
                    'min_samples_split': logUniform(1e-3, 1e2),
                    'min_samples_leaf': logUniform(1e-3, 1e1),
                    'max_features': ['sqrt', 'log2', None],
                    'bootstrap': [True, False]
                }
            else:
                param_grid = {
                    'n_estimators': [100, 200, 300],
                    'max_depth': [5, 10, 15, 20, None],
                    'min_samples_split': [2, 5, 10],
                    'min_samples_leaf': [1, 2, 4],
                    'max_features': ['sqrt', 'log2', None]
                }
        
        return param_grid
    
    def train(self, 
              X: pd.DataFrame,
              y: pd.Series,
              t1: pd.Series,
              sample_weight: Optional[pd.Series] = None,
              cv: int = 5,
              pctEmbargo: float = 0.01,
              use_hyperopt: bool = True,
              use_randomized: bool = False,
              rndSearchIter: int = 50,
              n_jobs: int = -1,
              **model_kwargs) -> Any:
        """
        訓練模型
        
        參數:
        -----------
        X : pd.DataFrame
            特徵數據
        y : pd.Series
            標籤
        t1 : pd.Series
            標籤的結束時間（ThruDate）
        sample_weight : pd.Series, optional
            樣本權重
        cv : int
            K 折交叉驗證的折數
        pctEmbargo : float
            禁制比例
        use_hyperopt : bool
            是否使用超參數調優
        use_randomized : bool
            是否使用 RandomizedSearchCV
        rndSearchIter : int
            RandomizedSearchCV 的迭代次數
        n_jobs : int
            並行作業數
        **model_kwargs : dict
            模型額外參數
            
        Returns:
        --------
        model : classifier
            訓練好的模型
        """
        print("=" * 60)
        print(f"🚀 開始訓練 {self.model_type.upper()} 模型")
        print("=" * 60)
        
        # 準備 sample_weight
        if sample_weight is None:
            sample_weight = pd.Series(1.0, index=X.index)
        
        if use_hyperopt:
            # 使用超參數調優
            print(f"\n📊 使用 {'RandomizedSearchCV' if use_randomized else 'GridSearchCV'} 進行超參數調優...")
            
            # 創建 pipeline（用於 clfHyperFit）
            pipe_clf = MyPipeline([
                ('scaler', StandardScaler()),
                (self.model_type, self._create_model(**model_kwargs))
            ])
            
            param_grid = self._get_param_grid(use_randomized=use_randomized)
            # 添加前綴以符合 pipeline 格式
            param_grid_prefixed = {f'{self.model_type}__{k}': v for k, v in param_grid.items()}
            
            fit_params = {f'{self.model_type}__sample_weight': sample_weight}
            
            # 使用 clfHyperFit 進行超參數調優
            best_pipeline = clfHyperFit(
                feat=X,
                lbl=y,
                t1=t1,
                pipe_clf=pipe_clf,
                param_grid=param_grid_prefixed,
                cv=cv,
                bagging=[0, None, 1.],  # 不使用 bagging
                rndSearchIter=rndSearchIter if use_randomized else 0,
                n_jobs=n_jobs,
                pctEmbargo=pctEmbargo,
                **fit_params
            )
            
            # 提取實際的分類器模型
            if 'bag' in best_pipeline.named_steps:
                # 如果是 bagging pipeline
                self.model = best_pipeline.named_steps['bag']
            else:
                # 提取分類器
                self.model = best_pipeline.named_steps[self.model_type]
            
            # 提取最佳參數
            if hasattr(best_pipeline, 'best_params_'):
                self.best_params = best_pipeline.best_params_
            
            print(f"\n✅ 超參數調優完成")
            if self.best_params:
                print(f"最佳參數:")
                for key, value in self.best_params.items():
                    print(f"  {key}: {value}")
        else:
            # 不使用超參數調優，直接訓練
            print(f"\n📊 直接訓練模型（不使用超參數調優）...")
            
            # 使用 cvScore 進行交叉驗證
            inner_cv = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
            clf = self._create_model(**model_kwargs)
            
            self.cv_scores = cvScore(
                clf=clf,
                X=X,
                y=y,
                sample_weight=sample_weight,
                scoring='f1',
                cvGen=inner_cv
            )
            
            print(f"\n交叉驗證分數: {self.cv_scores}")
            print(f"平均分數: {self.cv_scores.mean():.4f} ± {self.cv_scores.std():.4f}")
            
            # 在全部數據上訓練
            self.model = clf.fit(X, y, sample_weight=sample_weight.values)
        
        print("=" * 60)
        print("✅ 模型訓練完成")
        print("=" * 60)
        
        return self.model
    
    def predict(self, X: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        進行預測
        
        參數:
        -----------
        X : pd.DataFrame
            特徵數據
            
        Returns:
        --------
        predictions : np.ndarray
            類別預測
        probabilities : np.ndarray
            概率預測
        """
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        predictions = self.model.predict(X)
        probabilities = self.model.predict_proba(X)
        
        return predictions, probabilities
    
    def evaluate(self, 
                 X: pd.DataFrame,
                 y: pd.Series,
                 sample_weight: Optional[pd.Series] = None) -> Dict[str, float]:
        """
        評估模型性能
        
        參數:
        -----------
        X : pd.DataFrame
            特徵數據
        y : pd.Series
            真實標籤
        sample_weight : pd.Series, optional
            樣本權重
            
        Returns:
        --------
        metrics : dict
            評估指標
        """
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        y_pred, y_proba = self.predict(X)
        
        metrics = {
            'accuracy': accuracy_score(y, y_pred, sample_weight=sample_weight),
            'f1': f1_score(y, y_pred, sample_weight=sample_weight, average='weighted'),
        }
        
        # 如果是二分類，計算更多指標
        if len(np.unique(y)) == 2:
            metrics['roc_auc'] = roc_auc_score(y, y_proba[:, 1], sample_weight=sample_weight)
            metrics['f1_binary'] = f1_score(y, y_pred, sample_weight=sample_weight)
        
        return metrics
    
    def plot_results(self, 
                    X: pd.DataFrame,
                    y: pd.Series,
                    sample_weight: Optional[pd.Series] = None):
        """
        繪製模型評估結果
        
        參數:
        -----------
        X : pd.DataFrame
            特徵數據
        y : pd.Series
            真實標籤
        sample_weight : pd.Series, optional
            樣本權重
        """
        if self.model is None:
            raise ValueError("請先訓練模型")
        
        y_pred, y_proba = self.predict(X)
        metrics = self.evaluate(X, y, sample_weight)
        
        # 繪製結果
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. 混淆矩陣
        cm = confusion_matrix(y, y_pred, sample_weight=sample_weight)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
        axes[0, 0].set_title('Confusion Matrix', fontsize=14)
        axes[0, 0].set_ylabel('True Label', fontsize=12)
        axes[0, 0].set_xlabel('Predicted Label', fontsize=12)
        
        # 2. ROC 曲線（如果是二分類）
        if len(np.unique(y)) == 2:
            fpr, tpr, _ = roc_curve(y, y_proba[:, 1], sample_weight=sample_weight)
            axes[0, 1].plot(fpr, tpr, linewidth=2, label=f'ROC (AUC = {metrics["roc_auc"]:.4f})')
            axes[0, 1].plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
            axes[0, 1].set_xlabel('False Positive Rate', fontsize=12)
            axes[0, 1].set_ylabel('True Positive Rate', fontsize=12)
            axes[0, 1].set_title('ROC Curve', fontsize=14)
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
        else:
            axes[0, 1].text(0.5, 0.5, 'ROC Curve\n(Only for binary classification)', 
                          ha='center', va='center', fontsize=12)
            axes[0, 1].set_title('ROC Curve', fontsize=14)
        
        # 3. Precision-Recall 曲線（如果是二分類）
        if len(np.unique(y)) == 2:
            precision, recall, _ = precision_recall_curve(y, y_proba[:, 1], sample_weight=sample_weight)
            axes[1, 0].plot(recall, precision, linewidth=2)
            axes[1, 0].set_xlabel('Recall', fontsize=12)
            axes[1, 0].set_ylabel('Precision', fontsize=12)
            axes[1, 0].set_title('Precision-Recall Curve', fontsize=14)
            axes[1, 0].grid(True, alpha=0.3)
        else:
            axes[1, 0].text(0.5, 0.5, 'Precision-Recall Curve\n(Only for binary classification)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 0].set_title('Precision-Recall Curve', fontsize=14)
        
        # 4. 評估指標摘要
        axes[1, 1].axis('off')
        metrics_text = f"""
        Model: {self.model_type.upper()}
        
        Evaluation Metrics:
        --------------------
        Accuracy: {metrics['accuracy']:.4f}
        F1 Score: {metrics['f1']:.4f}
        """
        if 'roc_auc' in metrics:
            metrics_text += f"ROC AUC: {metrics['roc_auc']:.4f}\n"
        if 'f1_binary' in metrics:
            metrics_text += f"F1 (Binary): {metrics['f1_binary']:.4f}\n"
        
        axes[1, 1].text(0.1, 0.5, metrics_text, fontsize=12, 
                        verticalalignment='center', family='monospace')
        
        plt.tight_layout()
        plt.show()
        
        # 打印詳細報告
        print("\n" + "=" * 60)
        print("📊 詳細分類報告")
        print("=" * 60)
        print(classification_report(y, y_pred, sample_weight=sample_weight))
        print("=" * 60)



## 步驟8：Embargo Purged K-Fold Class交叉驗證

In [ ]:
# =====================================================
# Purged K-Fold Cross-Validation (Snippet 7.3 & 7.4)
# =====================================================

from sklearn.model_selection._split import _BaseKFold
import numpy as np
import pandas as pd
from typing import Optional, Generator, Tuple
from sklearn.metrics import log_loss, accuracy_score

class PurgedKFold(_BaseKFold):
    """
    Extend KFold class to work with labels that span intervals
    
    The train is purged of observations overlapping test-label intervals
    Test set is assumed contiguous (shuffle=False), w/o training samples in between
    
    說明:
    -----
    這個類別擴展了 scikit-learn 的 KFold，用於處理標籤重疊的情況。
    當標籤跨越時間區間時（例如 Triple Barrier Method），測試集的標籤
    可能會與訓練集的標籤重疊，導致數據洩漏。
    
    解決方法:
    ---------
    1. Purging（清除）: 從訓練集中移除與測試集標籤區間重疊的觀測值
    2. Embargo（禁制）: 在測試集結束後，額外移除一段時間的訓練樣本
    
    參數:
    -----------
    n_splits : int
        K 折交叉驗證的折數
    t1 : pd.Series
        標籤的結束時間（ThruDate），index 必須與 X 的 index 一致
    pctEmbargo : float
        禁制比例（0-1），例如 0.01 表示禁制 1% 的數據
    """
    
    def __init__(self, n_splits: int = 3, t1: Optional[pd.Series] = None, pctEmbargo: float = 0.0):
        """
        初始化 PurgedKFold
        
        參數:
        -----------
        n_splits : int
            K 折交叉驗證的折數
        t1 : pd.Series, optional
            標籤的結束時間（ThruDate），index 必須與 X 的 index 一致
        pctEmbargo : float
            禁制比例（0-1），例如 0.01 表示禁制 1% 的數據
        """
        if t1 is not None and not isinstance(t1, pd.Series):
            raise ValueError('Label Through Dates must be a pd.Series')
        
        super(PurgedKFold, self).__init__(n_splits, shuffle=False, random_state=None)
        self.t1 = t1
        self.pctEmbargo = pctEmbargo
    
    def split(self, X: pd.DataFrame, y: Optional[pd.Series] = None, 
              groups: Optional[np.ndarray] = None) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
        """
        生成訓練/測試集的索引
        
        參數:
        -----------
        X : pd.DataFrame
            特徵數據（index 必須與 t1 的 index 一致）
        y : pd.Series, optional
            標籤（未使用，但保留以符合 scikit-learn 接口）
        groups : np.ndarray, optional
            分組（未使用，但保留以符合 scikit-learn 接口）
            
        Yields:
        -------
        train_indices : np.ndarray
            訓練集索引
        test_indices : np.ndarray
            測試集索引
        """
        if self.t1 is None:
            raise ValueError('t1 (ThruDate) must be provided')
        
        # 驗證 X 和 t1 的 index 一致
        if (X.index == self.t1.index).sum() != len(self.t1):
            raise ValueError('X and ThruDateValues must have the same index')
        
        indices = np.arange(X.shape[0])
        mbrg = int(X.shape[0] * self.pctEmbargo)  # embargo 的樣本數
        
        # 將數據分成 n_splits 個連續的測試集
        test_starts = [(i[0], i[-1] + 1) for i in 
                      np.array_split(np.arange(X.shape[0]), self.n_splits)]
        
        for i, j in test_starts:
            # 測試集的起始時間
            t0 = self.t1.index[i]  # start of test set
            test_indices = indices[i:j]
            
            # 找到測試集中最晚的結束時間
            maxT1Idx = self.t1.index.searchsorted(self.t1[test_indices].max())
            
            # 左側訓練集：結束時間 <= t0 的所有樣本
            train_indices = self.t1.index.searchsorted(
                self.t1[self.t1 <= t0].index
            )
            
            # 右側訓練集：在測試集結束後，加上 embargo 期間
            if maxT1Idx < X.shape[0]:
                # right train (with embargo)
                train_indices = np.concatenate((
                    train_indices, 
                    indices[maxT1Idx + mbrg:]
                ))
            
            yield train_indices, test_indices


def cvScore(clf, X: pd.DataFrame, y: pd.Series, 
            sample_weight: Optional[pd.Series] = None,
            scoring: str = 'f1',
            t1: Optional[pd.Series] = None,
            cv: Optional[int] = None,
            cvGen: Optional[PurgedKFold] = None,
            pctEmbargo: Optional[float] = None) -> np.ndarray:
    """
    使用 PurgedKFold 進行交叉驗證評分
    
    這個函數避免了 scikit-learn 的 cross_val_score 在處理重疊標籤時的 bug。
    
    參數:
    -----------
    clf : sklearn classifier
        分類器模型
    X : pd.DataFrame
        特徵數據
    y : pd.Series
        標籤
    sample_weight : pd.Series, optional
        樣本權重
    scoring : str
        評分方法：'neg_log_loss' 或 'accuracy'
    t1 : pd.Series, optional
        標籤的結束時間（如果 cvGen 為 None，則必須提供）
    cv : int, optional
        K 折數（如果 cvGen 為 None，則必須提供）
    cvGen : PurgedKFold, optional
        自定義的交叉驗證生成器
    pctEmbargo : float, optional
        禁制比例（如果 cvGen 為 None，則必須提供）
        
    Returns:
    --------
    score : np.ndarray
        每折的評分結果
    """
    if scoring not in ['neg_log_loss', 'accuracy']:
        raise Exception('Wrong scoring method. Must be "neg_log_loss" or "accuracy".')
    
    # 如果沒有提供 cvGen，創建一個
    if cvGen is None:
        if t1 is None or cv is None or pctEmbargo is None:
            raise ValueError('If cvGen is None, t1, cv, and pctEmbargo must be provided')
        cvGen = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
    
    # 如果沒有提供 sample_weight，使用均勻權重
    if sample_weight is None:
        sample_weight = pd.Series(1.0, index=X.index)
    
    score = []
    
    # 對每一折進行訓練和評估
    for train_idx, test_idx in cvGen.split(X=X):
        # 訓練模型
        fit = clf.fit(
            X=X.iloc[train_idx, :],
            y=y.iloc[train_idx],
            sample_weight=sample_weight.iloc[train_idx].values
        )
        
        # 根據評分方法計算分數
        if scoring == 'neg_log_loss':
            # 使用負對數損失（因為 cross_val_score 返回負值）
            prob = fit.predict_proba(X.iloc[test_idx, :])
            score_ = -log_loss(
                y.iloc[test_idx],
                prob,
                sample_weight=sample_weight.iloc[test_idx].values,
                labels=clf.classes_
            )
        else:  # accuracy
            pred = fit.predict(X.iloc[test_idx, :])
            score_ = accuracy_score(
                y.iloc[test_idx],
                pred,
                sample_weight=sample_weight.iloc[test_idx].values
            )
        
        score.append(score_)
    
    return np.array(score)


## 步驟9 ： 超參數調整

In [ ]:
# =====================================================
# Hyperparameter Tuning with Purged K-Fold CV
# Snippet 9.1, 9.2, 9.3, 9.4
# =====================================================

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import rv_continuous, kstest
import matplotlib.pyplot as plt
from typing import Optional, Dict, Any, List

# =====================================================
# Snippet 9.2: Enhanced Pipeline Class
# =====================================================

class MyPipeline(Pipeline):
    """
    Enhanced Pipeline class that handles sample_weight argument
    
    說明:
    -----
    sklearn 的 Pipeline 的 fit 方法不接受 sample_weight 參數，
    而是期望 fit_params 關鍵字參數。這是一個已知的 bug。
    這個類別擴展了 Pipeline，重寫 fit 方法以處理 sample_weight。
    
    參考:
    -----
    - GitHub issue: sklearn Pipeline sample_weight bug
    - Stackoverflow: Understanding Python super with __init__ methods
    """
    
    def fit(self, X, y, sample_weight=None, **fit_params):
        """
        重寫 fit 方法以處理 sample_weight
        
        參數:
        -----------
        X : array-like
            特徵數據
        y : array-like
            標籤
        sample_weight : array-like, optional
            樣本權重
        **fit_params : dict
            其他 fit 參數
            
        Returns:
        --------
        self : MyPipeline
            擬合後的 pipeline
        """
        if sample_weight is not None:
            # 將 sample_weight 傳遞給最後一個步驟（通常是分類器）
            last_step_name = self.steps[-1][0]
            fit_params[f'{last_step_name}__sample_weight'] = sample_weight
        
        return super(MyPipeline, self).fit(X, y, **fit_params)


# =====================================================
# Snippet 9.4: Log-Uniform Distribution
# =====================================================

class logUniform_gen(rv_continuous):
    """
    Log-uniform distribution random variable generator
    
    說明:
    -----
    隨機變數 x 在 [a, b] 區間內遵循 log-uniform 分布，
    當且僅當 log[x] ~ U[log[a], log[b]]
    
    這個分布對於探索非線性響應的參數空間特別有效，
    例如 SVC 的 C 參數和 RBF kernel 的 gamma 參數。
    
    數學定義:
    ---------
    CDF: F[x] = log[x] - log[a] / log[b] - log[a]  for a ≤ x ≤ b
    PDF: f[x] = 1 / (x * log[b/a])                 for a ≤ x ≤ b
    """
    
    def _cdf(self, x):
        """累積分布函數 (CDF)"""
        return np.log(x / self.a) / np.log(self.b / self.a)


def logUniform(a=1, b=np.exp(1)):
    """
    創建 log-uniform 分布
    
    參數:
    -----------
    a : float
        下界（必須 > 0）
    b : float
        上界（必須 > a）
        
    Returns:
    --------
    logUniform_gen : logUniform_gen
        Log-uniform 分布生成器
    """
    return logUniform_gen(a=a, b=b, name='logUniform')


# =====================================================
# Snippet 9.1 & 9.3: Hyperparameter Fitting Function
# =====================================================

def clfHyperFit(feat: pd.DataFrame, 
                lbl: pd.Series,
                t1: pd.Series,
                pipe_clf: Pipeline,
                param_grid: Dict[str, List[Any]],
                cv: int = 3,
                bagging: List[Any] = [0, None, 1.],
                rndSearchIter: int = 0,
                n_jobs: int = -1,
                pctEmbargo: float = 0.0,
                **fit_params) -> Pipeline:
    """
    使用 Purged K-Fold 交叉驗證進行超參數調優
    
    說明:
    -----
    這個函數實現了帶有 purging 和 embargo 的 GridSearchCV 或 RandomizedSearchCV。
    它避免了 scikit-learn 的 cross_val_score 在處理重疊標籤時的 bug。
    
    參數:
    -----------
    feat : pd.DataFrame
        特徵數據
    lbl : pd.Series
        標籤
    t1 : pd.Series
        標籤的結束時間（ThruDate），index 必須與 feat 的 index 一致
    pipe_clf : Pipeline
        分類器 pipeline（應使用 MyPipeline）
    param_grid : dict
        參數網格（GridSearchCV）或參數分布（RandomizedSearchCV）
        - GridSearchCV: {'param1': [val1, val2], 'param2': [val3, val4]}
        - RandomizedSearchCV: {'param1': logUniform(1e-3, 1e3), ...}
    cv : int
        K 折交叉驗證的折數
    bagging : list
        Bagging 參數 [n_estimators, max_samples, max_features]
        - n_estimators: Bagging 分類器的數量
        - max_samples: 每個分類器使用的樣本比例（None 表示不 bagging）
        - max_features: 每個分類器使用的特徵比例
    rndSearchIter : int
        如果 > 0，使用 RandomizedSearchCV，否則使用 GridSearchCV
    n_jobs : int
        並行作業數（-1 表示使用所有 CPU）
    pctEmbargo : float
        禁制比例（0-1）
    **fit_params : dict
        其他 fit 參數（例如 sample_weight）
        
    Returns:
    --------
    gs : Pipeline
        調優後的分類器 pipeline（可能包含 bagging）
        
    評分方法:
    ---------
    - Meta-labeling (標籤為 {0, 1}): 使用 'f1' score
    - 其他應用: 使用 'neg_log_loss'
    
    說明:
    -----
    1. 對於 meta-labeling，使用 'f1' 而不是 'accuracy' 或 'neg_log_loss'，
       因為在負樣本很多的情況下，預測全部為負的模型會得到高分，
       但實際上沒有學到任何有用的模式。
    2. 對於投資策略，使用 'neg_log_loss' 而不是 'accuracy'，
       因為 log loss 考慮了預測概率，更適合評估高置信度預測的表現。
    """
    
    # 固定使用 F1 作為評分標準
    scoring = 'f1'
    
    # 1) 超參數搜尋，在訓練數據上
    inner_cv = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
    
    if rndSearchIter == 0:
        gs = GridSearchCV(
            estimator=pipe_clf,
            param_grid=param_grid,
            scoring=scoring,
            cv=inner_cv,
            n_jobs=n_jobs
        )
    else:
        gs = RandomizedSearchCV(
            estimator=pipe_clf,
            param_distributions=param_grid,
            scoring=scoring,
            cv=inner_cv,
            n_jobs=n_jobs,
            n_iter=rndSearchIter
        )
    
    gs = gs.fit(feat, lbl, **fit_params).best_estimator_
    
    # 2) 在全部數據上擬合驗證後的模型
    if bagging[1] is not None and bagging[1] > 0:
        gs = BaggingClassifier(
            base_estimator=MyPipeline(gs.steps),
            n_estimators=int(bagging[0]),
            max_samples=float(bagging[1]),
            max_features=float(bagging[2]),
            n_jobs=n_jobs
        )
        
        last_step_name = gs.base_estimator.steps[-1][0]
        sample_weight_key = f'{last_step_name}__sample_weight'
        sample_weight = fit_params.get(sample_weight_key)
        
        gs = gs.fit(feat, lbl, sample_weight=sample_weight)
        gs = Pipeline([('bag', gs)])
    
    return gs


## 步驟10 ： 執行預測

In [ ]:
final_weights = final_weights_B

In [ ]:
# =====================================================
# 步驟10：PredictionPipeline - 整合步驟7、8、9
# =====================================================

import pandas as pd
import numpy as np
from typing import Optional, Dict, Any, Tuple, List
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class PredictionPipeline:
    """
    預測 Pipeline：整合模型訓練、交叉驗證、超參數調整
    
    功能:
    -----
    1. 模型訓練（步驟7）
    2. PurgedKFold 交叉驗證（步驟8）
    3. 超參數調整（步驟9）
    4. 模型預測
    5. 結果評估和可視化
    
    使用方式:
    --------
    pipeline = PredictionPipeline(
        model_type='xgboost',  # 或 'rf'
        use_hyperopt=True,     # 是否使用超參數調整
        use_cv=True            # 是否使用交叉驗證
    )
    
    # 傳入準備好的數據
    pipeline.fit(
        X=X,                    # 特徵矩陣
        y=y,                    # 標籤
        t1=t1,                  # 標籤結束時間
        sample_weight=weights   # 樣本權重（可選）
    )
    
    # 預測
    predictions, probabilities = pipeline.predict(X_test)
    """
    
    def __init__(self,
                 model_type: str = 'xgboost',
                 use_hyperopt: bool = True,
                 use_cv: bool = True,
                 cv_splits: int = 5,
                 pct_embargo: float = 0.01,
                 hyperopt_method: str = 'randomized',  # 'randomized' 或 'grid'
                 n_iter: int = 30,
                 bagging: Optional[List[Any]] = None,
                 n_jobs: int = -1):
        """
        初始化預測 Pipeline
        
        參數:
        -----------
        model_type : str
            模型類型：'xgboost' 或 'rf'
        use_hyperopt : bool
            是否使用超參數調整（預設 True）
        use_cv : bool
            是否使用交叉驗證（預設 True）
        cv_splits : int
            交叉驗證折數（預設 5）
        pct_embargo : float
            禁制比例（預設 0.01）
        hyperopt_method : str
            超參數調整方法：'randomized' 或 'grid'（預設 'randomized'）
        n_iter : int
            隨機搜尋迭代次數（預設 30）
        bagging : list, optional
            Bagging 參數 [n_estimators, max_samples, max_features]
        n_jobs : int
            並行作業數（預設 -1）
        """
        self.model_type = model_type.lower()
        self.use_hyperopt = use_hyperopt
        self.use_cv = use_cv
        self.cv_splits = cv_splits
        self.pct_embargo = pct_embargo
        self.hyperopt_method = hyperopt_method
        self.n_iter = n_iter
        self.bagging = bagging if bagging is not None else [0, None, 1.]
        self.n_jobs = n_jobs
        
        # 內部變量
        self.model = None
        self.scaler = None
        self.best_params = None
        self.cv_scores = None
        self.best_cv_scores = None
        self.X_train = None
        self.y_train = None
        self.t1_train = None
        self.sample_weight_train = None
        self.feature_columns = None
        
    def _create_pipeline(self, **model_kwargs) -> MyPipeline:
        """
        創建模型 pipeline
        
        參數:
        -----------
        **model_kwargs : dict
            模型額外參數
            
        Returns:
        --------
        pipeline : MyPipeline
            包含 scaler 和分類器的 pipeline
        """
        from sklearn.preprocessing import StandardScaler
        
        if self.model_type == 'xgboost':
            from xgboost import XGBClassifier
            default_params = {
                'random_state': 42,
                'eval_metric': 'mlogloss',  # 多分類使用 mlogloss
                'use_label_encoder': False,
                'objective': 'multi:softprob'  # 明確指定多分類目標
            }
            default_params.update(model_kwargs)
            clf = XGBClassifier(**default_params)
            
        elif self.model_type == 'rf':
            from sklearn.ensemble import RandomForestClassifier
            default_params = {
                'random_state': 42,
                'class_weight': 'balanced'  # 使用平衡的類別權重
            }
            default_params.update(model_kwargs)
            clf = RandomForestClassifier(**default_params)
        else:
            raise ValueError(f"不支援的模型類型: {self.model_type}")
        
        pipe = MyPipeline([
            ('scaler', StandardScaler()),
            (self.model_type, clf)
        ])
        
        return pipe
    
    def _get_param_grid(self, use_randomized: bool = False) -> Dict[str, Any]:
        """
        取得參數網格或參數分布
        
        參數:
        -----------
        use_randomized : bool
            是否使用 RandomizedSearchCV
            
        Returns:
        --------
        param_grid : dict
            參數網格或參數分布
        """
        if self.model_type == 'xgboost':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10, 15],
                    f'{self.model_type}__learning_rate': logUniform(1e-3, 1e-1),
                    f'{self.model_type}__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5, 7],
                    f'{self.model_type}__gamma': logUniform(1e-5, 1e-1),
                    f'{self.model_type}__reg_alpha': logUniform(1e-5, 1e-1),
                    f'{self.model_type}__reg_lambda': logUniform(1e-5, 1e-1)
                }
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [3, 5, 7, 10],
                    f'{self.model_type}__learning_rate': [0.01, 0.05, 0.1, 0.2],
                    f'{self.model_type}__subsample': [0.8, 0.9, 1.0],
                    f'{self.model_type}__colsample_bytree': [0.8, 0.9, 1.0],
                    f'{self.model_type}__min_child_weight': [1, 3, 5],
                    f'{self.model_type}__gamma': [0, 0.1, 0.2],
                    f'{self.model_type}__reg_alpha': [0, 0.01, 0.1],
                    f'{self.model_type}__reg_lambda': [1, 1.5, 2]
                }
        
        elif self.model_type == 'rf':
            if use_randomized:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300, 500, 1000],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': logUniform(1e-3, 1e2),
                    f'{self.model_type}__min_samples_leaf': logUniform(1e-3, 1e1),
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None],
                    f'{self.model_type}__bootstrap': [True, False]
                }
            else:
                param_grid = {
                    f'{self.model_type}__n_estimators': [100, 200, 300],
                    f'{self.model_type}__max_depth': [5, 10, 15, 20, None],
                    f'{self.model_type}__min_samples_split': [2, 5, 10],
                    f'{self.model_type}__min_samples_leaf': [1, 2, 4],
                    f'{self.model_type}__max_features': ['sqrt', 'log2', None]
                }
        
        return param_grid
    
    def fit(self,
            X: pd.DataFrame,
            y: pd.Series,
            t1: pd.Series,
            sample_weight: Optional[pd.Series] = None,
            **model_kwargs) -> 'PredictionPipeline':
        """
        訓練模型（整合步驟7、8、9）
        
        參數:
        -----------
        X : pd.DataFrame
            特徵矩陣
        y : pd.Series
            標籤
        t1 : pd.Series
            標籤的結束時間（ThruDate）
        sample_weight : pd.Series, optional
            樣本權重
        **model_kwargs : dict
            模型額外參數（如果不使用超參數調整）
            
        Returns:
        --------
        self : PredictionPipeline
            訓練好的 pipeline
        """
        print("=" * 60)
        print(f"🚀 PredictionPipeline 開始訓練 ({self.model_type.upper()})")
        print("=" * 60)
        
        # 保存訓練數據
        self.X_train = X.copy()
        self.y_train = y.copy()
        self.t1_train = t1.copy()
        self.sample_weight_train = sample_weight.copy() if sample_weight is not None else None
        self.feature_columns = X.columns.tolist()
        
        # 數據驗證
        common_idx = X.index.intersection(y.index).intersection(t1.index)
        if len(common_idx) < len(X.index):
            print(f"⚠️ 警告：數據索引不完全一致，使用共同索引")
            print(f"   X: {len(X.index):,}, y: {len(y.index):,}, t1: {len(t1.index):,}")
            print(f"   共同索引: {len(common_idx):,}")
            self.X_train = X.loc[common_idx]
            self.y_train = y.loc[common_idx]
            self.t1_train = t1.loc[common_idx]
            if sample_weight is not None:
                self.sample_weight_train = sample_weight.loc[common_idx]
        
        print(f"\n📊 訓練數據統計:")
        print(f"   特徵數量: {len(self.X_train.columns):,}")
        print(f"   樣本數量: {len(self.X_train):,}")
        print(f"   標籤分布: {self.y_train.value_counts().to_dict()}")
        
        # =====================================================
        # 步驟8：交叉驗證（如果啟用）
        # =====================================================
        if self.use_cv:
            print("\n" + "=" * 60)
            print("📊 步驟8：執行 PurgedKFold 交叉驗證")
            print("=" * 60)
            
            # 創建基準模型進行交叉驗證
            baseline_pipe = self._create_pipeline(**model_kwargs)
            
            # 創建 PurgedKFold
            cv = PurgedKFold(
                n_splits=self.cv_splits,
                t1=self.t1_train,
                pctEmbargo=self.pct_embargo
            )
            
            # 確定評分方法
            scoring = 'accuracy'
            
            print(f"交叉驗證配置:")
            print(f"  K 折數: {self.cv_splits}")
            print(f"  禁制比例: {self.pct_embargo:.2%}")
            print(f"  評分方法: {scoring}")
            
            # 執行交叉驗證
            self.cv_scores = cvScore(
                clf=baseline_pipe,
                X=self.X_train,
                y=self.y_train,
                sample_weight=self.sample_weight_train,
                scoring=scoring,
                cvGen=cv
            )
            
            print(f"\n✅ 基準模型交叉驗證完成")
            print(f"   分數: {self.cv_scores}")
            print(f"   平均分數: {self.cv_scores.mean():.4f} ± {self.cv_scores.std():.4f}")
            print(f"   最小分數: {self.cv_scores.min():.4f}")
            print(f"   最大分數: {self.cv_scores.max():.4f}")
        
        # =====================================================
        # 步驟9：超參數調整（如果啟用）
        # =====================================================
        if self.use_hyperopt:
            print("\n" + "=" * 60)
            print("🔧 步驟9：執行超參數調整")
            print("=" * 60)
            
            # 創建 pipeline
            pipe_clf = self._create_pipeline(**model_kwargs)
            
            # 取得參數網格
            use_randomized = (self.hyperopt_method == 'randomized')
            param_grid = self._get_param_grid(use_randomized=use_randomized)
            
            print(f"超參數調整配置:")
            print(f"  方法: {'RandomizedSearchCV' if use_randomized else 'GridSearchCV'}")
            print(f"  迭代次數: {self.n_iter if use_randomized else '全部組合'}")
            print(f"  參數數量: {len(param_grid)}")
            
            # 準備 fit_params
            fit_params = {
                f'{self.model_type}__sample_weight': self.sample_weight_train.values 
                if self.sample_weight_train is not None else None
            }
            
            print(f"\n開始時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            # 執行超參數調整
            best_pipeline = clfHyperFit(
                feat=self.X_train,
                lbl=self.y_train,
                t1=self.t1_train,
                pipe_clf=pipe_clf,
                param_grid=param_grid,
                cv=self.cv_splits,
                bagging=self.bagging,
                rndSearchIter=self.n_iter if use_randomized else 0,
                n_jobs=self.n_jobs,
                pctEmbargo=self.pct_embargo,
                **fit_params
            )
            
            print(f"結束時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            # 提取最佳模型
            if 'bag' in best_pipeline.named_steps:
                self.model = best_pipeline.named_steps['bag']
            else:
                self.model = best_pipeline.named_steps[self.model_type]
            
            # 提取 scaler
            if 'bag' in best_pipeline.named_steps:
                self.scaler = best_pipeline.named_steps['bag'].base_estimator.named_steps['scaler']
            else:
                self.scaler = best_pipeline.named_steps['scaler']
            
            # 提取最佳參數（如果可用）
            if hasattr(best_pipeline, 'best_params_'):
                self.best_params = best_pipeline.best_params_
            elif hasattr(best_pipeline, 'named_steps'):
                if self.model_type in best_pipeline.named_steps:
                    inner_model = best_pipeline.named_steps[self.model_type]
                    if hasattr(inner_model, 'best_params_'):
                        self.best_params = inner_model.best_params_
            
            print(f"\n✅ 超參數調整完成")
            if self.best_params:
                print(f"最佳參數:")
                for key, value in list(self.best_params.items())[:5]:  # 只顯示前5個
                    print(f"  {key}: {value}")
                if len(self.best_params) > 5:
                    print(f"  ... (共 {len(self.best_params)} 個參數)")
            
            # 評估最佳模型（如果啟用交叉驗證）
            if self.use_cv:
                print(f"\n評估最佳模型...")
                cv_best = PurgedKFold(
                    n_splits=self.cv_splits,
                    t1=self.t1_train,
                    pctEmbargo=self.pct_embargo
                )
                
                scoring = 'accuracy' 
                
                self.best_cv_scores = cvScore(
                    clf=self.model,
                    X=self.X_train,
                    y=self.y_train,
                    sample_weight=self.sample_weight_train,
                    scoring=scoring,
                    cvGen=cv_best
                )
                
                print(f"最佳模型交叉驗證分數:")
                print(f"   分數: {self.best_cv_scores}")
                print(f"   平均分數: {self.best_cv_scores.mean():.4f} ± {self.best_cv_scores.std():.4f}")
                
                if self.cv_scores is not None:
                    improvement = self.best_cv_scores.mean() - self.cv_scores.mean()
                    print(f"\n與基準模型比較:")
                    print(f"   基準模型: {self.cv_scores.mean():.4f}")
                    print(f"   最佳模型: {self.best_cv_scores.mean():.4f}")
                    print(f"   改進: {improvement:+.4f} ({improvement/self.cv_scores.mean()*100:+.2f}%)")
        
        else:
            # 不使用超參數調整，直接訓練
            print("\n" + "=" * 60)
            print("📊 步驟7：直接訓練模型（不使用超參數調整）")
            print("=" * 60)
            
            pipe_clf = self._create_pipeline(**model_kwargs)
            
            # 準備 fit_params
            fit_params = {
                f'{self.model_type}__sample_weight': self.sample_weight_train.values 
                if self.sample_weight_train is not None else None
            }
            
            # 訓練模型
            self.model = pipe_clf.named_steps[self.model_type]
            self.scaler = pipe_clf.named_steps['scaler']
            
            # 標準化特徵
            X_scaled = pd.DataFrame(
                self.scaler.fit_transform(self.X_train),
                index=self.X_train.index,
                columns=self.X_train.columns
            )
            
            # 訓練分類器
            self.model.fit(
                X_scaled,
                self.y_train,
                sample_weight=self.sample_weight_train.values if self.sample_weight_train is not None else None
            )
            
            print(f"✅ 模型訓練完成")
        
        print("\n" + "=" * 60)
        print("✅ PredictionPipeline 訓練完成")
        print("=" * 60)
        
        return self
    
    def predict(self, X: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        執行預測
        
        參數:
        -----------
        X : pd.DataFrame
            特徵矩陣（必須與訓練時的特徵順序一致）
            
        Returns:
        --------
        predictions : np.ndarray
            類別預測
        probabilities : np.ndarray
            概率預測
        """
        if self.model is None:
            raise ValueError("請先執行 fit() 訓練模型")
        
        print("=" * 60)
        print("🔮 執行模型預測")
        print("=" * 60)
        
        # 確保特徵順序一致
        if self.feature_columns is not None:
            missing_cols = [col for col in self.feature_columns if col not in X.columns]
            if missing_cols:
                print(f"⚠️ 警告：缺少 {len(missing_cols)} 個特徵，將填充為 0")
                for col in missing_cols:
                    X[col] = 0
            X = X[self.feature_columns]
        
        # 標準化
        X_scaled = pd.DataFrame(
            self.scaler.transform(X),
            index=X.index,
            columns=X.columns
        )
        
        # 預測
        predictions = self.model.predict(X_scaled)
        probabilities = self.model.predict_proba(X_scaled)
        
        print(f"✅ 預測完成")
        print(f"   預測樣本數: {len(predictions):,}")
        print(f"   預測類別分布:")
        unique, counts = np.unique(predictions, return_counts=True)
        for u, c in zip(unique, counts):
            print(f"     類別 {u}: {c:,} ({c/len(predictions):.2%})")
        print("=" * 60)
        
        return predictions, probabilities
    
    def evaluate(self,
                 X: pd.DataFrame,
                 y: pd.Series,
                 sample_weight: Optional[pd.Series] = None) -> Dict[str, float]:
        """
        評估模型性能
        
        參數:
        -----------
        X : pd.DataFrame
            特徵矩陣
        y : pd.Series
            真實標籤
        sample_weight : pd.Series, optional
            樣本權重
            
        Returns:
        --------
        metrics : dict
            評估指標
        """
        from sklearn.metrics import (accuracy_score, f1_score, 
                                   precision_score, recall_score,
                                   roc_auc_score)
        
        if self.model is None:
            raise ValueError("請先執行 fit() 訓練模型")
        
        print("=" * 60)
        print("📊 評估模型性能")
        print("=" * 60)
        
        # 預測
        predictions, probabilities = self.predict(X)
        
        # 計算指標
        metrics = {
            'accuracy': accuracy_score(y, predictions, sample_weight=sample_weight),
            'precision': precision_score(y, predictions, average='weighted', 
                                        sample_weight=sample_weight, zero_division=0),
            'recall': recall_score(y, predictions, average='weighted', 
                                  sample_weight=sample_weight, zero_division=0),
            'f1': f1_score(y, predictions, average='weighted', 
                          sample_weight=sample_weight, zero_division=0),
        }
        
        # 如果是二分類，計算更多指標
        if len(np.unique(y)) == 2:
            metrics['f1_binary'] = f1_score(y, predictions, sample_weight=sample_weight, zero_division=0)
            metrics['roc_auc'] = roc_auc_score(y, probabilities[:, 1], sample_weight=sample_weight)
        
        # 打印結果
        print(f"\n評估指標:")
        for key, value in metrics.items():
            print(f"  {key}: {value:.4f}")
        
        # 混淆矩陣
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y, predictions, sample_weight=sample_weight)
        print(f"\n混淆矩陣:")
        print(cm)
        
        print("=" * 60)
        
        return metrics
    
    def plot_results(self,
                    X: pd.DataFrame = None,
                    y: pd.Series = None,
                    sample_weight: Optional[pd.Series] = None):
        """
        繪製結果（交叉驗證分數、預測結果等）
        
        參數:
        -----------
        X : pd.DataFrame, optional
            特徵矩陣（用於預測和評估）
        y : pd.Series, optional
            真實標籤（用於評估）
        sample_weight : pd.Series, optional
            樣本權重
        """
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. 交叉驗證分數對比
        if self.cv_scores is not None and self.best_cv_scores is not None:
            axes[0, 0].bar(range(len(self.cv_scores)), self.cv_scores, alpha=0.6, 
                          label='Baseline Model', color='blue', width=0.4)
            axes[0, 0].bar([i + 0.4 for i in range(len(self.best_cv_scores))], 
                          self.best_cv_scores, alpha=0.6, 
                          label='Best Model (After Tuning)', color='red', width=0.4)
            axes[0, 0].axhline(self.cv_scores.mean(), color='blue', linestyle='--', 
                             label=f'Baseline Mean: {self.cv_scores.mean():.4f}', linewidth=1.5)
            axes[0, 0].axhline(self.best_cv_scores.mean(), color='red', linestyle='--', 
                             label=f'Best Mean: {self.best_cv_scores.mean():.4f}', linewidth=1.5)
            axes[0, 0].set_xlabel('Fold', fontsize=12)
            axes[0, 0].set_ylabel('Score', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores: Baseline vs Best Model', fontsize=14)
            axes[0, 0].legend()
            axes[0, 0].grid(True, alpha=0.3)
            axes[0, 0].set_xticks(range(len(self.cv_scores)))
            axes[0, 0].set_xticklabels([f'Fold {i+1}' for i in range(len(self.cv_scores))])
        elif self.cv_scores is not None:
            axes[0, 0].bar(range(len(self.cv_scores)), self.cv_scores, alpha=0.7, color='blue')
            axes[0, 0].axhline(self.cv_scores.mean(), color='red', linestyle='--', 
                             label=f'Mean: {self.cv_scores.mean():.4f}', linewidth=2)
            axes[0, 0].set_xlabel('Fold', fontsize=12)
            axes[0, 0].set_ylabel('Score', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores', fontsize=14)
            axes[0, 0].legend()
            axes[0, 0].grid(True, alpha=0.3)
        else:
            axes[0, 0].text(0.5, 0.5, 'No CV scores available', 
                          ha='center', va='center', fontsize=12)
            axes[0, 0].set_title('Cross-Validation Scores', fontsize=14)
        
        # 2. 分數分布對比
        if self.cv_scores is not None and self.best_cv_scores is not None:
            axes[0, 1].boxplot([self.cv_scores, self.best_cv_scores], 
                             labels=['Baseline', 'Best Model'])
            axes[0, 1].scatter([1] * len(self.cv_scores), self.cv_scores, 
                             alpha=0.3, s=30, color='blue')
            axes[0, 1].scatter([2] * len(self.best_cv_scores), self.best_cv_scores, 
                             alpha=0.3, s=30, color='red')
            axes[0, 1].set_ylabel('Score', fontsize=12)
            axes[0, 1].set_title('Score Distribution Comparison', fontsize=14)
            axes[0, 1].grid(True, alpha=0.3)
        elif self.cv_scores is not None:
            axes[0, 1].boxplot([self.cv_scores], labels=['Baseline'])
            axes[0, 1].scatter([1] * len(self.cv_scores), self.cv_scores, 
                             alpha=0.3, s=30, color='blue')
            axes[0, 1].set_ylabel('Score', fontsize=12)
            axes[0, 1].set_title('Score Distribution', fontsize=14)
            axes[0, 1].grid(True, alpha=0.3)
        else:
            axes[0, 1].text(0.5, 0.5, 'No CV scores available', 
                          ha='center', va='center', fontsize=12)
            axes[0, 1].set_title('Score Distribution', fontsize=14)
        
        # 3. 預測結果（如果有 X 和 y）
        if X is not None and y is not None:
            predictions, probabilities = self.predict(X)
            
            # 混淆矩陣
            from sklearn.metrics import confusion_matrix
            cm = confusion_matrix(y, predictions, sample_weight=sample_weight)
            sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', ax=axes[1, 0])  # 使用浮點數格式
            axes[1, 0].set_title('Confusion Matrix', fontsize=14)
            axes[1, 0].set_ylabel('True Label', fontsize=12)
            axes[1, 0].set_xlabel('Predicted Label', fontsize=12)
            
            # 預測分布
            pred_counts = pd.Series(predictions).value_counts().sort_index()
            axes[1, 1].bar(pred_counts.index, pred_counts.values, alpha=0.7)
            axes[1, 1].set_xlabel('Predicted Label', fontsize=12)
            axes[1, 1].set_ylabel('Count', fontsize=12)
            axes[1, 1].set_title('Prediction Distribution', fontsize=14)
            axes[1, 1].grid(True, alpha=0.3)
        else:
            axes[1, 0].text(0.5, 0.5, 'No prediction data\n(provide X and y)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 0].set_title('Confusion Matrix', fontsize=14)
            axes[1, 1].text(0.5, 0.5, 'No prediction data\n(provide X and y)', 
                          ha='center', va='center', fontsize=12)
            axes[1, 1].set_title('Prediction Distribution', fontsize=14)
        
        plt.tight_layout()
        plt.show()

In [ ]:
# =====================================================
# 步驟10：執行預測 - 修正版（修正索引錯誤）
# =====================================================

# 第0步：指定數據
y = bins['bin']
t1 = events['t1']
sample_weight = final_weights

# 準備特徵矩陣 X
event_indices = events.index
features_list = []

if 'fracdiff_features_all' in globals():
    fracdiff_selected = fracdiff_features_all.loc[event_indices]
    selected_cols = [col for col in fracdiff_selected.columns if 'NAS100' in col or 'SPX500' in col or 'US2000' in col or 'XAUUSD' in col or 'UK100' in col or 'HKG33' in col or 'SPX500' in col] 
    if len(selected_cols) > 0:
        features_list.append(fracdiff_selected[selected_cols])

strategy_features = pd.DataFrame(index=event_indices)
strategy_aligned = df_filtered.loc[event_indices]
if 'HL115%' in strategy_aligned.columns:
    strategy_features['HL115%'] = strategy_aligned['HL115%']
if '突破signal價%' in strategy_aligned.columns:
    strategy_features['突破signal價%'] = strategy_aligned['突破signal價%']
features_list.append(strategy_features)

X = pd.concat(features_list, axis=1).fillna(method='ffill').fillna(0).replace([np.inf, -np.inf], 0)

# 數據對齊
common_idx = X.index.intersection(y.index).intersection(t1.index)
X = X.loc[common_idx]
y = y.loc[common_idx]
t1 = t1.loc[common_idx]
sample_weight = sample_weight.loc[common_idx] if sample_weight is not None else None

# 標籤映射：將 -1, 0, 1 映射為 0, 1, 2
label_mapping = {-1: 0, 0: 1, 1: 2}
reverse_mapping = {0: -1, 1: 0, 2: 1}
y_mapped = y.map(label_mapping)

# 第0.5步：按指定時間切分訓練/測試集
train_start = '2015-11-01'
test_start = '2021-01-01'

X_train = X.loc[(X.index >= train_start) & (X.index < test_start)]
X_test = X.loc[X.index >= test_start]
y_train = y_mapped.loc[(y_mapped.index >= train_start) & (y_mapped.index < test_start)]
y_test = y_mapped.loc[y_mapped.index >= test_start]  # 修正：使用 y_mapped.index
t1_train = t1.loc[(t1.index >= train_start) & (t1.index < test_start)]
t1_test = t1.loc[t1.index >= test_start]
sample_weight_train = sample_weight.loc[(sample_weight.index >= train_start) & (sample_weight.index < test_start)] if sample_weight is not None else None
sample_weight_test = sample_weight.loc[sample_weight.index >= test_start] if sample_weight is not None else None

# 第1步：初始化 Pipeline
pipeline = PredictionPipeline(
    model_type='rf',
    use_hyperopt=True,
    use_cv=True,
    cv_splits=5,
    pct_embargo=0.01,
    hyperopt_method='grid',
    n_iter=3,
    n_jobs=-1
)

# 第2步：訓練模型
pipeline.fit(X_train, y_train, t1_train, sample_weight_train)

# 第3步：可視化
pipeline.plot_results(X=X_test, y=y_test, sample_weight=sample_weight_test)

# 第4步：預測
predictions_test_mapped, probabilities_test = pipeline.predict(X_test)
predictions_test = pd.Series(predictions_test_mapped).map(reverse_mapping).values

# 可視化概率分布
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].hist(probabilities_test[:, 0], bins=50, alpha=0.7, label='Class 0 (原-1)', color='red')
axes[0, 0].hist(probabilities_test[:, 1], bins=50, alpha=0.7, label='Class 1 (原0)', color='blue')
axes[0, 0].hist(probabilities_test[:, 2], bins=50, alpha=0.7, label='Class 2 (原1)', color='green')
axes[0, 0].set_xlabel('Probability', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Probability Distribution by Class', fontsize=14)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

mean_probs = probabilities_test.mean(axis=0)
axes[0, 1].bar(range(3), mean_probs, alpha=0.7, color=['red', 'blue', 'green'])
axes[0, 1].set_xticks(range(3))
axes[0, 1].set_xticklabels(['Class 0\n(原-1)', 'Class 1\n(原0)', 'Class 2\n(原1)'])
axes[0, 1].set_ylabel('Mean Probability', fontsize=12)
axes[0, 1].set_title('Mean Probability by Class', fontsize=14)
axes[0, 1].grid(True, alpha=0.3)

pred_counts = pd.Series(predictions_test_mapped).value_counts().sort_index()
axes[1, 0].bar(pred_counts.index, pred_counts.values, alpha=0.7, color=['red', 'blue', 'green'])
axes[1, 0].set_xlabel('Predicted Class', fontsize=12)
axes[1, 0].set_ylabel('Count', fontsize=12)
axes[1, 0].set_title('Predicted Class Distribution', fontsize=14)
axes[1, 0].set_xticks(range(3))
axes[1, 0].set_xticklabels(['Class 0\n(原-1)', 'Class 1\n(原0)', 'Class 2\n(原1)'])
axes[1, 0].grid(True, alpha=0.3)

true_counts = y_test.value_counts().sort_index()
axes[1, 1].bar(true_counts.index, true_counts.values, alpha=0.7, color=['red', 'blue', 'green'])
axes[1, 1].set_xlabel('True Class', fontsize=12)
axes[1, 1].set_ylabel('Count', fontsize=12)
axes[1, 1].set_title('True Class Distribution (Test Set)', fontsize=14)
axes[1, 1].set_xticks(range(3))
axes[1, 1].set_xticklabels(['Class 0\n(原-1)', 'Class 1\n(原0)', 'Class 2\n(原1)'])
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印概率統計
print("\n" + "=" * 60)
print("📊 預測概率統計")
print("=" * 60)
print(f"Class 0 (原-1) 概率: 平均={probabilities_test[:, 0].mean():.4f}, 中位數={np.median(probabilities_test[:, 0]):.4f}")
print(f"Class 1 (原0) 概率: 平均={probabilities_test[:, 1].mean():.4f}, 中位數={np.median(probabilities_test[:, 1]):.4f}")
print(f"Class 2 (原1) 概率: 平均={probabilities_test[:, 2].mean():.4f}, 中位數={np.median(probabilities_test[:, 2]):.4f}")
print("=" * 60)

# 第5步：評估
y_test_original = y.loc[y_test.index]

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

metrics_test = {
    'accuracy': accuracy_score(y_test_original, predictions_test, sample_weight=sample_weight_test),
    'precision': precision_score(y_test_original, predictions_test, average='weighted', 
                                 sample_weight=sample_weight_test, zero_division=0),
    'recall': recall_score(y_test_original, predictions_test, average='weighted', 
                          sample_weight=sample_weight_test, zero_division=0),
    'f1': f1_score(y_test_original, predictions_test, average='weighted', 
                  sample_weight=sample_weight_test, zero_division=0),
}

print("\n" + "=" * 60)
print("📊 測試集評估結果")
print("=" * 60)
print(f"評估樣本數: {len(y_test_original):,}")
for key, value in metrics_test.items():
    print(f"  {key}: {value:.4f}")

cm = confusion_matrix(y_test_original, predictions_test, sample_weight=sample_weight_test)
print(f"\n混淆矩陣:")
print(cm)
print("=" * 60)

# 第6步：創建預測結果 DataFrame
pred_df = pd.DataFrame(index=X_test.index)
pred_df['t1'] = t1_test
pred_df['side'] = events.loc[X_test.index, 'side']
pred_df['true_label'] = y_test_original
pred_df['predicted_label'] = predictions_test
pred_df['prob_class_0'] = probabilities_test[:, 0]
pred_df['prob_class_1'] = probabilities_test[:, 1]
pred_df['prob_class_2'] = probabilities_test[:, 2]
pred_df['prediction_confidence'] = np.max(probabilities_test, axis=1)
pred_df['is_correct'] = (pred_df['predicted_label'] == pred_df['true_label'])
pred_df['trade_signal'] = np.where(
    (pred_df['predicted_label'] == 1) & (pred_df['side'] == 1), 1,
    np.where((pred_df['predicted_label'] == 1) & (pred_df['side'] == -1), -1, 0)
)

pred_df